# Import

In [5]:
#path_essentials = '/Users/Setup3-PC/Desktop/qudi/logic/setup3_logic/script_logic_essentials.py'
path_essentials = 'user_scripts/Timo/from_Simon/script_logic_essentials.py'
path_toolkit = 'user_scripts/Timo/own/logic_toolkit.py'
path_curdir = 'user_scripts/Timo/own/'
import os, time, sys
import numpy as np
import copy as cp

from core.util.helpers import csv_2_list
from logic.pulsed.predefined_generate_methods.pentacene_methods import DeerAltModes
from logic.pulsed.predefined_generate_methods.multi_nv_methods import DQTAltModes, TomoRotations, TomoInit
from logic.pulsed.sampling_functions import DDMethods
# legacny nvision code
#from logic.pulsed.sampling_function_defs.sampling_functions_nvision import EnvelopeMethods as Evm
from logic.pulsed.pulse_objects import PulseEnvelopeType as Evm
from user_scripts.Timo.own.console_toolkit import Tk_file, Tk_string

# import essentials via run
exec(open(path_essentials).read())
exec(open(path_toolkit).read())
# usual module imports
sys.path.append(path_curdir)


import logging
logging.basicConfig(filename='logfile.log', filemode='w', level=logging.DEBUG)
logger = logging.getLogger(__name__)

# configure this notebook
config_matplotlib_jupyter()

memory_dict=OrderedDict()
print_welcome_msg()
0

0

[Wed Feb  1 12:36:41 2023] Hi, jupyter ready. working dir: C:\Users\Setup3-PC\Desktop\qudi


In [8]:
awg8190.get_sample_rate()

12000000000.0

In [11]:
# helps sometimes to get output from print - nevertheless need the cell run 5-10 times!
old_stdout = sys.stdout
old_stdout

# Common parameters

In [14]:
# for setups where all digitial channels needed
setup['gate_channel'] = 'a_ch2'
setup['analog_trigger_voltage'] = 0.25

In [17]:
class Common(object):
    def __init__(self):
        """
        A | B | C | D
        --------------
        | 2838.1 | 2800  | 2772  | 2733 MHz
        | 91     | 216   | 117   | 82.5 ns  (@ mw_amp= 0.15 V)
        """
        
        
        # MW parameters
        # rabi _37: C: 138.0ns, D: 162.8ns
        #self.rabi_period_rect = 116.4e-9 #allOrientations~ 82.69e-9, shaped: 
        self.rabi_period_rect = 64.0e-9 #46.89e-9   #22.40e-9
        #34.3e-9 #12.75e-9#57.2e-9#94.5e-9 # 87.0e-9 #118.1e-9 #M2 149.1e-9#760e-9#245.5e-9 #159.5e-9
        #self.rabi_shaped = {'shaped_on':True, 'period': 83.4e-9, 'env_type': Evm.parabola, 'order_P':1}
        self.rabi_shaped = {'shaped_on':False, 'period': 92.0e-9, 'env_type': Evm.parabola, 'order_P':1} #  3.5*0.25/10 # max= 0.25
        #self.rabi_shaped = {'shaped_on':False, 'period': 173.9e-9, 'env_type': Evm.parabola, 'order_P':1} #  3.5*0.25/10 # max= 0.25
        #self.rabi_shaped = {'shaped_on':False, 'period': self.rabi_period_rect, 'env_type': Evm.parabola, 'order_P':1000} 
        
        #self.rabi_shaped = {'shaped_on':True, 'period': 58.0e-9, 'env_type': Evm.parabola, 'order_P':6}
        self.mw_freq = 2729.8e6# 2803.6e6 #2.780e9 # B:2832.1e6 #3086.1e6 #3026.7e6 #3087.7e6 
        #self.mw_freq = 2850.5e6
        #nv1214.1: 2779.7 #nv1209.2: 2778.5e6 #nv1215.6:  2729.5e6#2784.1e6 #2732.0e6 # 2774.0e6# 2786.2e6 #2775.0e6#827.64e6 #M2 1503.592e6 #M15, 1507.455e6  #1507.747#7e6  #1503.336e6  #2656.5e6#NV M3 1.497129e9 #NV M2 1.503316e9 
        #nv_sA_1221.1: 
        #self.mw_freq = 2313.90e6 # only right HFS line
        self.mw_amp = 0.25 # max= 0.25
        #self.mw_amp = 0.93*0.3584*0.25 # max= 0.25

        # Readout
        # RELOAD logic_essentials and upload new seq when changing!
        self.t_laser_pulsed = 3e-6   # for pulsed mes (non NI acq)
        #self.t_laser_pulsed = 10e-6   
        #self.t_laser_pulsed = 1.25e-6   # for pulsed mes (non NI acq)
        self.t_wait_pulsed = 1e-6
        #self.t_wait_pulsed = 7e-6  # FOR DEBUGGING START TRIGGER LOSSES ONLY
        self.t_laser = None
        self.t_laser_green_init = 3e-6
        self.t_laser_red_read = 3000e-9

        # Ramsey
        self.mw_ramsey_offset = 4000e3   # Hz, detuning, 
        self.mw_ramsey_off_from_calib = 0e3 #0.1e6 #[0.66e6, 0.72e6]   # Hz, detuning,taken for checking valid

        # MFL XY8
        self.gamma_n_c13 = 1.0705e3 * 2*np.pi # 13-C
        
        # multi NV
        self.nv_order = [1,2]
        
        # charge init
        self.t_cinit_green = 500e-9
        self.t_cinit_red = 3e-3
        self.t_cinit_wait_between = 1e-6
        self.t_aom_safety = 750e-9  # as used in laser_strob. In theory should == setup['laser_delay]
    
    @property
    def rabi_period(self):

        if self.rabi_shaped['shaped_on']:
            return self.rabi_shaped['period']
        return self.rabi_period_rect
        
    @property
    def rabi_period_2(self):
        if self.rabi_shaped['shaped_on']:
            return self.t_rabi_2_shaped
        return self.t_rabi_2_rect
            
            
    @staticmethod
    def f_res_2_B_gauss(f_res):
        # only true if B well aligned
        return abs(f_res-2.87e9)/2.8e6
    
    def swap_nvs(self, order=[2,1]):
        
        if order == self.nv_order:
            return self
        if order == list(reversed(self.nv_order)):
            new_common = cp.deepcopy(self)

            new_common.nv_order = order
            new_common.rabi_period_rect = self.t_rabi_2_rect[0]
            new_common.t_rabi_2_rect = [self.rabi_period_rect]

            new_common.mw_freq = self.mw_freq_2[0]
            new_common.mw_freq_2 = [self.mw_freq]

            new_common.mw_amp = self.mw_amps_2[0]
            new_common.mw_amps_2 = [self.mw_amp]

            new_common.rabi_shaped['period'] = self.t_rabi_2_shaped[0]
            new_common.t_rabi_2_shaped = [self.rabi_shaped['period']]
            
        else:
            raise ValueError()
            
        return new_common 
        
    

    
common = Common()
b_str = f"B= {Common.f_res_2_B_gauss(common.mw_freq):.2f} G from common.mw_freq"
b_str
logger.info(b_str)

"""
# MW parameters
common.rabi_period_rect = 12.83e-9#57.2e-9#94.5e-9 # 87.0e-9 #118.1e-9 #M2 149.1e-9#760e-9#245.5e-9 #159.5e-9
common.rabi_shaped = {'shaped_on':False, 'period': 18.55-9, 'env_type': Evm.parabola, 'order_P':1}
common.mw_freq =  1507.23e6#2784.1e6 #2732.0e6 # 2774.0e6# 2786.2e6 #2775.0e6#827.64e6 #M2 1503.592e6 #M15, 1507.455e6  #1507.747#7e6  #1503.336e6  #2656.5e6#NV M3 1.497129e9 #NV M2 1.503316e9 
#common.mw_freq = 2811.9e6 # only left HFS line
common.mw_amp = 0.25

# Readout
# RELOAD logic_essentials and upload new seq when changing!
common.t_laser_pulsed = 3e-6   # for pulsed mes (non NI acq)
common.t_wait_pulsed = 1e-6
common.t_laser = None
common.t_laser_green_init = 3e-6
common.t_laser_red_read = 1000e-9

# Ramsey
common.mw_ramsey_offset = 1000e3   # Hz, detuning, 
common.mw_ramsey_off_from_calib = 0e3 #0.1e6 #[0.66e6, 0.72e6]   # Hz, detuning,taken for checking valid


# MFL XY8
gamma_n_c13 = 1.0705e3 * 2*np.pi # 13-C
"""

common.rabi_shaped

{'shaped_on': False, 'period': 9.2e-08, 'env_type': <PulseEnvelopeType.parabola: 'parabola'>, 'order_P': 1}

## for 2x NV experiments

In [20]:
common.mw_freq_2 = [2930.71e6]#[2830.7e6]
common.mw_amps_2 = [0.25] #7
#common.mw_amps_2 =  [0.85*0.7648*0.25] # [0.98*0.7648*0.25] #[0.25]
#common.t_rabi_2_rect = [110.1e-9]
common.t_rabi_2_rect = [99.78e-9]#[126.7e-9]#[59.3e-9] #[47.8e-9]
common.t_rabi_2_shaped = [144.3e-9]  ##[88.1e-9]  # p=1: [118.8e-9]
#common.t_rabi_2_shaped = common.t_rabi_2_rect # debug only


common.dd_tau = 1.0e-6
common.dd_type = DDMethods.XY8
common.dd_order = 2

common.dd_tau*common.dd_type.suborder*common.dd_order 
(common.mw_freq_2[0] - common.mw_freq)/1e6

200.91

In [23]:
common.swap_nvs([2,1]).__dict__
#common.__dict__

{'rabi_period_rect': 9.978e-08, 'rabi_shaped': {'shaped_on': False, 'period': 1.443e-07, 'env_type': <PulseEnvelopeType.parabola: 'parabola'>, 'order_P': 1}, 'mw_freq': 2930710000.0, 'mw_amp': 0.25, 't_laser_pulsed': 3e-06, 't_wait_pulsed': 1e-06, 't_laser': None, 't_laser_green_init': 3e-06, 't_laser_red_read': 3e-06, 'mw_ramsey_offset': 4000000.0, 'mw_ramsey_off_from_calib': 0.0, 'gamma_n_c13': 6726.149871335747, 'nv_order': [2, 1], 't_cinit_green': 5e-07, 't_cinit_red': 0.003, 't_cinit_wait_between': 1e-06, 't_aom_safety': 7.5e-07, 'mw_freq_2': [2729800000.0], 'mw_amps_2': [0.25], 't_rabi_2_rect': [6.4e-08], 't_rabi_2_shaped': [9.2e-08], 'dd_tau': 1e-06, 'dd_type': <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>, 'dd_order': 2}

In [26]:
len(common.dd_type.value)

8

# Library

## nb specific

In [29]:
gate_translation = np.array([(TomoRotations.ux90_on_1, 'rx', np.pi/2, '0'),
                             (TomoRotations.ux90_on_2, 'rx', np.pi/2, '1'),
                             (TomoRotations.uy90_on_1, 'ry', np.pi/2, '0'),
                             (TomoRotations.uy90_on_2, 'ry', np.pi/2, '1'),
                             (TomoRotations.ux90min_on_1, 'rx', -np.pi/2, '0'),
                             (TomoRotations.ux90min_on_2, 'rx', -np.pi/2, '1'),
                             (TomoRotations.uy90min_on_1, 'ry', -np.pi/2, '0'),
                             (TomoRotations.uy90min_on_2, 'ry', -np.pi/2, '1'),
                             (TomoRotations.ux180_on_1, 'rx', np.pi, '0'),
                             (TomoRotations.ux180_on_2, 'rx', np.pi, '1'),
                             (TomoRotations.uy180_on_1, 'ry', np.pi, '0'),
                             (TomoRotations.uy180_on_2, 'ry', np.pi, '1'),
                             (TomoRotations.ux180min_on_1, 'rx', -np.pi, '0'),
                             (TomoRotations.ux180min_on_2, 'rx', -np.pi, '1'),
                             (TomoRotations.uy180min_on_1, 'ry', -np.pi, '0'),
                             (TomoRotations.uy180min_on_2, 'ry', -np.pi, '1'),
                             (TomoRotations.c2not1, 'cx', np.pi, '0, 1')],
                             dtype=[('op_target', 'i2'),
                                    ('op_code', 'U10'), ('rotation', 'f4'), ('idx_qubit', 'U10')])

def setup_rand_benchmark_params(
                             rotations=[[TomoRotations.none]], x_max=None,
                             f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             tau_cnot=100e-9, dd_type_cnot=DDMethods.SE, dd_order=1,
                             tau_dd_fix=None, xticks_ncliff=False,
                             alternating=True, charge_init=False):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0:
        is_multi_drive = True
    
    # for awg8190 name must be < 32 chars incl ch extension
    qmeas['name'] = f'rand_benchmark'  
    qmeas['experiment'] = 'rand_benchmark'
    qmeas['savetag'] = f'rand_benchmark'  
    
    if charge_init:
        qmeas['name'] = 'rand_benchmark_fci'
        qmeas['experiment'] = 'rand_benchmark_fci'
        qmeas['sequence_mode'] = True
        qmeas = add_fci(qmeas)

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
    
    if rotations:
        if type(rotations) == list:
            qmeas['rotations'] = list_2_csv(rotations, line_delimiter=";")
        elif type(rotations) == str:  # rotations given in file
            fname = os.path.abspath(rotations)
            with open(fname) as f:
                qudi_gates = json.load(f)

            circs = parse_rbjson(qudi_gates)
            circs_qudi, n_cliff =  [], []
            for circ in circs:
                idx_info = circ['header_idx']
                skip_circ = False
                if x_max is not None:
                    skip_circ = (idx_info['n_cliff'] > x_max)
                if skip_circ:
                    continue
                
                gates = circ['gate_steps']
                rots_qudi = []
                for step in gates:
                    rots_qudi.append(gate_step_2_qudi(step))
                circs_qudi.append(rots_qudi)
                n_cliff.append(idx_info['n_cliff'])
            
            qmeas['rotations_file'] = rotations   # not used by predefined method, but safed
            qmeas['rotations'] = list_2_csv(circs_qudi, line_delimiter=";")
            if xticks_ncliff:
                qmeas['xticks'] = list_2_csv(n_cliff)
        else:
            raise ValueError("Don't understand given rotations.")
            
    else:
        # leaving will default init to "None" rotation
        pass
    
    qmeas['name'] = qmeas['name'] + '_2q' if "c2not1" in qmeas['rotations'] else qmeas['name'] + '_1q'
    
    qmeas['init_state_kwargs'] = {}   # todo, needed for cnot/ent state
    #{'dd_type': DDMethods.CPMG}
    
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix}   
    qmeas['init_state_kwargs'] = {}
    
    
    qmeas['tau_cnot'] = tau_cnot
    qmeas['dd_type_cnot'] = dd_type_cnot
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    
    if tau_cnot < 0 and tau_dd_fix == None:
        raise ValueError(f"Negative tau_cnot {tau_cnot} not allowed for cnot based on deer_dd_par!")
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'decay_exp'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas


def gate_step_2_qudi(step):
    op_code = step[0][0]
    op_params = step[0][1]
    idx_qubit = list_2_csv(step[1])
    
    # currently supported gate set 
    gate_set = ['rx', 'ry', 'cx']
    ignore_codes = ['measure', 'barrier'] 
    
    if op_code not in gate_set + ignore_codes:
        raise ValueError(f"Unsupported op_code: {op_code}")
    if op_code in ignore_codes:
        return TomoRotations.none
    try:
        if op_code == 'cx':
            rotation = np.pi
        else:
            rotation = np.float32(op_params[0])

        gt = gate_translation
    
        idx_translate = np.where((gt['op_code']==op_code) & (gt['idx_qubit']==idx_qubit) & (gt['rotation']==rotation))
        rot = gate_translation[idx_translate][0][0]
    except IndexError:
        logger.error(f"op_code {op_code}, idx {idx_qubit}")
        raise ValueError(f"Couldn't find given step in translation table: {step}")
    
    return TomoRotations(rot)
        

def parse_rbjson(data):
    # data is a list of list
    # idx 0: specify n_cliffors
    # idx 1: specify n_random
    # idx 2: info level
    # idx 3: sequence steps
    
    out_list = []
    
    for idx_cl, circ_list in enumerate(data):
        for idx_r, _ in enumerate(circ_list):
            
            out_dict = {}
            out_dict['header_raw'] = data[idx_cl][idx_r][0]
            header_str = out_dict['header_raw'].split('\n')[1].split("#")
            out_dict['header_metainfo'] = json.loads(header_str[2])
            out_dict['header_idx'] = (json.loads(header_str[1]))['idx_info']
            
            gates = data[idx_cl][idx_r][1]
            out_gates = []
            for gate in gates:
                out_gates.append(gate)
            out_dict['gate_steps'] = out_gates
            out_list.append(out_dict)
            
    return out_list

def add_fci(qmeas=None, enable=False):
    
    if qmeas is None:
        qmeas = {}
        qmeas['experiment'] = "n/a"
    
    # sequence that support (fast) charge init have '_fci' in their name
    if '_fci' in qmeas['experiment'] or enable:
        awg8190._dynamic_sequence_mode = False
        awg8190.write(':TRIG:SOUR:ADV TRIG')
        awg8190.write(':ARM:TRIG:IMP HIGH')
        awg8190.write(':ARM:TRIG:LEV 1.5')
        
        # fci sequences must have those parameters
        if qmeas['experiment']:
            # todo: strip old generate_methods that make use of kwargs
            qmeas['t_cinit_green'] = common.t_cinit_green 
            qmeas['t_cinit_red'] = common.t_cinit_red 
            qmeas['t_cinit_wait_between'] = common.t_cinit_wait_between 
            qmeas['t_aom_safety'] = common.t_aom_safety 
            
            qmeas['laser_read_ch'] = 'd_ch3'
            qmeas['add_gate_ch'] = 'd_ch4'
            qmeas['done_ch'] = 'd_ch1'
            
            # new: setting via generation parameters
            qmeas['t_laser_fci_green'] = common.t_cinit_green 
            qmeas['t_laser_fci_red'] = common.t_cinit_red 
            qmeas['t_wait_fci'] = common.t_cinit_wait_between 
            qmeas['t_safety_fci'] = common.t_aom_safety 
            
            qmeas['laser_read_red_ch'] = 'd_ch3'
            qmeas['done_fci_ch'] = 'd_ch4'
            qmeas['add_gate_ch'] = 'd_ch1'
            
    return qmeas

def setup_podmr_rfpi_params(f_res, p_scale=10, f_step=0.1e6, n_f=50,
                            f_rf=1e6, t_pi_rf=1e-6, ampl_rf=5e-3, alternating_no_rf=False):

    qmeas=OrderedDict()

    if common.rabi_shaped['shaped_on']:
        raise ValueError
    else:
        qmeas['name'] = 'pulsedodmr_rf_pi'
        qmeas['experiment'] = 'pulsedodmr_deer_pi'

    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed


    qmeas['rabi_period'] = common.rabi_period*p_scale
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    qmeas['f_mw_deer'] = f_rf
    qmeas['t_pi_deer'] = t_pi_rf
    qmeas['deer_ampl'] = ampl_rf 
    qmeas['alternating_nodeerpi'] = alternating_no_rf
    
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common.mw_amp/(p_scale)
    else:
        qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['measurement_time'] = 9999 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'

    qmeas = add_shaped(qmeas)

    return qmeas


def add_shaped(qmeas):
    if common.rabi_shaped['shaped_on']:
        # old, nvision way by passing to generate_method param
        #qmeas['env_type'] = common.rabi_shaped['env_type']
        #qmeas['order_P'] = common.rabi_shaped['order_P']
        evm = Evm(common.rabi_shaped['env_type'])
        evm.parameters['order_P'] = common.rabi_shaped['order_P']
        qmeas['pulse_envelope'] = evm
        
    else:
        # to make sure that default of generate method is overwritten
        qmeas['pulse_envelope'] = Evm.rectangle
        
    return qmeas
    

def calc_nuclear_rabi_hfs_boost(B_gauss, ms=-1, n_nuc=15):
    # Chen (2015)
    # Sangtawesin (2016)
    
    if n_nuc == 14:
        q_quadro = -4.945e6
        a_par = -2.162e6 # 14-n
        a_perp = 2.62e6
        gamma_n = -0.308e3  
    elif n_nuc == 15:
        q_quadro = 0  # spin=1/2 has no quadropolar moment
        a_par = 3.03e6
        a_perp = 3.65e6
        gamma_n = 0.432e3
        
    else:
        raise ValueError("Can only be 14-N or 15-N")
        
    gamma_e = 2.8e6
    delta = 2.87e9
    
    if ms == -1:
        # in nuc_rabi need pi init of the nv spin
        alpha = 1 + gamma_e/gamma_n* a_perp / (delta-gamma_e*B_gauss-a_par+gamma_n*B_gauss+q_quadro)
    else:
        raise ValueError("m_s can only be 0/1/-1")
    
    return abs(alpha)

def setup_ise_ramsey_dd_params(tau, n_tau, n_order_dd, f_rf, t_pi_rf, amp_rf=0.25, rf_ch='a_ch2', dd_type=DDMethods.SE,
                               t_laser=1e-6, mw_sweep_speed=3e12, f_ise_res=1e9, df_mw_sweep=30e6, amp_mw_sweep=0.25,
                               alternating=True, autosave=-1, red_readout=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'ise+ramsey+rf_dd'
    qmeas['experiment'] = 'pol_ramsey_rf_dd'
    
    if not red_readout:
        setup['laser_length'] = common.t_laser_pulsed
    else:
        qmeas['t_laser_init'] = common.t_laser_green_init
        qmeas['laser_read_ch'] = 'd_ch3'
        setup['laser_length'] = common.t_laser_red_read
    
    setup['wait_time'] = common.t_wait_pulsed
    
    qmeas['microwave_frequency'] = common.mw_freq + common.mw_ramsey_offset 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['tau'] = tau
    qmeas['n_tau'] = n_tau
    qmeas['n_order_pi_rf'] = n_order_dd
    qmeas['dd_type'] = dd_type
    qmeas['f_rf'] = f_rf
    qmeas['t_pi_rf'] = t_pi_rf
    qmeas['amp_rf'] = amp_rf
    qmeas['rf_channel'] = rf_ch
    qmeas['t_laser'] = t_laser
    qmeas['mw_sweep_speed'] = mw_sweep_speed
    qmeas['amp_mw_sweep'] = amp_mw_sweep
    qmeas['f_ise_res'] = f_ise_res
    qmeas['df_mw_sweep'] = df_mw_sweep
    
    qmeas['measurement_time'] = 99999 
    qmeas['optimize_time'] = None
    qmeas['autosave_s'] = autosave
    
    qmeas['alternating'] = alternating
    
    return qmeas

def setup_deer_dd_f_params(f_res, dd_tau=200e-9, f_step=2e6, n_f=50, t_rabi_2=None, deer_ampl=None,
                           dd_type=DDMethods.SE, dd_order=2):
    qmeas=OrderedDict()
    
    if not common.rabi_shaped['shaped_on']:
        logger.error("DEER_dd via nvision method must use pulse shaping")
        return 
    
    qmeas['experiment'] = 'DEER_DD_freq'
    qmeas['name'] = 'deer_dd_spectrum'
    
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['freq_incr'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (f_step * qmeas['num_of_points'])/2.
    qmeas['rabi_period'] = common.rabi_period
    
    qmeas['measurement_time'] = 150000 
    qmeas['optimize_time'] = None
    
    qmeas['alternating'] = True

    qmeas['tau'] = dd_tau
    if not t_rabi_2:
        qmeas['rabi_period2'] = common.rabi_period    
    else:
        qmeas['rabi_period2'] = t_rabi_2
    if deer_ampl == None:
        qmeas['mw_amp2'] = common.mw_amp
    else:
        qmeas['mw_amp2'] = deer_ampl
    
    # 1 ns safety between pi pulses
    #qmeas['tau2'] = dd_tau/2 - qmeas['rabi_period2']/4 - common.rabi_period/4 - 1e-9
    qmeas['tau2_rel_to_pi1'] = True
    qmeas['tau2'] = 0e-9
    
    qmeas['DD_type'] = dd_type
    qmeas['order'] = dd_order
    
    qmeas = add_shaped(qmeas)
    return qmeas


def setup_deer_dd_tau_params(f2, dd_tau=200e-9, tau2_start=0e-9, tau2_step=5e-9, n_tau=50,
                             t_rabi_2=None, deer_ampl=None,
                             dd_type=DDMethods.SE, dd_order=2, read_phase=0, charge_init=False):                    
    qmeas=OrderedDict()
    
    # legacny nvision deer methods
    #if common.rabi_shaped['shaped_on']:
    #    qmeas['experiment'] = 'DEER_DD_tau'
    #    qmeas['name'] = 'deer_dd_tau_shaped'
    #else:
    if dd_tau>0:
        qmeas['experiment'] = 'deer_dd_tau'
        qmeas['name'] = 'deer_dd_rect_tau'
    else:
        qmeas['experiment'] = 'deer_dd_par_tau'
        qmeas['name'] = 'deer_dd_rect_par_tau'
    
    if charge_init:
        qmeas['name'] = 'deer_dd_tau_fci'
        qmeas['experiment'] = 'deer_dd_tau_fci'
        qmeas['sequence_mode'] = True
        qmeas = add_fci(qmeas)
    
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['rabi_period'] = common.rabi_period
    
    #if common.rabi_shaped['shaped_on']:
        # old legacy nvision method
        #qmeas['tau2_rel_to_pi1'] = True
        #qmeas['DD_type'] = dd_type
        #qmeas['order'] = dd_order

        #qmeas['mw_freq2'] = f2
        #qmeas['num_of_points'] = n_tau
        #qmeas['tau2_start'] = tau2_start
        #qmeas['tau2_incr'] = tau2_step
        #qmeas['tau'] = dd_tau
        #if not t_rabi_2:
        #    qmeas['rabi_period2'] = common.rabi_period    
        #else:
        #    qmeas['rabi_period2'] = t_rabi_2
        #if deer_ampl == None:
        #    qmeas['mw_amp2'] = common.mw_amp
        #else:
        #    qmeas['mw_amp2'] = deer_ampl
        #    
        #qmeas['read_phase'] = read_phase
    #else:
    # own method and pulse shaping    
    # parallel or sequential pi pulses via qmeas['experiment']
    qmeas['dd_type'] = dd_type
    qmeas['dd_order'] = dd_order

    qmeas['num_of_points'] = n_tau
    qmeas['tau_start'] = tau2_start
    # old:
    # shifting pulses in parallel, so negative values make no sense
    # qmeas['tau_step'] = abs(tau2_step)
    qmeas['tau_step'] = tau2_step

    qmeas['f_mw_2'] = list_2_csv(f2)
    if not t_rabi_2:
        qmeas['rabi_period_mw_2'] = list_2_csv(common.rabi_period)    
    else:
        qmeas['rabi_period_mw_2'] = list_2_csv(t_rabi_2)
    if deer_ampl == None:
        qmeas['ampl_mw_2'] = list_2_csv(common.mw_amp)
    else:
        qmeas['ampl_mw_2'] = list_2_csv(deer_ampl)

    qmeas['read_phase_deg'] = read_phase

    # sequential pi pulses
    qmeas['tau1'] = dd_tau
    
    # 1 ns safety between pi pulses
    #qmeas['tau2'] = dd_tau/2 - qmeas['rabi_period2']/4 - common.rabi_period/4 - 1e-9


    
    qmeas['measurement_time'] = 150000 
    qmeas['optimize_time'] = None
    
    qmeas['alternating'] = True


    qmeas = add_shaped(qmeas)
    return qmeas

def setup_deer_f_params(f_res, he_tau=200e-9, f_step=2e6, n_f=50, two_deer_pi=True, pi_len=None, read_phases='0, 180',
                       deer_ampl=None):
    qmeas=OrderedDict()
    qmeas['name'] = 'deer_spectrum'
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed

    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    qmeas['rabi_period'] = common.rabi_period

    qmeas['measurement_time'] = 150000 
    qmeas['optimize_time'] = None
    
    qmeas['alternating'] = True
    qmeas['two_deer_pi'] = two_deer_pi
    qmeas['he_tau'] = he_tau
    if not deer_ampl:
        qmeas['deer_amp'] = common.mw_amp
    else:
        qmeas['deer_amp'] = deer_ampl
    if not pi_len:
        qmeas['pi_len'] = common.rabi_period/2
    else:
        qmeas['pi_len'] = pi_len
    
    qmeas['read_phase_degree'] = read_phases
    
    return qmeas

def setup_deer_rabi_params(deer_freq, tau_start=10e-9, tau_step=1e-6, n_tau=50, 
                      he_tau=300e-9, read_phases='0, 180', deer_ampl=None,
                      two_deer_pi=True):
    qmeas=OrderedDict()
    qmeas['name'] = 'deer_rabi'
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed

    qmeas['tau_start'] = tau_start
    qmeas['num_of_taus'] = n_tau
    qmeas['tau_step'] = tau_step

    qmeas['measurement_time'] = 99999 
    qmeas['optimize_time'] = None
    

    qmeas['he_tau'] = he_tau
    qmeas['two_deer_pi'] = two_deer_pi
    if not deer_ampl:
        qmeas['deer_amp'] = common.mw_amp
    else:
        qmeas['deer_amp'] = deer_ampl
    qmeas['deer_freq'] = deer_freq

    
    qmeas['alternating'] = True
    qmeas['read_phase_degree'] = read_phases
    
    return qmeas


def setup_deer_params(deer_freq, tau_start=10e-9, tau_step=1e-6, n_tau=50, 
                      he_tau=300e-9, second_rabi_period=None,
                      two_deer_pi=True):
    qmeas=OrderedDict()
    qmeas['name'] = 'DEER'
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed

    qmeas['tau_start'] = tau_start
    qmeas['num_of_points'] = n_tau
    qmeas['tau_step'] = tau_step

    qmeas['measurement_time'] = 99999 
    qmeas['optimize_time'] = None
    

    qmeas['he_tau'] = he_tau
    qmeas['two_deer_pi'] = two_deer_pi

    qmeas['deer_amp'] = common.mw_amp
    qmeas['deer_freq'] = deer_freq
    if not second_rabi_period:
        qmeas['second_rabi_period'] = common.rabi_period
    else:
        qmeas['second_rabi_period'] = second_rabi_period
   
    qmeas['alternating'] = True
    
    return qmeas


def setup_ramsey_deer_pi_params(f_deer, t_pi_deer, 
                                tau_start=100e-9, n_tau=1, tau_step=None,
                                alternating_mode=DeerAltModes.NVPi3Half,
                                two_deer_pi=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'Ramsey_deer_pi'
    qmeas['experiment'] = 'ramsey_deer_pi'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 50e-9
    else:
        qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    qmeas['alternating_mode'] = alternating_mode
    
    qmeas['from_list'] = False
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'sine'
     
    qmeas['f_mw_deer'] = f_deer
    qmeas['t_pi_deer'] = t_pi_deer
    qmeas['two_deer_pi'] = two_deer_pi
    
    return qmeas

def setup_podmr_deer_pi_params(f_res, f_deer, t_pi_deer, 
                               p_scale=10, p_scale_onlyperiod=1, f_step=0.1e6, n_f=50, t_pi_manual=None):
    # @param p_scale_onlyperiod: scales rabi period according to given power factor.
    #                            useful, if externally controlling poewr
    qmeas=OrderedDict()
    qmeas['name'] = 'pulsedodmr_deer_pi'
    qmeas['experiment'] = 'pulsedodmr_deer_pi'
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    if t_pi_manual:
        qmeas['rabi_period'] = 2*t_pi_manual
    else:
        qmeas['rabi_period'] = common.rabi_period*p_scale*p_scale_onlyperiod
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common.mw_amp/(p_scale)
        qmeas['deer_amplitude'] = common.mw_amp
    else:
        qmeas['microwave_amplitude'] = common.mw_amp
        qmeas['deer_amplitude'] = common.mw_amp
    qmeas['measurement_time'] = 1500 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'
    
    qmeas['f_mw_deer'] = f_deer
    qmeas['t_pi_deer'] = t_pi_deer
    
    return qmeas



def run_deer_f(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag='deer_f'
    do_experiment(experiment='deer_spectrum', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')

def run_deer(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag='deer'
    do_experiment(experiment='DEER', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')


def run_deer_rabi(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag='deer_rabi'
    do_experiment(experiment='deer_rabi', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')


    
def run_deer_pi_start(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None:
        save_tag='deer_pi_start'
    do_experiment(experiment='DEER_pi_start', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')
    
def setup_double_hh_f_params(f_start=0e6, n_f=50, f_step=1e6, spinlock_length=100e-9,
                             ampl_spinlock=0.25, ampl_dress2=0.25):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_f'
    qmeas['experiment'] = 'HH_double_fsweep'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['f_start'] = f_start
    if f_step is None:
        qmeas['f_step'] = 1e6
    else:
        qmeas['f_step'] = f_step
    qmeas['num_of_points'] = n_f
    
    qmeas['spinlock_length'] = spinlock_length
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_dress2'] = ampl_dress2
    qmeas['phase_dress2'] = 90
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_double_hh_tau_params(f_dress2=10e6, n_tau=50, tau_start=10e-9, tau_step=100e-9,
                             ampl_spinlock=0.25, ampl_dress2=0.25):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_tau'
    qmeas['experiment'] = 'HH_double_tau'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_dress2'] = ampl_dress2
    qmeas['phase_dress2'] = 90
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_double_hh_dsl_tau_params(f_dress2=10e6, n_tau=50, tau_start=10e-9, tau_step=100e-9,
                             ampl_spinlock=0.25, ampl_pi2_dress2=0.1, ampl_dress2=0.25, t_rabi_dress2=100e-9,
                                  alternating_dress2_init=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_dsl_tau'
    qmeas['experiment'] = 'HH_double_dsl_tau'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_dress2'] = ampl_dress2
    qmeas['ampl_pi2_dress2'] = ampl_pi2_dress2
    qmeas['t_rabi_dress2'] = t_rabi_dress2
    qmeas['phase_dress2'] = 90
    qmeas['alternating_pi2_dress2'] = alternating_dress2_init
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas


def setup_double_hh_ise_tau_params(f_dress2=10e6, n_tau=50, tau_start=10e-9, tau_step=100e-9,
                             ampl_spinlock=0.25, df_ise=300e3, ampl_ise=0.1, sweep_speed_ise=0.1e12,
                                  alternating_depol=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_ise_tau'
    qmeas['experiment'] = 'HH_double_ise_tau'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_ise'] = ampl_ise
    qmeas['sweep_speed_ise'] = sweep_speed_ise
    qmeas['df_ise'] = df_ise
    qmeas['alternating_pol'] = alternating_depol
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_double_hh_ise_ramsey_params(f_dress2=10e6, n_tau=50, tau_start=10e-9, tau_step=100e-9, tau_ise=10e-6,
                             ampl_spinlock=0.25, df_ise=300e3, ampl_ise=0.1, sweep_speed_ise=0.1e12,
                                  alternating_depol=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_ise_ramsey'
    qmeas['experiment'] = 'HH_double_ise_ramsey'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_ram_start'] = tau_start
    qmeas['tau_ram_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['tau_ise'] = tau_ise
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_ise'] = ampl_ise
    qmeas['sweep_speed_ise'] = sweep_speed_ise
    qmeas['df_ise'] = df_ise
    qmeas['alternating_pol'] = alternating_depol
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_double_hh_ise_dhh_f_params(f_dress2=10e6, n_tau=50, f_start=10e-9, f_step=100e-9, tau_ise=10e-6,
                             ampl_spinlock=0.25, df_ise=300e3, ampl_ise=0.1, sweep_speed_ise=0.1e12,
                             ampl_dhh_dress2=0.1, t_dhh_lock=10e-6,
                             alternating_depol=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'HH_double_ise_dhh_f'
    qmeas['experiment'] = 'HH_double_ise_dhh_f'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['f_dhh_start'] = f_start
    qmeas['f_dhh_step'] = f_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['tau_ise'] = tau_ise
    qmeas['dress2_f'] = f_dress2
    qmeas['ampl_spinlock'] = ampl_spinlock
    qmeas['ampl_ise'] = ampl_ise
    qmeas['sweep_speed_ise'] = sweep_speed_ise
    qmeas['df_ise'] = df_ise
    qmeas['alternating_pol'] = alternating_depol
    qmeas['ampl_dhh_dress2'] = ampl_dhh_dress2
    qmeas['t_dhh_lock'] = t_dhh_lock
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_hh_tau_params(tau_start=0e-6, tau_step=1e-6, n_tau=50, amp_sl=1e-3):
    qmeas=OrderedDict()
    qmeas['name'] = 'hh_tau'
    qmeas['experiment'] = 'HHtau'
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    
    qmeas['spinlock_amp'] = amp_sl
    
    qmeas['wait_time'] = common.t_wait_pulsed
    
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
 
    #qmeas = setup_penta_laser_analysis(qmeas)
    #qmeas = setup_penta_fastcounter_settings(qmeas)
    
    return qmeas

def setup_dd_mollow_n_params(tau=500e-9, dd_start=1, dd_step=1, n_dd=20, dd_type=DDMethods.SE, 
                             ampl_mol=1.5e-3, f_mol=0e6, phase_mod='0',
                             alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'dd_mollow_n'
    qmeas['name'] = f"dd_mollow_{str(dd_type).replace('DDMethods.','')}_n"
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['tau'] = tau
    qmeas['dd_order_start'] = dd_start
    qmeas['dd_step'] = dd_step
    qmeas['dd_type'] = dd_type
    qmeas['ampl_mol'] = ampl_mol
    qmeas['f_mol'] = f_mol
    qmeas['phase_mod'] = phase_mod
        
    
    qmeas['num_of_points'] = n_dd
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_dd > 1:
        qmeas['fit_experiment'] = 'sine_decay'
     
    qmeas = add_shaped(qmeas)
    
    return qmeas

def setup_rnovel_ppol_propi_params(dd_tau=500e-9, dd_order=1, dd_type=DDMethods.CPMG, 
                             ampl_mol=1.5e-3, n_pol=100, tau_ppol=0.5e-6, order_ppol=10, m_read_step=30,
                             alternating=True, shape_only_ppol=False):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'rnovel_ppol_propi'

    qmeas['name'] = f"rnovel_ppol_propi"
    qmeas['sequence_mode'] = True
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    if shape_only_ppol:
        qmeas['dd_t_rabi_rect'] = common.rabi_period_rect
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    # rnovel params
    qmeas['dd_mol_tau'] = tau
    qmeas['dd_mol_order'] = dd_order
    qmeas['dd_mol_ampl'] = ampl_mol
    qmeas['dd_mol_type'] = dd_type
    qmeas['n_pol'] = n_pol
    # propi params
    qmeas['m_read_step'] = m_read_step
    qmeas['tau_ppol'] = tau_ppol
    qmeas['order_ppol'] = order_ppol
        

    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    
    qmeas = add_shaped(qmeas)
        
    if m_read_step > 1:
        qmeas['fit_experiment'] = 'decay_exp'
 
    return qmeas

def setup_ppol_tau_params(tau_start=0.5e-6, tau_step=20e-9,
                         num_of_points=50, order=8,
                             alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'Poltau'

    qmeas['name'] = f"ppol_tau"
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['order'] = order
    
    qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if num_of_points > 1:
        qmeas['fit_experiment'] = 'lor_dip'
     
    
    qmeas = add_shaped(qmeas)
        
    return qmeas

def setup_ppol_n_params(tau=500e-9, n_start=1, n_step=1,
                         num_of_points=50, alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'Pol20_order'
    
    qmeas['name'] = f"ppol_n"
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['n_start'] = n_start
    qmeas['n_step'] = n_start
    qmeas['tau'] = tau
    
    qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if num_of_points > 1:
        qmeas['fit_experiment'] = 'sine'
    
    qmeas = add_shaped(qmeas)
 
    return qmeas

def setup_ppol_propi_params(tau=500e-9, n_start=1, n_step=1,
                         num_of_points=50, alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'Pol20_order'
    qmeas['name'] = f"ppol_n"
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['n_start'] = n_start
    qmeas['n_step'] = n_start
    qmeas['tau'] = tau
    
    qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if num_of_points > 1:
        qmeas['fit_experiment'] = 'sine'
     
    qmeas = add_shaped(qmeas)
        
    return qmeas


def setup_ppol_2x_propi_params(tau=500e-9, n_pol=100, m_read=20, order_ppol=8,
                         alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'ppol_2x_propi'
    qmeas['name'] = f"ppol_2x_propi"
    qmeas['sequence_mode'] = True
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['n_pol'] = n_pol
    qmeas['m_read_step'] = m_read
    qmeas['tau_ppol'] = tau
    qmeas['order_ppol'] = order_ppol
    
    #qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if m_read > 1:
        qmeas['fit_experiment'] = 'decay_exp'
     
    qmeas = add_shaped(qmeas)
        
    return qmeas

def setup_ise_propi_params(tau_ppol=500e-9, n_pol=100, m_read=20, order_ppol=8,
                           df_mw_sweep=3e6, t_ise=1e-6, mw_sweep_speed=1e12, amp_mw_sweep=0.1,
                           ppol_read_dir='down',
                           alternating=True):

    qmeas=OrderedDict()
    qmeas['experiment'] = 'ise_ppol_propi'
    qmeas['name'] = f"ise_ppol_propi"
    qmeas['sequence_mode'] = True
    setup['laser_length'] = common.t_laser_pulsed
    
    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['n_pol'] = n_pol
    qmeas['m_read_step'] = m_read
    qmeas['tau_ppol'] = tau_ppol
    qmeas['order_ppol'] = order_ppol
    qmeas['ppol_read_dir'] = ppol_read_dir
    
    qmeas['t_ise'] = t_ise
    qmeas['df_mw_sweep'] = df_mw_sweep
    qmeas['mw_sweep_speed'] = mw_sweep_speed
    qmeas['amp_mw_sweep'] = amp_mw_sweep
    
    
    #qmeas['num_of_points'] = num_of_points
    qmeas['alternating'] = alternating
        
    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if m_read > 1:
        qmeas['fit_experiment'] = 'decay_exp'
     
    qmeas = add_shaped(qmeas)
        
    return qmeas


def setup_dd_dqt_sigamp_params(tau=100e-9, n_amp=50, amp_step=0.01e-3,
                               t_rabi1=None, t_rabi2=None,  
                               f_mw1=[], f_mw2=[], ampl_mw2=0.0,
                               dd_type=DDMethods.XY8, dd_order=1, alternating=True):

    qmeas=OrderedDict()
    
    if not t_rabi1==None:
        qmeas['rabi_period'] = t_rabi1
    else:
        qmeas['rabi_period'] = common.rabi_period 
    
    is_multi_drive = False
    if len(f_mw2) or len(f_mw1) > 1:
        is_multi_drive = True
        
    
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        qmeas['name'] = 'dd_dqt_sigamp'
        qmeas['experiment'] = 'dd_dqt_sigamp'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
 
    qmeas['tau'] = tau
    qmeas['amp_start'] = 0e-3
    qmeas['amp_step'] = amp_step
    qmeas['dd_type'] = dd_type
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    qmeas['num_of_points'] = n_amp
    
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw2'] = list_2_csv(f_mw2)
        qmeas['t_rabi_mw2'] = t_rabi2 if t_rabi2 else t_rabi1
        qmeas['ampl_mw2'] = ampl_mw2
        if len(f_mw1) == 0:
            qmeas['microwave_frequency'] = common.mw_freq
        else:
            qmeas['microwave_frequency'] = f_mw1[0]
            qmeas['f_mw1_add'] = list_2_csv(f_mw1[1:])
            
    else:
         qmeas['microwave_frequency'] = common.mw_freq #+ 0.5*3.03e6
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas

def setup_ent_create_bell_params(tau_start=0.5e-6, tau_step=0.01e-6, n_tau=50,
                             f_mw_1=[], f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             dd_type=DDMethods.SE, dd_order=1, alternating=True):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0 or len(f_mw_1) > 1:
        is_multi_drive = True
        
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        qmeas['name'] = 'ent_create_bell'
        qmeas['experiment'] = 'ent_create_bell'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
 
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['read_phase_deg'] = 90
    qmeas['dd_type'] = dd_type
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    qmeas['num_of_points'] = n_tau

    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas

def setup_bell_hahnecho_params(tau_start=0.5e-6, tau_step=0.01e-6, n_tau=50, tau_cnot=100e-9,
                             tau_dd_fix=None,
                             f_mw_1=[], f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             dd_type=DDMethods.SE, dd_order=1, alternating=True):
   
    qmeas = setup_bell_ramsey_params(tau_start=tau_start, tau_step=tau_step, n_tau=n_tau, tau_cnot=tau_cnot,
                             tau_dd_fix=tau_dd_fix,
                             f_mw_1=f_mw_1, f_mw_2=f_mw_2, ampl_mw_2=ampl_mw_2, rabi_period_mw_2=rabi_period_mw_2,
                             dd_type=dd_type, dd_order=dd_order, alternating=alternating)
    qmeas['name'] = 'bell_hahn'
    qmeas['experiment'] = 'bell_hahnecho'
                                     
    qmeas['fit_experiment'] = 'decay_exp'
    return qmeas

def setup_bell_ramsey_params(tau_start=0.5e-6, tau_step=0.01e-6, n_tau=50, tau_cnot=100e-9,
                             tau_dd_fix=None,
                             f_mw_1=[], f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             dd_type=DDMethods.SE, dd_order=1, alternating=True):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0 or len(f_mw_1) > 1:
        is_multi_drive = True
        
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        qmeas['name'] = 'bell_ramsey'
        qmeas['experiment'] = 'bell_ramsey'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
 
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    qmeas['tau_cnot'] = tau_cnot
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix} 
        
    qmeas['dd_type'] = dd_type
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    qmeas['num_of_points'] = n_tau

    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    
  

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas

def setup_tomography_params(tau_start=0e-9, tau_step=None, n_tau=50, 
                             read_rotation=TomoRotations.none, init_state=TomoInit.none,
                             rabi_on_nv=1, rabi_phase_deg=0,
                             f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             tau_cnot=100e-9, dd_type_cnot=DDMethods.SE, dd_order=1,
                             use_shaped_cnot=False, tau_dd_fix=None,
                             alternating=True):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0:
        is_multi_drive = True
        
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        # for awg8190 name must be < 32 chars incl ch extension
        qmeas['name'] = f'tomo_rot={read_rotation.name}'  
        qmeas['experiment'] = 'tomography'
        qmeas['savetag'] = f'tomography_on{rabi_on_nv}_rot={read_rotation.name}_phi={rabi_phase_deg}'  

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period
    
    qmeas['tau_start'] = tau_start
    qmeas['tau_step'] = tau_step
    if tau_step is None:
        if rabi_on_nv == 1:
            qmeas['tau_step'] = common.rabi_period/20
        elif rabi_on_nv == 2:
            qmeas['tau_step'] = common.rabi_period_2[0]/20
        else:
            raise ValueError
        
    qmeas['rabi_on_nv'] = rabi_on_nv
    qmeas['rabi_phase_deg'] = rabi_phase_deg
    qmeas['rotation'] = read_rotation
    qmeas['init_state'] = init_state
    qmeas['init_state_kwargs'] = {}   # todo, needed for cnot/ent state
    #{'dd_type': DDMethods.CPMG}
    
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix}   
    if use_shaped_cnot:
        cnot_kwargs = {'env_type':Evm.parabola, 
                   'order_P': 1,
                   'tau_dd_fix': tau_dd_fix,
                   'rabi_period': common.rabi_shaped['period'],
                   'rabi_period_mw_2': list_2_csv(common.t_rabi_2_shaped)}
        qmeas['cnot_kwargs'] = cnot_kwargs
    
    
    qmeas['tau_cnot'] = tau_cnot
    qmeas['dd_type_cnot'] = dd_type_cnot
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    qmeas['num_of_points'] = n_tau
    
    if tau_cnot < 0 and tau_dd_fix == None:
        raise ValueError(f"Negative tau_cnot {tau_cnot} not allowed for cnot based on deer_dd_par!")
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas


def setup_tomography_single_params(
                             read_rotation=[TomoRotations.none], init_state=[TomoInit.none],
                             f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             tau_cnot=100e-9, dd_type_cnot=DDMethods.SE, dd_order=1,
                             tau_dd_fix=None,
                             alternating=True):

    qmeas=OrderedDict()

    is_multi_drive = False
    if len(f_mw_2) > 0:
        is_multi_drive = True
        
    if common.rabi_shaped['shaped_on']:
        raise ValueError("Shaping not supported")
    else:
        # for awg8190 name must be < 32 chars incl ch extension
        qmeas['name'] = f'tomography_single'  
        qmeas['experiment'] = 'tomography_single'
        qmeas['savetag'] = f'tomography_single'  

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['rabi_period'] = common.rabi_period

    qmeas['rotations'] = str(read_rotation)
    qmeas['init_states'] = str(init_state)
    qmeas['init_state_kwargs'] = {}   # todo, needed for cnot/ent state
    #{'dd_type': DDMethods.CPMG}
    
    qmeas['cnot_kwargs'] = {'tau_dd_fix': tau_dd_fix}   
    qmeas['init_state_kwargs'] = {}
    
    
    qmeas['tau_cnot'] = tau_cnot
    qmeas['dd_type_cnot'] = dd_type_cnot
    qmeas['dd_order'] = dd_order
    qmeas['alternating'] = alternating
    
    if tau_cnot < 0 and tau_dd_fix == None:
        raise ValueError(f"Negative tau_cnot {tau_cnot} not allowed for cnot based on deer_dd_par!")
    
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['rabi_period_mw_2'] = list_2_csv(rabi_period_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_mw_2)            
    

    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)

    return qmeas

0

0

## Basics

In [32]:

import pickle
from scipy.optimize import curve_fit
from scipy.stats import poisson

def toggle_pg(on=False):
    if on:
        pulsedmeasurementlogic.toggle_measurement_pause(pause=False)
    else:
        pulsedmeasurementlogic.toggle_measurement_pause(pause=True)

def enable_listfile(qmeas):
    mes_params = qmeas
    try:
        enable_listfile = False
        if 'listfile' in mes_params.keys():
            if mes_params['listfile']:
                enable_listfile = True
        if enable_listfile:
            data_dir =  os.path.abspath(savelogic.get_daily_directory())
            name = f"{mes_params['savetag']}.mpa"
            myfastcounter.change_save_mode(2)
            myfastcounter.change_filename(data_dir + r'\\' + name)
            logger.info(f"Enabling listfile, will .lst and .mpa write to: {myfastcounter.get_filename()}")
            # saving mpa done in logic_essentials.py. Need to change_save_mode(0) and save again
        else:
            # no list file
            myfastcounter.change_save_mode(0)
    except:
        logger.warning("Enabling listfile failed. Does you fastcounter support it?")
        

def setup_nuc_podmr_params(f_start=2.7e6,  f_step=5e3, n_f=50, t_nuc_rabi=50e6, rf_ampl=50e-3):
    qmeas=OrderedDict()

    if common.rabi_shaped['shaped_on']:
        raise NotImplemented
    else:
        qmeas['name'] = 'nuc_pulsedodmr'
        qmeas['experiment'] = 'nuc_pulsedodmr'

    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    qmeas['microwave_frequency'] = common.mw_freq #+ 0.5*3.03e6
    qmeas['rabi_period'] = common.rabi_period
    qmeas['freq_start'] = f_start
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    
    qmeas['rf_chnl'] = 'a_ch2'
    qmeas['rf_ampl'] = rf_ampl
    qmeas['t_nuc_rabi'] = t_nuc_rabi
    qmeas['nv_pi_init'] = True
    qmeas['alternating_no_rf'] = True
    

    qmeas['measurement_time'] = 1500 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'

    return qmeas

def setup_nuc_rabi_params(tau_step=1e-6, rf_freq=2.6e6, rf_ampl=50e-3, n_tau=50):

    qmeas=OrderedDict()

    if common.rabi_shaped['shaped_on']:
        raise NotImplemented
    else:
        qmeas['name'] = 'nuc_rabi'
        qmeas['experiment'] = 'nuc_rabi'

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['rabi_period'] = common.rabi_period

    qmeas['tau_step'] = tau_step
    qmeas['tau_start'] = 1e-6    
    qmeas['num_of_points'] = n_tau
    qmeas['microwave_frequency'] = common.mw_freq #+ 0.5*3.03e6
    qmeas['microwave_amplitude'] = common.mw_amp
    
    qmeas['rf_chnl'] = 'a_ch2'
    qmeas['rf_ampl'] = rf_ampl
    qmeas['rf_freq'] = rf_freq
    qmeas['nv_pi_init'] = True
    qmeas['alternating_no_rf'] = True
    
    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5


    return qmeas

def setup_podmr_params(f_res, p_scale=10, p_scale_onlyperiod=1, f_step=0.1e6, n_f=50, t_pi_manual=None):
    # @param p_scale_onlyperiod: scales rabi period according to given power factor.
    #                            useful, if externally controlling poewr
    qmeas=OrderedDict()

    if common.rabi_shaped['shaped_on']:
        qmeas['name'] = 'pulsedodmr_shaped'
        qmeas['experiment'] = 'pulsedodmr_shaped'
        # empirically, don't know where the scaling difference comes from
        #p_scale_onlyperiod *= 0.5
    else:
        qmeas['name'] = 'pulsedodmr'
        qmeas['experiment'] = 'pulsedodmr'

    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed

    if t_pi_manual:
        qmeas['rabi_period'] = 2*t_pi_manual
    else:
        qmeas['rabi_period'] = common.rabi_period*p_scale*p_scale_onlyperiod
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common.mw_amp/(p_scale)
    else:
        qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['measurement_time'] = 1500 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip_1x'

    qmeas = add_shaped(qmeas)

    return qmeas


def setup_fake_cwodmr_params(f_res, f_step=0.1e6, n_f=50, t_single=10e-6, mw_ampl=0.050):

    qmeas=OrderedDict()


    qmeas['name'] = 'fake_cwodmr'
    qmeas['experiment'] = 'fakecwodmr_pentacene'

    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
        
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['rabi_period'] = common.rabi_period
    qmeas['freq_step'] = f_step
    qmeas['num_of_points'] = n_f
    qmeas['freq_start'] = f_res - (qmeas['freq_step'] * qmeas['num_of_points'])/2.
    qmeas['t_single'] = t_single

    qmeas['alternating_no_mw'] = True
    qmeas['mw_ampl'] = mw_ampl
    
    qmeas['measurement_time'] = 1500 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = 'lor_dip'
    
    
    qmeas['analysis_method'] = {'method': 'mean',
                                  'signal_start': 0, 'signal_end': 0.9*t_single,
                                 }

    return qmeas

def list_2_csv(in_list, line_delimiter=";"):
    """
    :param line_delimter: if given lists of lists, will create lines per out list 
                          that are seperated by the line_delimiter
    """
    str_list = ""
    
    if type(in_list) != list:
        in_list = [in_list]
        
    for el in in_list:
        if type(el) == list:
            str_list += f"{list_2_csv(el)}{line_delimiter} " 
        else:
            str_list += f"{repr(el)}, "
        #str_list += f"{el}, "

        
    if len(str_list) > 0:
        str_list = str_list[:-2]
        
    return str_list
    

def setup_rabi_params(tau_step=None, p_scale=1, n_tau=50, leave_out_tau_idx='', 
                      f_mw1=[], f_mw2=[], ampl_mw2=0.0,
                      alternating=True, charge_init=False):

    qmeas=OrderedDict()
    
    is_multi_drive = False
    if len(f_mw2) > 0:
        is_multi_drive = True
        
    if is_multi_drive:
        qmeas['name'] = 'rabi_multi'
        qmeas['experiment'] = 'rabi_dqt_p'
    else:
        qmeas['name'] = 'rabi'
        qmeas['experiment'] = 'rabi'
    if charge_init:
        qmeas['name'] = 'rabi_fci'
        qmeas['experiment'] = 'rabi_fci'
        qmeas['sequence_mode'] = True
        qmeas = add_fci(qmeas)

    setup['laser_length'] = common.t_laser_pulsed
    qmeas['rabi_period'] = common.rabi_period

    if tau_step is None:   
        tau_step = 5e-9
    qmeas['tau_step'] = tau_step
    qmeas['tau_start'] = 0e-9
    qmeas['alternating'] = alternating
    qmeas['leave_out_tau_idx'] = leave_out_tau_idx
    if is_multi_drive:
        # currently drives 1,2 have same amplitude, respectively
        qmeas['f_mw_2'] = list_2_csv(f_mw2)
        qmeas['ampl_mw_2'] = ampl_mw2
        if alternating:
            qmeas['alternating_mode'] = DQTAltModes.DQT_12_alternating
        else:
            qmeas['alternating_mode'] = DQTAltModes.DQT_both
            
        if len(f_mw1) == 0:
            qmeas['microwave_frequency'] = common.mw_freq
        else:
            qmeas['microwave_frequency'] = f_mw1[0]
            qmeas['f_mw_1_add'] = list_2_csv(f_mw1[1:])
            
    else:
         qmeas['microwave_frequency'] = common.mw_freq #+ 0.5*3.03e6
    
    qmeas['num_of_points'] = n_tau
    if p_scale != 1:
        qmeas['microwave_amplitude'] = common.mw_amp/(p_scale)
    else:
        qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['measurement_time'] = None
    qmeas['optimize_time'] = None
    qmeas['freq_optimize_time'] = None


    qmeas['fit_experiment'] = 'sine'
    #qmeas['analysis_interval'] = 3
    #qmeas['n_sweeps'] = 5

    qmeas = add_shaped(qmeas)
    if common.rabi_shaped['shaped_on']:
        qmeas['name'] += '_shaped'

    return qmeas

def setup_ramsey_params(tau_start=100e-9, n_tau=1, tau_step=None, alternating=False, read_phases='0, 180',
                       red_readout=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'Ramsey'
    qmeas['experiment'] = 'ramsey_s3p'
    if not red_readout:
        setup['laser_length'] = common.t_laser_pulsed
    else:
        qmeas['t_laser_init'] = common.t_laser_green_init
        qmeas['laser_read_ch'] = 'd_ch3'
        setup['laser_length'] = common.t_laser_red_read

    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq + common.mw_ramsey_offset
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 50e-9
    else:
        qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    qmeas['alternating'] = alternating

    qmeas['from_list'] = False

    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'sine_decay'

    qmeas['read_phases_degree'] = read_phases

    return qmeas

def setup_hahn_params(tau_start=50e-9, n_tau=50, tau_step=0.5e-6, alternating=False):
    qmeas=OrderedDict()
    qmeas['name'] = 'hahnecho'
    qmeas['experiment'] = 'hahnecho'
    setup['laser_length'] = common.t_laser_pulsed

    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq
    qmeas['microwave_amplitude'] = common.mw_amp
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 50e-9
    else:
        qmeas['tau_step'] = tau_step
    qmeas['num_of_points'] = n_tau
    qmeas['alternating'] = alternating

    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'exp'

    return qmeas


def setup_xy8_tausweep_params(tau_start=100e-9, n_tau=20, tau_step=None, xy8_order=1, alternating=False):
    qmeas=OrderedDict()
    qmeas['experiment'] = 'xy8_tau'
    qmeas['name'] = 'xy8_tau'
    setup['laser_length'] = common.t_laser_pulsed

    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp

    qmeas['xy8_order'] = xy8_order
    qmeas['tau_start'] = tau_start
    if tau_step is None:
        qmeas['tau_step'] = 100e-9
    else:
        qmeas['tau_step'] = tau_step


    qmeas['num_of_points'] = n_tau
    qmeas['alternating'] = alternating

    qmeas['from_list'] = False

    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_tau > 1:
        qmeas['fit_experiment'] = 'Lorentz dip'

    return qmeas

def setup_xy8_nsweep_params(tau=500e-9, n_n=20, xy8_start=1, xy8_step=1, alternating=False):
    qmeas=OrderedDict()
    qmeas['experiment'] = 'xy8_nsweep'
    qmeas['name'] = 'xy8_nsweep'
    setup['laser_length'] = common.t_laser_pulsed

    qmeas['rabi_period'] = common.rabi_period
    qmeas['microwave_frequency'] = common.mw_freq 
    qmeas['microwave_amplitude'] = common.mw_amp

    qmeas['tau'] = tau
    qmeas['xy8_start'] = xy8_start
    qmeas['xy8_step'] = xy8_step



    qmeas['num_of_points'] = n_n
    qmeas['alternating'] = alternating

    qmeas['from_list'] = False

    qmeas['optimize_time'] = None
    #qmeas['freq_optimization_time'] = None
    #qmeas['analysis_interval'] = 3
    if n_n > 1:
        qmeas['fit_experiment'] = 'sine_decay'

    return qmeas

def run_generic(mes_params, save_tag=None, generate_new=True):
    laser_off()
    if save_tag is None and 'savetag' in mes_params.keys():
        save_tag = mes_params['savetag']
    if save_tag is None:
        save_tag=f"{mes_params['name']}"
    
    mes_params['savetag'] = save_tag
    
    enable_listfile(mes_params)
        
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement,
                  meas_info=add_conventional_information,
                  generate_new=generate_new, save_tag=save_tag, load_tag='')

def run_generic_autoexp(qmeas_list, generate_new=False, optimize_between_exps=False, opt_on_poi=None,
                        poi_list=None):
        
    autoexp = OrderedDict()
    automeas = OrderedDict()

    automeas['list_pois'] = poi_list if poi_list else [""]
    automeas['generate_new'] = generate_new
    automeas['optimize_between_experiments'] = optimize_between_exps
    automeas['optimize_on_poi'] = opt_on_poi

        

    for qmeas in qmeas_list:
        exp_name = qmeas['name']
        qmeas['type'] = qmeas['experiment']
        qmeas['meas_type'] = conventional_measurement
        qmeas['meas_info'] = add_conventional_information

        autoexp[exp_name] = cp.deepcopy(qmeas)
    
    do_automized_measurements(qm_dict=automeas, autoexp=autoexp)
    

def run_pdomr(mes_params, save_tag=None, generate_new=True):

    up_to = (mes_params['freq_start'] + mes_params['freq_step'] * mes_params['num_of_points'])
    laser_off()
    if save_tag is None:
        save_tag='pOdmr_{:.2f}_{:.2f}MHz'.format(mes_params['freq_start']/1e6, up_to/1e6)
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information, 
                  generate_new=generate_new, save_tag=save_tag, load_tag='')

def run_rabi(mes_params):
    laser_off()
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information,
                  generate_new=True, save_tag='Rabi_'+str(round(mes_params['microwave_amplitude']*1000))+'mV',load_tag='')

def run_xy8_tausweep(mes_params):
    laser_off()
    do_experiment(experiment='xy8_tau', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information,
                  generate_new=True, save_tag='xy8-o={}_tau_'.format(mes_params['xy8_order']),load_tag='')

def run_xy8_nsweep(mes_params):
    laser_off()
    do_experiment(experiment='xy8_nsweep', qm_dict=mes_params, meas_type=conventional_measurement, meas_info=add_conventional_information,
                  generate_new=True, save_tag='xy8_tau={}_nsweep'.format(mes_params['tau']),load_tag='')



def run_ramsey(mes_params, n_sweeps_stop=0):

    if n_sweeps_stop > 0:
        # else don't stop by sweeps
        run_ramsey_pj_params['n_sweeps'] = n_sweeps_stop

    laser_off()
    do_experiment(experiment=mes_params['experiment'], qm_dict=mes_params, meas_type=conventional_measurement, 
        meas_info=add_conventional_information, generate_new=True, save_tag='Ramsey',load_tag='')   

    if get_current_pulsed_mes_running():
        logger.warning("Mes still running after do_experiment().")
        return (None, None)

    else:
        mes = get_current_pulsed_mes()
        x = mes.signal_data[0]
        y = mes.signal_data[1]

    return (x,y)  


def create_temp_dir():
    #qudi_dir = os.getcwd()

    folder_dir = 'temp'
    try:
        os.mkdir(folder_dir)
    except FileExistsError as e:
        pass   # already exists

    return folder_dir 

def get_counts(t_avg_s=0.1):

    # stop refocusing for getting counts
    refocus_running = True
    try:
        if poimanagerlogic.time_until_refocus == -1:
            refocus_running = False
        else:
            poimanagerlogic.toggle_periodic_refocus(False)
    except NameError:
        refocus_running = False


    f_count = counterlogic.get_count_frequency()
    n_samples = int(t_avg_s * f_count)
    n_samples_max = counterlogic.get_count_length()
    if n_samples > n_samples_max:
        n_samples = n_samples_max

    # wait until refocus done for sure
    if refocus_running:
        timeout = False
        timeout_t_s = 5
        t_start = time.time()

        while(time.time() - poimanagerlogic._last_refocus < t_avg_s and not timeout):
            time.sleep(0.1)
            if time.time() - t_start > timeout_t_s:
                timeout = True

    cts = np.median(counterlogic.countdata[0][-n_samples:])    # counts per s

    if refocus_running:
        poimanagerlogic.toggle_periodic_refocus(True)

    return cts

0

0

# Laser options

## Optimize position

In [80]:
laser_on()
optimize_position()

5.153703451156616

In [87]:
opt_on_poi

In [123]:
optimize_poi(opt_on_poi, update_shift=True)

'1e-09'

## Laser On

In [71]:
#laser_on(pulser_on = False)
laser_on()
counterlogic.startCount()

0

## Laser Off

In [38]:
laser_off(pulser_on=False)
#counterl'o'gic.stopCount()


# Frequency Optimization parameters

In [ ]:
optimize_freq_dict=OrderedDict()
optimize_freq_dict['mw_optimization_method'] = 'pulsedodmr'
optimize_freq_dict['optimize_generate_new'] = False
optimize_freq_dict['save_tag'] = 'optimize_frequency'
optimize_freq_dict['optimize_fit_method'] = 'N15'
optimize_freq_dict['parameters2update'] = {'center': 'microwave_frequency'}

optimize_freq_dict['name'] = 'Pulsed_ODMR'
optimize_freq_dict['rabi_period'] = 1800e-9
optimize_freq_dict['freq_start'] = 4.127e9
optimize_freq_dict['freq_step'] = 0.2e6
optimize_freq_dict['num_of_points'] = 50
optimize_freq_dict['microwave_amplitude'] = common.mw_amp/15
optimize_freq_dict['meas_type'] = conventional_measurement
optimize_freq_dict['meas_info'] = add_conventional_information

optimize_freq_dict['measurement_time'] = 30
optimize_freq_dict['optimize_time'] = None
optimize_freq_dict['optimize_generate_new'] = True

#  Preperation mes

## cw odmr

In [27]:
f_center = 2.780e9
f_center = 2.78e9

qexp =  setup_fake_cwodmr_params(f_res=f_center, f_step=3e6, n_f=80, t_single=10e-6, mw_ampl=50e-3)

uglobals.abort.clear()
run_generic(qexp)

qexp

## Pulsed ODMR

In [50]:
p_scale_manual = 1
f_center = 2.880e9

"""
common.rabi_shaped['shaped_on'] = False
if common.rabi_shaped['shaped_on']:
    logger.warning("pODMR does not support shaped pulses yet. Using potentially wrong calibrated pi pulse.")
"""

if p_scale_manual != 1:
    logger.warning("Manually reduce AWG ampl / AMP gain by a factor of {} / {}!".format(2*p_scale_manual, np.sqrt(2*p_scale_manual)))
    #qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=50, p_scale_onlyperiod=p_scale_manual, f_step=20e3) # range ~ 1 MHz
#qmeas = setup_podmr_params(f_res=800e6, p_scale=1, f_step=4e6, n_f=100)
#qmeas = setup_podmr_params(f_res=3e9, p_scale=1, f_step=3e6, n_f=150)     # wide band, ~ 150 MHz


#common = common.swap_nvs([2,1])

#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3e6, n_f=75)
qmeas = setup_podmr_params(f_res=f_center, p_scale=2, f_step=4e6, n_f=100)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=2000e3, n_f=200)     # wide band
#qmeas = setup_podmr_params(f_res=2.87e9, p_scale=1, f_step=3000e3, n_f=200)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=100)     
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=1000e3, n_f=150)
#qmeas =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=3000e3, n_f=70)

#common.rabi_shaped['period'] = 80e-9
#qmeas = setup_podmr_params(f_res=2.77e9, p_scale=1, f_step=3000e3, n_f=100)
#qmeas = setup_podmr_params(f_res=2.88e9, p_scale=3, f_step=3000e3, n_f=200)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=500e3, n_f=75)

#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=5, f_step=1000e3, n_f=50)     # wide band, ~ 150 MHz
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=5, f_step=500e3, n_f=75)

#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=20, f_step=250e3, n_f=100) # range ~ 5 MHz
#qmeas = setup_podmr_params(f_res=2.78e9, p_scale=20, f_step=200e3, n_f=100)#, t_pi_manual=8e-6)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=30, f_step=100e3, n_f=120)

#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=40, f_step=250e3, n_f=75) # range ~ 5 MHz
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=40, f_step=100e3, n_f=100) # range ~ 3*6 MHz
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=60, f_step=50e3, n_f=100) # range ~ 1 MHz, 
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=60, f_step=100e3, n_f=150) # range ~ 1 MHz, 

# for extremely good T2* / Rabi drive efficiency
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=500, f_step=20e3, n_f=150) # range ~ 1 MHz, 
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=250, f_step=20e3, n_f=150) # range ~ 1 MHz, 
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1500, f_step=20e3, n_f=250)


#common = common.swap_nvs([1,2])
uglobals.abort.clear()
run_pdomr(qmeas)



[(('nvmin_init_ch1.bin', 'nvmin_init_ch2.bin'), {'repetitions': 5, 'event_trigger': 'OFF', 'event_jump_to': 0, 'wait_for': 'OFF', 'go_to': 0, 'ensemble': 'nvmin_init', 'flag_trigger': [], 'flag_high': []}), (('charge_read_ch1.bin', 'charge_read_ch2.bin'), {'repetitions': 0, 'event_trigger': 'OFF', 'event_jump_to': 0, 'wait_for': 'OFF', 'go_to': 0, 'ensemble': 'charge_read', 'flag_trigger': [], 'flag_high': []}), (('epoch_done_ch1.bin', 'epoch_done_ch2.bin'), {'repetitions': 0, 'event_trigger': 'OFF', 'event_jump_to': 0, 'wait_for': 'OFF', 'go_to': 0, 'ensemble': 'epoch_done', 'flag_trigger': [], 'flag_high': []})]

### Fake cw odmr

In [41]:
f_center = common.mw_freq
f_center = 2.88e9

try:
    qmeas = setup_fake_cwodmr_params(f_res=f_center, f_step=3e6, n_f=70, t_single=10e-6, mw_ampl=50e-3)
    #qmeas = setup_fake_cwodmr_params(f_res=f_center, f_step=3e6, n_f=70, t_single=5e-6, mw_ampl=100e-3)
except:
    logger.exception("")
    
uglobals.abort.clear()
run_generic(qmeas)

qmeas

OrderedDict([('name', 'fake_cwodmr'), ('experiment', 'fakecwodmr_pentacene'), ('microwave_amplitude', 0.25), ('rabi_period', 6.4e-08), ('freq_step', 3000000.0), ('num_of_points', 70), ('freq_start', 2775000000.0), ('t_single', 1e-05), ('alternating_no_mw', True), ('mw_ampl', 0.05), ('measurement_time', 1500), ('optimize_time', None), ('fit_experiment', 'lor_dip'), ('analysis_method', {'method': 'mean', 'signal_start': 0.0, 'signal_end': 8.9984e-06}), ('savetag', 'fake_cwodmr'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 140), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', ''), ('gate_channel', 'a_ch2'), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 1.7e-07), ('channel_amp', 1.0), ('microwave_

### fine pODMR on 2x NVs

In [727]:
uglobals.abort.clear()

qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=2, f_step=0.8e6, n_f=70)
#qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=0.8e6, n_f=70)
qmeas['measurement_time'] = 1.5*60
qmeas['name'] = 'podmr_fine_A'

run_generic(qmeas)


#qmeas = setup_podmr_params(f_res=common.mw_freq_2[0], p_scale=3, f_step=1.0e6, n_f=100)
qmeas = setup_podmr_params(f_res=common.mw_freq_2[0], p_scale=2, f_step=0.8e6, n_f=70)
#qmeas = setup_podmr_params(f_res=common.mw_freq_2[0], p_scale=1, f_step=0.8e6, n_f=70)
qmeas['measurement_time'] = 1.5*60
qmeas['rabi_period'] = common.rabi_period_2[0] /(qmeas['microwave_amplitude']/common.mw_amp)
qmeas['microwave_amplitude'] = common.mw_amps_2[0] *qmeas['microwave_amplitude']/common.mw_amp
qmeas['name'] = 'podmr_fine_B'

#run_generic(qmeas)
#qmeas

In [235]:
qmeas

OrderedDict([('name', 'podmr_fine_A'), ('experiment', 'pulsedodmr'), ('rabi_period', 1.76e-07), ('freq_step', 800000.0), ('num_of_points', 70), ('freq_start', 2701300000.0), ('microwave_amplitude', 0.125), ('measurement_time', 90.0), ('optimize_time', None), ('fit_experiment', 'lor_dip'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>)])

### power series

In [363]:
uglobals.abort.clear()
pscales = np.linspace(1, 25, 10)[::]
common.mw_amp = 0.25
p_start = 1

for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        p_phys = p_start*p
        qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=p_phys, f_step=500e3, n_f=75)

        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 40
        savetag = f"podmr_idx={i}_pscale={p_phys:.2f}"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

In [361]:
pscales

array([1.        , 0.91111111, 0.82222222, 0.73333333, 0.64444444,
       0.55555556, 0.46666667, 0.37777778, 0.28888889, 0.2       ])

### pODMR + pi RF

In [29]:
uglobals.abort.clear()
qmeas = setup_podmr_rfpi_params(f_res=common.mw_freq, p_scale=60, f_step=100e3, n_f=20,
                            f_rf=366.82e3, t_pi_rf=4.554e-6, ampl_rf=0.125, alternating_no_rf=True)
run_generic(qmeas)

In [ ]:
np.linspace(0.5, 1.5, 9)[::]

### t_pi series

In [ ]:
uglobals.abort.clear()
tscales = np.linspace(0.2, 1.8, 9)[::]
t_start = 4.554e-6

for i, t in enumerate(tscales):
    if uglobals.abort.is_set():
        break
    try:
        t_phys = t_start*t
        qmeas = setup_podmr_rfpi_params(f_res=common.mw_freq, p_scale=60, f_step=100e3, n_f=20,
                                    f_rf=366.82e3, t_pi_rf=4.554e-6, ampl_rf=0.125, alternating_no_rf=True)
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 10*60
        savetag = f"podmr_rfpi_idx={i}_tpi={1e6*t_phys:.2f}uV"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

## Rabi

In [77]:
uglobals.abort.clear()
#uglobals.abort.set()

#qmeas = setup_rabi_params(tau_step=500*1e-9, p_scale=250, alternating=False, n_tau=200)   # 10e-9
#qmeas = setup_rabi_params(tau_step=25*1e-9, p_scale=25, alternating=False)   # 10e-9
#qmeas = setup_rabi_params(tau_step=1e-9, p_scale=1, alternating=False)   # 10e-9

#qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=40, alternating=True) 
#qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=75, alternating=True) 

"""
leave_out_idx = np.asarray([np.arange(20,100), np.arange(120,200), np.arange(220,300)]).flatten() 
qmeas = setup_rabi_params(tau_step=2.5e-9*2, n_tau=320, p_scale=2, leave_out_tau_idx=np.array2string(leave_out_idx,
                                                                                                      separator=',',
                                                                                                      max_line_width=9999))   # 10e-9
"""

qmeas = setup_rabi_params(tau_step=common.rabi_period/15, n_tau=50, alternating=False) 
#qmeas = setup_rabi_params(tau_step=common.rabi_period/5, n_tau=20, alternating=False) 

# on nv2
#qmeas = setup_rabi_params(tau_step=common.t_rabi_2[0]/20, n_tau=75, alternating=False) 
#qmeas['microwave_frequency'] = common.mw_freq_2[0]
#qmeas['name'] = "rabi_B"



#qmeas['optimize_time'] = 30
#qmeas['optimize_func_toggle_pause'] = toggle_pg

# de
#env = Evm.parabola
#env.parameters['order_P'] = 2
#qmeas['pulse_envelope'] = env

#run_rabi(qmeas)

qmeas['env_type'].parameters


{'order_P': 2}

In [128]:

env.parameters

{'order_P': 2}

### on 2x NVs

In [243]:
uglobals.abort.clear()

qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=75, alternating=False) 
qmeas['name'] += "_A"
qmeas['measurement_time'] = 1.5*60
run_generic(qmeas)

# on nv2
qmeas = setup_rabi_params(tau_step=common.rabi_period_2[0]/20, n_tau=75, alternating=False) 
qmeas['microwave_frequency'] = common.mw_freq_2[0]
qmeas['microwave_amplitude'] = common.mw_amps_2[0]
qmeas['rabi_period'] = common.rabi_period_2[0]
qmeas['name'] += "_B"
qmeas['measurement_time'] = 1.5*60

run_generic(qmeas)
qmeas

OrderedDict([('name', 'rabi_B'), ('experiment', 'rabi'), ('rabi_period', 9.87e-08), ('tau_step', 4.935e-09), ('tau_start', 0.0), ('alternating', False), ('leave_out_tau_idx', ''), ('microwave_frequency', 2930710000.0), ('num_of_points', 75), ('microwave_amplitude', 0.25), ('measurement_time', 90.0), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('pulse_envelope', <PulseEnvelopeType.rectangle: 'rectangle'>), ('savetag', 'rabi_B'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 75), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', ''), ('gate_channel', 'a_ch2'), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 1.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', '/

In [167]:
common.rabi_period_2[0]/20
common.rabi_shaped['shaped_on']

True

### drive on mutliple transitions

In [309]:
# drive all HFS lines
common.mw_freq = 2735.99e6
common.mw_amp = 0.25/2.2

ampl_mw2 = 0.25/1.8#[common.mw_amp]

f_mw1_add = [2725.11e6, 2729.41e6]
f_mw1 = sorted([common.mw_freq] + f_mw1_add)
f_mw2 = [2946.45e6, 2948.6e6, 2950.75e6]

# drive in left, right middle between 3 HFS lines
f_mw2 = [np.mean(f_mw2[0:2]), np.mean(f_mw2[1:])]
f_mw1 = [np.mean(f_mw1[0:2]), np.mean(f_mw1[1:])]



In [383]:
common.mw_freq = 2735.99e6
common.mw_amp = 0.25/(2*1.3)

mw_freq_nv2 = 2820.47e6
mw_amp_nv2 = 0.25/1.6
t_rabi_nv2 = 162.1e-9#common.rabi_period

ampl_mw2 = mw_amp_nv2#[common.mw_amp]
f_mw2 = [mw_freq_nv2]
f_mw1 = [common.mw_freq]

mw_amp_nv2 + common.mw_amp

0.25240384615384615

In [385]:
# for calibration, drive transition 1,2 alternating
qmeas = setup_rabi_params(tau_step=1.5*0.5*common.rabi_period/20, n_tau=75,
                          f_mw2=f_mw2, f_mw1=f_mw1, ampl_mw2=ampl_mw2,
                          alternating=True)

# do DQT
#qmeas = setup_rabi_params(tau_step=3*common.rabi_period/20, n_tau=75,
#                          f_mw2=f_mw2, ampl_mw2=ampl_mw2,
#                          alternating=False)

uglobals.abort.clear()
run_rabi(qmeas)
#qmeas

In [335]:
f_mw1

[2727260000.0]

In [35]:

uglobals.abort.clear()
pscales = np.linspace(1, 0.05, 75)#[::-1]


common.mw_freq = 2727.26e6
common.mw_amp = 0.25/1.8

f_mw1_add = [2725.11e6, 2729.41e6]
f_mw2 = [2946.45e6, 2948.6e6, 2950.75e6]
ampl_mw2 = 0.25#[common.mw_amp]

p_start_1 = common.mw_amp
p_start_2 = ampl_mw2


for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        p_phys_1 = p_start_1*p
        common.mw_amp = p_phys_1
        p_phys_2 = p_start_2*p
        
        #qmeas = setup_rabi_params(tau_step=2*common.rabi_period/20, n_tau=75,
        #                  f_mw2=f_mw2, ampl_mw2=p_phys_2,
        #                  alternating=False)
        
        qmeas = setup_rabi_params(tau_step=2*common.rabi_period/20, n_tau=200,
                           f_mw2=f_mw2, ampl_mw2=p_phys_2, f_mw1_add=f_mw1_add,
                           alternating=True)
        
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 300
        qmeas['fit_experiment'] = "sine_2x"
        savetag = f"rabi_multi_idx={i}_ampl1={1e3*p_phys_1:.2f}mV_ampl2={1e3*p_phys_2:.2f}mV"
        run_generic(qmeas, save_tag=savetag)
        
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

In [ ]:

uglobals.abort.clear()
dfs = np.linspace(-5e6, 5e6, 20)#[::-1]
p_start = common.mw_amp

for i, df in enumerate(dfs):
    if uglobals.abort.is_set():
        break
    try:
        f2 = f_mw2[0] + df
        qmeas = setup_rabi_params(tau_step=2*common.rabi_period/20, n_tau=75,
                          f_mw2=[f2], ampl_mw2=ampl_mw2,
                          alternating=False)
        del(qmeas['alternating_mode'])
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 60
        qmeas['fit_experiment'] = "sine_2x"
        savetag = f"rabi_multi_idx={i}_df2={1e-3*df:.2f}MHz"
        run_generic(qmeas, save_tag=savetag)
        
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

### power series

In [263]:
uglobals.abort.clear()
pscales = np.linspace(1, 0.05, 8)[::-1]
p_start = common.mw_amp

for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        p_phys = p_start*p
        qmeas = setup_rabi_params(tau_step=common.rabi_period/20*common.mw_amp/p_phys,
                                  n_tau=75, p_scale=common.mw_amp/p_phys)   # 10e-9
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 15
        savetag = f"rabi_idx={i}_ampl={1e3*p_phys:.2f}mV"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

In [47]:
add_shaped(qmeas)

OrderedDict([('name', 'rabi_shaped'), ('experiment', 'rabi_shaped'), ('rabi_period', 7.42e-08), ('tau_step', 3.7100000000000002e-09), ('tau_start', 0.0), ('alternating', False), ('leave_out_tau_idx', ''), ('num_of_points', 75), ('microwave_frequency', 2766530000.0), ('microwave_amplitude', 0.15), ('measurement_time', None), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.parabola: 'SinEnvelopeParabolaN'>), ('order_P', 1)])

### charge-init Rabi

In [59]:


toggle_pg

<function toggle_pg at 0x000000EB4558DB70>

In [534]:
uglobals.abort.clear()

qmeas = setup_rabi_params(tau_step=common.rabi_period/5, n_tau=20, alternating=False, charge_init=True) 
qmeas['name'] = "rabi_A"




qmeas['optimize_time'] = 20*60
qmeas['optimize_func_toggle_pause'] = toggle_pg


#qmeas['generate_new'] = False
run_generic(qmeas)

qmeas


OrderedDict([('name', 'rabi_A'), ('experiment', 'rabi_fci'), ('sequence_mode', True), ('t_cinit_green', 5e-07), ('t_cinit_red', 0.003), ('t_cinit_wait_between', 1e-06), ('t_aom_safety', 7.5e-07), ('laser_read_ch', 'd_ch3'), ('add_gate_ch', 'd_ch1'), ('done_ch', 'd_ch1'), ('t_laser_fci_green', 5e-07), ('t_laser_fci_red', 0.003), ('t_wait_fci', 1e-06), ('t_safety_fci', 7.5e-07), ('laser_read_red_ch', 'd_ch3'), ('done_fci_ch', 'd_ch4'), ('rabi_period', 6.23e-08), ('tau_step', 1.2459999999999999e-08), ('tau_start', 0.0), ('alternating', False), ('leave_out_tau_idx', ''), ('microwave_frequency', 2729800000.0), ('num_of_points', 20), ('microwave_amplitude', 0.25), ('measurement_time', None), ('optimize_time', 1200), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('optimize_func_toggle_pause', <function toggle_pg at 0x000000CA59B0D488>), ('savetag', 'rabi_A'), ('gated', True), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr

## Stability

In [192]:
uglobals.abort.clear()


idx = 0
while True:
    if uglobals.abort.is_set():
        break
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        optimize_position()
        laser_off()
        #"""
    except Exception as e:
        logger.warning("auto run {}, skipping failed tracking: {}".format(i, str(e)))
    if uglobals.abort.is_set():
        break
    
    logger.info(f"Starting mes {idx} in stability test")
    qmeas = setup_podmr_params(f_res=common.mw_freq, p_scale=5, f_step=0.8e6, n_f=70)
    qmeas['measurement_time'] = 1.5*60
    qmeas['name'] = 'podmr_fine_A'
    run_generic(qmeas)

    qmeas = setup_rabi_params(tau_step=common.rabi_period/20, n_tau=75, alternating=False) 
    qmeas['name'] = "rabi_A"
    if common.rabi_shaped['shaped_on']:
        qmeas['name'] += "_shaped" 
    qmeas['measurement_time'] = 1.5*60

    run_generic(qmeas)
    idx += 1

        
  

## Nuclear (15-N)

### nuclear pODMR

In [233]:
t_nuc_rabi = 15.8e-6
rf_ampl = 40e-3

qmeas = setup_nuc_podmr_params(f_start=2.6e6,  f_step=10e3, n_f=50, t_nuc_rabi=t_nuc_rabi, rf_ampl=rf_ampl)

uglobals.abort.clear()
run_generic(qmeas)

In [161]:
qmeas

OrderedDict([('name', 'nuc_pulsedodmr'), ('experiment', 'nuc_pulsedodmr'), ('microwave_frequency', 1490708000.0), ('rabi_period', 3.762e-08), ('freq_start', 2700000.0), ('freq_step', 5000.0), ('num_of_points', 50), ('rf_chnl', 'a_ch2'), ('rf_ampl', 0.05), ('t_nuc_rabi', 5e-05), ('nv_pi_init', True), ('alternating_no_rf', True), ('measurement_time', 1500), ('optimize_time', None), ('fit_experiment', 'lor_dip'), ('savetag', 'nuc_pulsedodmr'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('freq_optimize_time', None), ('analysis_interval', None)])

### nuclear Rabi

In [236]:
f_nuc_rabi = 2814e3

qmeas = setup_nuc_rabi_params(tau_step=1e-6, rf_freq=2.81e6, rf_ampl=rf_ampl, n_tau=50)
savetag = f"nucRabi_{round(qmeas['rf_ampl']*1000)}mV"

uglobals.abort.clear()


In [107]:
#run_generic(qmeas, save_tag=savetag)

'nucRabi_40mV'

### power series

In [239]:
uglobals.abort.clear()
pscales = np.linspace(1, 0.1, 8)[::]
p_start = 60e-3 #common.mw_amp

for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:
        p_phys = p_start*p
        qmeas = setup_nuc_rabi_params(tau_step=t_nuc_rabi/20*rf_ampl/p_phys,
                                     rf_freq=f_nuc_rabi, rf_ampl=p_phys, n_tau=50)   # 10e-9
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 40
        savetag = f"nucRabi_idx={i}_ampl={1e3*p_phys:.2f}mV"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")
            

## Ramsey


In [191]:
tau_start = 0.05e-6
n_sweeps = 0#1e3

uglobals.abort.clear()
#qmeas = setup_ramsey_params(tau_start, n_tau=125, tau_step=25e-9, alternating=True)
qmeas = setup_ramsey_params(tau_start, n_tau=150, tau_step=25e-9, alternating=True)

#qmeas = setup_ramsey_params(tau_start, n_tau=70, tau_step=25e-9, alternating=True, read_phases='-90, +90', red_readout=False)
#qmeas = setup_ramsey_params(tau_start, n_tau=50, tau_step=50e-9, alternating=True, read_phases='-90, +90', red_readout=False)

#qmeas = setup_ramsey_params(tau_start, n_tau=50, tau_step=20e-9, alternating=True, read_phases='0, 180', red_readout=False)

#qmeas = setup_ramsey_params(tau_start, n_tau=100, tau_step=100e-9)     
#qmeas = setup_ramsey_params(tau_start, n_tau=120, tau_step=20e-9, alternating=True)
#qmeas = setup_ramsey_params(tau_start, n_tau=200, tau_step=250e-9)      # 75ns: res like mfl
#qmeas = setup_ramsey_params(tau_start, n_tau=400, tau_step=50e-9)     # high res    
#qmeas = setup_ramsey_params(tau_start, n_tau=400, tau_step=100e-9)     # high res  

# debug
#qmeas = setup_ramsey_params(300e-9, n_tau=1, tau_step=20e-9, alternating=True, read_phases='-90, +90', red_readout=False)


x, z = run_ramsey(qmeas, n_sweeps_stop=n_sweeps)
z_binary = 0 #majority_vote(z[0], z_thres=common.mfl_z_thres)

logger.info("Ramsey with offset {} MHz, {} sweeps: {} -> {}".format(common.mw_ramsey_offset/1e6, n_sweeps, z, z_binary))


### Ramsey for (AC Stark) crosstalk

In [991]:
tau_start = 0.05e-6

f_mw_2 = common.mw_freq_2[0] - 20e6  # can't use NV2, as we're also reading it!
ampls_2 = np.linspace(0, common.mw_amps_2[0], 8)[::-1]
t_rabi_2 = common.rabi_period_2[0]


uglobals.abort.clear()
try:
    for ampl_2 in ampls_2:
                
        qmeas = setup_ramsey_params(tau_start, n_tau=int(0.75*100), tau_step=1/0.75*15e-9, alternating=True)
        qmeas['experiment'] = 'ramsey_crosstalk'
        qmeas['savetag'] = f'ramsey_crosstalk_ampl2={1e3*ampl_2:.1f}mV'
        qmeas['measurement_time'] = 10*60


        qmeas['f_mw_2'] = list_2_csv(f_mw_2)
        qmeas['ampl_mw_2'] = list_2_csv(ampl_2)

        run_generic(qmeas)

    # reference Ramsey
    qmeas = setup_ramsey_params(tau_start, n_tau=100, tau_step=15e-9, alternating=True)
    qmeas['measurement_time'] = 15*60
    run_generic(qmeas)
except:
    logger.exception("")
qmeas

OrderedDict([('name', 'Ramsey'), ('experiment', 'ramsey_s3p'), ('rabi_period', 1.207e-07), ('microwave_frequency', 2740000000.0), ('microwave_amplitude', 0.075), ('tau_start', 5e-08), ('tau_step', 1.5e-08), ('num_of_points', 100), ('alternating', True), ('from_list', False), ('optimize_time', None), ('fit_experiment', 'sine_decay'), ('read_phases_degree', '0, 180'), ('measurement_time', 900), ('savetag', 'Ramsey'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', ''), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 1.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', '/Dev1/PFI0'), ('readout

### Compensation of AC Stark

In [1669]:
def add_ac_stark_compensation(qmeas, f_mw_2=None, ampl_comp=[0,0], df_offset=[0e6,0e6], fac_comp=5):
    """
    @param list ampl_comp: amplitude for [0] compensation of effect of f2 on f1. [1] f1 on f2.
    @param float df_offset: linearly add offset to compensation, eg. to balance calibration taken at different f2
    """
    if not 'f_mw_2' in qmeas.keys() or not 'ampl_mw_2' in qmeas.keys():
        raise ValueError("Supplied mes dict doesn't support multi frequency pulses.")

    f_mw_1 = common.mw_freq
    if f_mw_2 == None:
        f_mw_2 = csv_2_list(qmeas['f_mw_2']) 
    if len(f_mw_2) > 1:
        raise NotImplementedError("Jupyter notebook atm only supports 1 frequency per NV")
    f_mw_2 = f_mw_2[0] 
    
    delta_stark_symm_2on1 = f_mw_2 - f_mw_1 
    delta_stark_2on1 = delta_stark_symm_2on1/fac_comp
    f_compensation_2on1 = f_mw_1 - delta_stark_2on1 + df_offset[0]
    
    delta_stark_symm_1on2 = f_mw_1 - f_mw_2 
    delta_stark_1on2 = delta_stark_symm_1on2/fac_comp
    f_compensation_1on2 = f_mw_2 - delta_stark_1on2 + df_offset[1]
    
    # order of fs is important to match t_rabi in pulse generation
    # pi_on_1 requires in parallel f_comp_1on2
    qmeas['f_mw_2'] = list_2_csv([f_compensation_1on2, f_mw_2, f_compensation_2on1])
    ampl_2 = csv_2_list(qmeas['ampl_mw_2'])[0]
    t_rabi_2 = csv_2_list(qmeas['rabi_period_mw_2'])[0]
    
    qmeas['ampl_mw_2'] = list_2_csv([ampl_comp[1], ampl_2, ampl_comp[0]])
    # same length for compensation pulse and actually intended pulse
    qmeas['rabi_period_mw_2'] = list_2_csv([common.rabi_period, t_rabi_2, t_rabi_2])
    
    return qmeas

tau_start = 0.05e-6

common = common.swap_nvs([1,2])

df_calib_offet = -15e6
f_mw_2 = common.mw_freq_2[0] + df_calib_offet
ampl_2 = common.mw_amps_2[0]

ampl_acstark_comp = np.sqrt(2)*0.032
fac_comp = 3

uglobals.abort.clear()
try:
    qmeas = setup_ramsey_params(tau_start, n_tau=int(80), tau_step=1/0.75*15e-9, alternating=True)
    qmeas['experiment'] = 'ramsey_crosstalk'
   
    qmeas['measurement_time'] = 1000*60
    # create AC Stark bias field (at some offset from NV2 resonance)
    qmeas['f_mw_2'] = list_2_csv([f_mw_2])
    qmeas['ampl_mw_2'] = list_2_csv([ampl_2])
    qmeas['rabi_period_mw_2'] = list_2_csv([ampl_2])
 
    qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp,0], 
                                  fac_comp=fac_comp, df_offset=[0,0])
    qmeas['savetag'] = f'ramsey_crosstalk_ampl2={1e3*ampl_2:.1f}mV_amplComp={1e3*ampl_acstark_comp:.1f}mV'

    run_generic(qmeas)

except:
    logger.exception("")
qmeas

OrderedDict([('name', 'Ramsey'), ('experiment', 'ramsey_crosstalk'), ('rabi_period', 1.162e-07), ('microwave_frequency', 2740500000.0), ('microwave_amplitude', 0.075), ('tau_start', 5e-08), ('tau_step', 1.9999999999999997e-08), ('num_of_points', 80), ('alternating', True), ('from_list', False), ('optimize_time', None), ('fit_experiment', 'sine_decay'), ('read_phases_degree', '0, 180'), ('measurement_time', 60000), ('f_mw_2', '2834900000.0, 2810300000.0, 2711900000.0'), ('ampl_mw_2', '0, 0.175, 0.04525483399593905'), ('rabi_period_mw_2', '1.162e-07, 0.175, 0.175'), ('savetag', 'ramsey_crosstalk_ampl2=175.0mV_amplComp=45.3mV'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', ''), ('laser_channel

In [1585]:
# compensation from resonance
(csv_2_list(qmeas['f_mw_2'])[2] - common.mw_freq)/1e6

-29.52

In [ ]:
# power series (of compensation)

In [1928]:
(np.concatenate([np.linspace(4/5*ampl_comp_max, ampl_comp_max, 6)])-5*0.004)

array([0.068 , 0.0724, 0.0768, 0.0812, 0.0856, 0.09  ])

In [1935]:
tau_start = 0.05e-6

common = common.swap_nvs([2,1])




# place the compensation with low power close to f_mw1 (instead of same power with opposite detuning)
# in this way, ac stark on other nv will be lower (due to fraction of delta_starks)
fac_comp_stark = 3
df_calib_offet = -15e6

f_mw_2 = common.mw_freq_2[0] + df_calib_offet
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]

# set up compensation
delta_stark_symm = f_mw_2 - common.mw_freq
delta_stark = delta_stark_symm/fac_comp_stark
f_compensation = common.mw_freq - delta_stark
ampl_comp_max = 0.25 - ampl_2
ampl_comp_max = 0.11
ampls_comp = np.linspace(0, ampl_comp_max, 12)#[::-1]
ampls_comp = np.linspace(4/5*ampl_comp_max, ampl_comp_max, 6)#[::-1]
ampls_comp = (np.linspace(4/5*ampl_comp_max, ampl_comp_max, 6)-5*0.0044)[::-1]
#ampls_comp = np.ravel([ampls_comp[::2], ampls_comp[1::2][::-1]],order='F')
# this is the expected point of good compensation
ampl_comp_theo = np.sqrt(1/fac_comp_stark)*ampl_2

logger.info(f"AC Stark compensation at delta= {delta_stark/1e6:.1f} MHz, "\
            f"ampl_theo={1e3*ampl_comp_theo:.1f} mV, ampl<= {1e3*ampl_comp_max:.1f} mV")

uglobals.abort.clear()
try:
    for ampl_3 in ampls_comp:  
        
        qmeas = setup_ramsey_params(tau_start, n_tau=int(100), tau_step=1/0.75*15e-9, alternating=True)
        qmeas['experiment'] = 'ramsey_crosstalk'
        qmeas['savetag'] = f'ramsey_crosstalk_ampl2={1e3*ampl_2:.1f}mV_amplComp={1e3*ampl_3:.1f}mV'
        qmeas['measurement_time'] = 20*60
        # create AC Stark bias field (at some offset from NV2 resonance)
        qmeas['f_mw_2'] = list_2_csv([f_mw_2])
        qmeas['ampl_mw_2'] = list_2_csv([ampl_2])
        # turn compensation on
        qmeas['f_mw_2'] = list_2_csv([0, f_mw_2, f_compensation])
        qmeas['ampl_mw_2'] = list_2_csv([0, ampl_2, ampl_3])

        run_generic(qmeas)

    # reference Ramsey
    #qmeas = setup_ramsey_params(tau_start, n_tau=int(0.75*100), tau_step=1/0.75*15e-9, alternating=True)
    #qmeas['measurement_time'] = 15*60
    #run_generic(qmeas)
except:
    logger.exception("")
qmeas

OrderedDict([('name', 'Ramsey'), ('experiment', 'ramsey_crosstalk'), ('rabi_period', 1.11e-07), ('microwave_frequency', 2829300000.0), ('microwave_amplitude', 0.175), ('tau_start', 5e-08), ('tau_step', 1.9999999999999997e-08), ('num_of_points', 100), ('alternating', True), ('from_list', False), ('optimize_time', None), ('fit_experiment', 'sine_decay'), ('read_phases_degree', '0, 180'), ('savetag', 'ramsey_crosstalk_ampl2=75.0mV_amplComp=66.0mV'), ('measurement_time', 1200), ('f_mw_2', '0, 2721500000.0, 2859900000.0'), ('ampl_mw_2', '0, 0.075, 0.066'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', ''), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_leng

In [1333]:
delta_stark/1e6, common.mw_freq, f_mw_2

(-17.183333333333334, 2824400000.0, 2721300000.0)

## Hahn


In [69]:
uglobals.abort.clear()

he_tau_step = 0.250e-6
he_n_tau = 3*25
#he_n_tau = 50

logger.info(f"Tau max= {he_tau_step*he_n_tau}")

qmeas = setup_hahn_params(tau_start=100e-9, n_tau=he_n_tau, tau_step=he_tau_step, alternating=True)
#qmeas = setup_hahn_params(tau_start=14e-6, n_tau=he_n_tau, tau_step=he_tau_step, alternating=True)
run_generic(qmeas)

## XY 8 

In [111]:
common.__dict__

{'rabi_period_rect': 9.06e-08, 'rabi_shaped': {'shaped_on': False, 'period': 1.101e-07, 'env_type': <EnvelopeMethods.parabola: 'SinEnvelopeParabolaN'>, 'order_P': 1}, 'mw_freq': 2727260000.0, 'mw_amp': 0.1388888888888889, 't_laser_pulsed': 3e-06, 't_wait_pulsed': 1e-06, 't_laser': None, 't_laser_green_init': 3e-06, 't_laser_red_read': 1e-06, 'mw_ramsey_offset': -1000000.0, 'mw_ramsey_off_from_calib': 0.0, 'gamma_n_c13': 6726.149871335747}

tau sweep

In [167]:
#tau_start = 0.2e-6
tau_start = 50e-9
tau_start = 10e-9

uglobals.abort.clear()
# narrow resonance NV M17
qmeas = setup_xy8_tausweep_params(tau_start, n_tau=30, tau_step=50e-9, xy8_order=4, alternating=True)
qmeas['measurement_time'] = 20*60
run_xy8_tausweep(qmeas)

common = common.swap_nvs([2,1])
qmeas = setup_xy8_tausweep_params(tau_start, n_tau=30, tau_step=50e-9, xy8_order=4, alternating=True)
qmeas['measurement_time'] = 20*60
common = common.swap_nvs([1,2])



#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=10, tau_step=25e-9, xy8_order=800, alternating=True)


# larger sweeps
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=10e-9, xy8_order=4, alternating=True) 
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=3e-9, xy8_order=4, alternating=True) 
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=250, tau_step=20e-9, xy8_order=4, alternating=True)
#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=75, tau_step=5e-9, xy8_order=8, alternating=True) 

#qmeas = setup_xy8_tausweep_params(tau_start, n_tau=50, tau_step=100e-9, xy8_order=8, alternating=True)


run_xy8_tausweep(qmeas)

Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A760B5EF68>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A760B5EFC0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A760B5EF68>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator o

    yield IdentityTransform(), self
SystemError: error return without exception set
Exception ignored in: <generator object CompositeGenericTransform._iter_break_from_left_to_right at 0x000000A760B5EEB8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 2457, in _iter_break_from_left_to_right
    yield lh_compliment, rh_compliment + self._b
SystemError: error return without exception set
Exception ignored in: <generator object Transform._iter_break_from_left_to_right at 0x000000A760B5EEB8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 1324, in _iter_break_from_left_to_right
    yield IdentityTransform(), self
SystemError: error return without exception set
Exception ignored in: <generator object CompositeGenericTransform._iter_break_from_left_to_right at 0x000000A760B5EE60>
Traceback (most recent call last):
  File "C:\ProgramDa

Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000A6BF90BCA8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set
Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000A7B03C8468>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set
Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000A716975410>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom !=

n sweep

In [179]:
tau = 900e-9#440.0e-9
tau = 100e-9#440.0e-9
#tau = 441.6e-9

uglobals.abort.clear()
#qmeas = setup_xy8_nsweep_params(tau, n_n=25, xy8_start=4, xy8_step=1, alternating=True)
#qmeas = setup_xy8_nsweep_params(tau, n_n=15, xy8_start=4, xy8_step=100, alternating=True)  # long t_evol, mem limit
qmeas = setup_xy8_nsweep_params(tau, n_n=30, xy8_start=1, xy8_step=1, alternating=True)  
qmeas['name'] = 'xy8_n_A'
qmeas['measurement_time'] = 30*60
#run_xy8_nsweep(qmeas)


common = common.swap_nvs([2,1])
qmeas = setup_xy8_nsweep_params(tau, n_n=30, xy8_start=1, xy8_step=1, alternating=True)  
qmeas['name'] = 'xy8_n_B'
qmeas['measurement_time'] = 30*60
run_xy8_nsweep(qmeas)
common = common.swap_nvs([1,2])

Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A711C30BF8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A711C30AF0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A711C30AF0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator o

    yield lh_compliment, rh_compliment + self._b
SystemError: error return without exception set
Exception ignored in: <generator object Transform._iter_break_from_left_to_right at 0x000000A711C30AF0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 1324, in _iter_break_from_left_to_right
    yield IdentityTransform(), self
SystemError: error return without exception set
Exception ignored in: <generator object CompositeGenericTransform._iter_break_from_left_to_right at 0x000000A711C30FC0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 2457, in _iter_break_from_left_to_right
    yield lh_compliment, rh_compliment + self._b
SystemError: error return without exception set
Exception ignored in: <generator object _process_plot_var_args._getdefaults.<locals>.<genexpr>.<genexpr> at 0x000000A711C30888>
Traceback (most recent call last):

    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set
Exception ignored in: <generator object Gcf.destroy_fig.<locals>.<genexpr> at 0x000000A70A738728>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\_pylab_helpers.py", line 67, in <genexpr>
    if manager.canvas.figure == fig), None)
SystemError: error return without exception set


In [185]:
qmeas

OrderedDict([('experiment', 'xy8_nsweep'), ('name', 'xy8_n'), ('rabi_period', 6.65e-08), ('microwave_frequency', 2727900000.0), ('microwave_amplitude', 0.15), ('tau', 1e-07), ('xy8_start', 1), ('xy8_step', 1), ('num_of_points', 10), ('alternating', True), ('from_list', False), ('optimize_time', None), ('fit_experiment', 'sine_decay'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('measurement_time', None), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 8000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', ''), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 1.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', '/Dev1/PFI0'), ('readout_end', 3e-07), ('max_tau', 0.001), ('max_tau_start', 0.

In [ ]:
a few xy8 tau sweeps

In [54]:
uglobals.abort.clear()

tau_start = 0.5e-6
xy_orders = np.arange(5, 16, 1)

for i, o in enumerate(xy_orders):
    if uglobals.abort.is_set():
        break
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        optimize_position()
        laser_off()
        #"""
    except Exception as e:
        logger.warning("auto XY8 run {}, skipping failed tracking: {}".format(i, str(e)))
    if uglobals.abort.is_set():
        break
        
    qmeas = setup_xy8_tausweep_params(tau_start, n_tau=50, tau_step=20e-9, xy8_order=o, alternating=True)     
    qmeas['measurement_time'] = 180
    run_xy8_tausweep(qmeas)

### DD DQT

In [167]:
# drive all HFS lines
common.mw_freq = 2727.26e6
common.mw_amp = 0.25/1.8


t_rabi_dqt = 93.8e-9
t_rabi1 = 97.2e-9  # driving in between HFS lines
t_rabi1 = 93.56e-9  # driving only central line
ampl_mw2 = 0.25#[common.mw_amp]

f_mw1_add = [2725.11e6, 2729.41e6]
#f_mw1 = sorted([common.mw_freq] + f_mw1_add)
f_mw1 = [common.mw_freq] 
f_mw2 = [2946.45e6, 2948.6e6, 2950.75e6]

# drive in left, right middle between 3 HFS lines
#f_mw2 = [np.mean(f_mw2[0:2]), np.mean(f_mw2[1:])]
#f_mw1 = [np.mean(f_mw1[0:2]), np.mean(f_mw1[1:])]



In [169]:
f_mw1, f_mw2

([2727260000.0], [2946450000.0, 2948600000.0, 2950750000.0])

In [175]:
# only lower drive (with potential multiple drives for HFS), no DQT
qmeas = setup_dd_dqt_sigamp_params(tau=200e-9, n_amp=50, amp_step=5e-3,
                                    dd_order=1, t_rabi1=t_rabi1,  
                      f_mw1=f_mw1, f_mw2=[], ampl_mw2=0,
                      dd_type=DDMethods.XY8, alternating=True)
#qmeas['amp_start'] = 20e-3

uglobals.abort.clear()
run_generic(qmeas)

In [51]:
qmeas = setup_dd_dqt_sigamp_params(tau=200e-9, n_amp=50, amp_step=0.002e-3,
                                    dd_order=1, t_rabi1=t_rabi_dqt,  
                      f_mw1=f_mw1, f_mw2=f_mw2, ampl_mw2=ampl_mw2,
                      dd_type=DDMethods.XY8, alternating=True)

uglobals.abort.clear()
#run_generic(qmeas)

In [127]:
qmeas

OrderedDict([('rabi_period', 9.356e-08), ('name', 'dd_dqt_sigamp'), ('experiment', 'dd_dqt_sigamp'), ('microwave_amplitude', 0.1388888888888889), ('tau', 2e-07), ('amp_start', 0.0), ('amp_step', 0.01), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 1), ('alternating', True), ('num_of_points', 50), ('microwave_frequency', 2727260000.0), ('measurement_time', None), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>)])

## Hartmann Hahn

In [509]:
uglobals.abort.clear()

qmeas = setup_hh_tau_params(tau_start=0e-6, tau_step=0.6e-6, n_tau=50, amp_sl=1.35e-3)


run_generic(qmeas)

### power series

In [512]:
uglobals.abort.clear()
pscales = np.linspace(1, 0.68, 20)
p_start_sl = 1.6e-3  # 1.65 ~ 523 kHz = Lamor

for i, p in enumerate(pscales):
    if uglobals.abort.is_set():
        break
    try:

        qmeas = setup_hh_tau_params(tau_start=0e-6, tau_step=0.65e-6, n_tau=50, amp_sl=p_start_sl*p)
        #qmeas = setup_rabi_params(tau_step=250e-9, p_scale=10)
        qmeas['measurement_time'] = 2*60
        savetag = f"hh_tau_idx={i}_ampl_lock={1e3*p_start_sl*p:.2f}mV"
        run_generic(qmeas, save_tag=savetag)
    except Exception as e:
        stack_trace = traceback.format_exc()
        logger.warning("auto run {} failed: {}.\n{}".format(i, str(e), stack_trace))

logger.info("Autopilot has landed.")

Double resonance

## DEER

In [483]:
# deprecated, better use params in common.(..)

he_tau = 0.3e-6

t_rabi_2 = 100e-9#200.0e-9
f_center = 100e6#2.8307e9

common

### Spectrum

In [44]:
uglobals.abort.clear()

f_center = 1.407e9
f_center = 3.05e9
#f_center = 1.305e9
#f_center = 0.735e9
#f_center = 0.850e9
#t_pi_fix = 6000e-9   # if already calibrated by DEER Rabi
t_pi_fix = None #138.0e-9/2    # else

# wide band
#qmeas = setup_deer_f_params(f_center, f_step=5e6, n_f=100, two_deer_pi=True,
#                            pi_len=t_pi_fix)
#qmeas = setup_deer_f_params(f_center, f_step=3e6, n_f=150, two_deer_pi=True,
#                            he_tau=common.he_tau, pi_len=t_pi_fix, read_phases='-90, 90')
qmeas = setup_deer_f_params(f_center, f_step=3e6, n_f=50, two_deer_pi=True,
                            he_tau=common.he_tau, pi_len=t_pi_fix, read_phases='0, 180')
#qmeas = setup_deer_f_params(f_center, f_step=6e6, n_f=75, two_deer_pi=False,
#                            he_tau=common.he_tau, pi_len=t_pi_fix, read_phases='-90, 90', deer_ampl=0.25/5.)
#qmeas = setup_deer_f_params(f_center, f_step=1e6, n_f=200, two_deer_pi=False,
#                            pi_len=t_pi_fix)
# high res
#qmeas = setup_deer_f_params(f_center, f_step=500e3, n_f=75, two_deer_pi=True,
#                           he_tau=common.he_tau, pi_len=t_pi_fix)
# wide band
#qmeas = setup_deer_f_params(f_center, f_step=2000e3, n_f=120, two_deer_pi=True,
#                           he_tau=300e-9)

#run_deer_f(qmeas)

### DD Spectrum

Remember to set common.mw_freq and rabi to the correct NV

In [171]:
# main resonance
# {'shaped_on':True, 'period': 244.5e-9, 'env_type': Evm.parabola, 'order_P':1}
#        self.mw_freq = 3086.1e6 #3026
#t_rabi_2 = 2*t_pi_fix if t_pi_fix else None
#common.he_tau = 0.2e-6

f_center = common.mw_freq_2[0]
dd_tau = common.dd_tau
dd_order = common.dd_order
dd_type = common.dd_type #DDMethods.XY8
t_rabi_2 = common.rabi_period_2[0]

try:
    qmeas = setup_deer_dd_f_params(f_center, dd_tau=dd_tau, f_step=3e6, n_f=50,
                                   t_rabi_2=t_rabi_2, dd_type=dd_type, dd_order=dd_order)
    #qmeas = setup_deer_dd_f_params(f_center, dd_tau=common.he_tau, f_step=3e6, n_f=2,
    #                               t_rabi_2=t_rabi_2, dd_type=DDMethods.SE, dd_order=2)
    #qmeas['order_P'] = 10
    #qmeas['mw_amp2'] = 0
except:
    logger.exception("")
uglobals.abort.clear()
run_generic(qmeas)

In [165]:
qmeas

OrderedDict([('experiment', 'DEER_DD_freq'), ('name', 'deer_dd_spectrum'), ('microwave_frequency', 2738400000.0), ('microwave_amplitude', 0.25), ('freq_incr', 3000000.0), ('num_of_points', 50), ('freq_start', 2745500000.0), ('rabi_period', 5.74e-08), ('measurement_time', 150000), ('optimize_time', None), ('alternating', True), ('tau', 3e-07), ('rabi_period2', 1.086e-07), ('mw_amp2', 0.25), ('tau2_rel_to_pi1', True), ('tau2', 0.0), ('DD_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('order', 15), ('env_type', <EnvelopeMethods.parabola: 'SinEnvelopeParabolaN'>), ('order_P', 1)])

### DD tau / CNOT

Implement CNOT by read_phase= 90! Mind that this works better with pulse shaping turned on, especially if nuclei are around.
ATTENTION: In mixed rect/shaped mode, for calibration of CNOT shaping must be turned on!

In [50]:
common.dd_tau, common.dd_order,  common.dd_type#, -tau2_pspacing/2

(1e-06, 2, <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>)

In [703]:
common = common.swap_nvs(order=[1,2])

common.dd_tau

1e-06

In [239]:
#common.he_tau = 0.2e-6
#t_rabi_2 = 196.5e-9
#f_deer= f_center

# to switch between C2NOT1 (order=1,2) and C1NOT2
common = common.swap_nvs(order=[1,2])

f_deer = common.mw_freq_2[0]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]
dd_tau = common.dd_tau  # -1 for deer_dd_par
#dd_tau = -1
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


# Test contrast
# may reduce dd_order to see whether coherence is as expected
qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, tau2_start=0e-9, tau2_step=-6e-9, n_tau=30,
                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=4,  # dd_order  
                             read_phase=0) 
qmeas['tau2_rel_to_pi1'] = False
#qmeas['ampl_mw_2'] = '0'

tau2_pspacing = dd_tau-common.rabi_period/2-t_rabi_2/2
# for optimal control pulses of length 100ns
tau2_pspacing = dd_tau-2*100e-9
if dd_tau > 0:
    n_tau = 20#20
    tau2_step = tau2_pspacing/n_tau
    tau2_start = -tau2_pspacing/2
else:
    n_tau = 20
    tau2_start = common.rabi_period/2
    tau2_step = (1/70e3/(common.dd_order*common.dd_type.suborder))/n_tau

uglobals.abort.clear()
inits = [[0,1]]
#inits =  [[0,0],[0,1],[1,0],[1,1]]#[::-1]
for (init_1, init_2) in inits:
    qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, 
                                 tau2_start=tau2_start, tau2_step=tau2_step, n_tau=n_tau, # 40
                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=dd_order, ## dd_order
                             read_phase=90, charge_init=False) #read_phase=90 for CNOT
    qmeas['tau2_rel_to_pi1'] = False

    qmeas['init_pix_on_1'] = init_1  #1
    qmeas['init_pix_on_2'] = init_2  #1
    qmeas['end_pix_on_2'] = 1  #1

    # for shiting
    #qmeas['env_type_1'] = Evm.optimal
    #qmeas['env_type_2'] = Evm.optimal
    # for parallel
    #qmeas['env_type'] = Evm.optimal
    
    

    # AC Stark compensation, get values from doing a calibration with AC Stark Ramsey
    # ATTENTION: adding compensation must not exceed maximum MW amplitude.
    # Safe to do, if NV1/2 pulses don't overlap.
    #ampl_acstark_comp_2on1 = 0.045
    #ampl_acstark_comp_1on2 = 0#0.075
    #df_calib_offset = -15e6
    #df_offset_2on1 = (ampl_acstark_comp_2on1/common.mw_amp)**2*df_calib_offset
    #df_offset_1on2 = (ampl_acstark_comp_1on2/common.mw_amps_2[0])**2*df_calib_offset
    #fac_comp = 3

    #qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp_2on1,ampl_acstark_comp_1on2], 
    #                                 fac_comp=fac_comp, df_offset=[df_offset_2on1,df_offset_1on2])


   

    #qmeas['measurement_time'] = 120*60
    qmeas['optimize_time'] = 30*60
    qmeas['optimize_func_toggle_pause'] = toggle_pg
    
    qmeas['alternating'] = True
    #qmeas['generate_new'] = False  # todo: not working
    
    qmeas['savetag'] = f"deer_dd_{dd_type.name}-{dd_order}_init={init_1}{init_2}"
    qmeas['savetag'] = qmeas['savetag'] + "_fci" if 'fci' in qmeas['experiment'] else qmeas['savetag']
    
    run_generic(qmeas)

#qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=dd_tau, 
#                                 tau2_start=-tau2_pspacing/2, tau2_step=tau2_pspacing/40, n_tau=40, # 75
#                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=DDMethods.XY8, dd_order=4, ## dd_order
#                             read_phase=90) #90 for CNOT
#qmeas['tau2_rel_to_pi1'] = False
#qmeas['end_pix_on_2'] = 1  #1
#qmeas['init_pix_on_2'] = 0  #1

qmeas

OrderedDict([('experiment', 'deer_dd_tau'), ('name', 'deer_dd_rect_tau'), ('microwave_frequency', 2729800000.0), ('microwave_amplitude', 0.25), ('rabi_period', 6.23e-08), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 2), ('num_of_points', 20), ('tau_start', -4e-07), ('tau_step', 4e-08), ('f_mw_2', '2930710000.0'), ('rabi_period_mw_2', '9.87e-08'), ('ampl_mw_2', '0.25'), ('read_phase_deg', 90), ('tau1', 1e-06), ('measurement_time', 150000), ('optimize_time', 1800), ('alternating', True), ('pulse_envelope', <PulseEnvelopeType.rectangle: 'rectangle'>), ('tau2_rel_to_pi1', False), ('init_pix_on_1', 0), ('init_pix_on_2', 1), ('end_pix_on_2', 1), ('optimize_func_toggle_pause', <function toggle_pg at 0x0000005612318158>), ('savetag', 'deer_dd_XY8-2_init=01'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 40), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 1

For debugging, stop sequence for a single tau2 after each of its pi (on NV1 and NV2) pulses.

In [236]:
qmeas['experiment'] = 'deer_dd_tau_interm'
qmeas['name'] = 'deer_dd_tau_interm'
qmeas['tau_start'] = 0
qmeas['tau2'] = qmeas['tau_start'] 

In [230]:
run_generic(qmeas)

In [1389]:
df_offset_2on1/1e6, df_offset_1on2/1e6

(-2.7306666666666675, -1.1284897959183673)

For different read phases

In [979]:
uglobals.abort.clear()
# optimize read phase
read_phases = np.linspace(90, 0, 20)
debug = []

for phi in read_phases:
    qmeas = setup_deer_dd_tau_params(f_deer, dd_tau=common.dd_tau, tau2_start= 0e-9, tau2_step=-dd_tau/75, n_tau=75,
                             t_rabi_2=t_rabi_2, deer_ampl=ampl_2, dd_type=dd_type, dd_order=dd_order,
                             read_phase=phi) #90 for CNOT


    qmeas['end_pix_on_2'] = 1  #1
    qmeas['init_pix_on_2'] = 0  #1
    qmeas['measurement_time'] = 30*60
    qmeas['fit_experiment'] = 'sine'   
    qmeas['savetag'] = f"deer_dd_cnot_readphi={phi:.1f}_init_on2={qmeas['init_pix_on_2']}"
    
    debug.append(qmeas)
    run_generic(qmeas)
    
    qmeas['end_pix_on_2'] = 1  #1
    qmeas['init_pix_on_2'] = 1  #1
    qmeas['measurement_time'] = 30*60
    qmeas['fit_experiment'] = 'sine'   
    qmeas['savetag'] = f"deer_dd_cnot_readphi={phi:.1f}_init_on2={qmeas['init_pix_on_2']}"
    
    run_generic(qmeas)
    
debug[-1]
    

OrderedDict([('experiment', 'DEER_DD_tau'), ('name', 'deer_dd_tau_shaped'), ('microwave_frequency', 2736200000.0), ('microwave_amplitude', 0.075), ('rabi_period', 1.863e-07), ('tau2_rel_to_pi1', True), ('DD_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('order', 4), ('mw_freq2', 2825900000.0), ('num_of_points', 75), ('tau2_start', 0.0), ('tau2_incr', -6.666666666666666e-09), ('tau', 5e-07), ('rabi_period2', 1.84e-07), ('mw_amp2', 0.175), ('read_phase', 0.0), ('measurement_time', 1800), ('optimize_time', None), ('alternating', True), ('env_type', <EnvelopeMethods.parabola: 'SinEnvelopeParabolaN'>), ('order_P', 1), ('end_pix_on_2', 1), ('init_pix_on_2', 1), ('fit_experiment', 'sine'), ('savetag', 'deer_dd_cnot_readphi=0.0_init_on2=1'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('freq_optimize_time', None), ('analysis_interval', None)])

In [527]:
list_2_csv(1.)

'1.0'

### DEER Rabi

In [49]:
uglobals.abort.clear()

f_deer = 1.3989e9
f_deer = 1.9e9
#f_deer = 0.737e9


qmeas = setup_deer_rabi_params(f_deer, tau_start=5e-9, tau_step=25e-9, n_tau=40, he_tau=common.he_tau, 
                               two_deer_pi=False, read_phases='-90, 90', deer_ampl=0.25)

#qmeas = setup_deer_rabi_params(f_deer, tau_start=27.5e-9, tau_step=1e-9, n_tau=1, he_tau=common.he_tau)
#qmeas = setup_deer_rabi_params(f_deer, tau_start=5e-9, tau_step=4e-9, n_tau=80, he_tau=300e-9, two_deer_pi=True)

#qmeas = setup_deer_rabi_params(f_deer, tau_start=5e-9, tau_step=4e-9, n_tau=80, he_tau=300e-9, two_deer_pi=False)
run_deer_rabi(qmeas)


In [197]:
qmeas

OrderedDict([('name', 'deer_rabi'), ('tau_start', 5e-09), ('num_of_taus', 80), ('tau_step', 4e-09), ('measurement_time', 99999), ('optimize_time', None), ('he_tau', 3e-07), ('two_deer_pi', False), ('deer_amp', 0.25), ('deer_freq', 820000000.0), ('alternating', True), ('experiment', 'deer_rabi'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 8000000000.0), ('bin_width', 1e-09), ('wait_time', 1e-06), ('laser_delay', 9e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', ''), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 8.699999999999999e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', '/Dev1/PFI0'), ('readout_end', 3e-07), ('max_tau', 0.001), ('max_tau_start', 0.001), ('max_rabi_period', 0.001), ('min_microwave_freq

### time DEER

In [152]:
uglobals.abort.clear()

f_deer = 1.407e9
#f_deer = 1.00e9
f_deer = 1.3795e9
f_deer = 3026.7e6
deer_rabi_period = 138.0e-9

# wide band
#qmeas = setup_deer_f_params(f_center, f_step=5e6, n_f=50, two_deer_pi=True)
# high res

qmeas = setup_deer_params(f_deer, tau_start=5e-9, tau_step=100e-9, n_tau=50, 
                      he_tau=common.he_tau, second_rabi_period=deer_rabi_period,
                      two_deer_pi=True)
#qmeas = setup_deer_params(f_deer, tau_start=282e-9, tau_step=0e-9, n_tau=1, 
#                      he_tau=common.he_tau, second_rabi_period=deer_rabi_period,
#                      two_deer_pi=True)
#qmeas['wait_time'] = 1e-6
qmeas['two_deer_pi'] = True


run_deer(qmeas)

#qmeas['name'] = 'deer_pi_start'
#run_deer_pi_start(qmeas)

In [215]:
qmeas

OrderedDict([('name', 'DEER'), ('tau_start', 5e-09), ('num_of_points', 40), ('tau_step', 2e-09), ('measurement_time', 99999), ('optimize_time', None), ('he_tau', 3e-07), ('two_deer_pi', True), ('deer_amp', 0.25), ('deer_freq', 1398900000.0), ('second_rabi_period', 2e-07), ('alternating', True), ('wait_time', 1e-06)])

### T1 like DEER

If the second electron spin is polarized by the laser pulse and its T1 << T1(NV), we can do the following experiment: increasing the t_wait of the NV initialization won't hurt the NV sensor too much. However, the second electron spin starts to decay right after the laser pulse. If it's an ensemble spin, the lowered polarization due to its T1 decay will lower the effective coupling to the sensor. Thus, the frequency observed in the FFT of deer should lower

In [38]:
t_waits = np.linspace(1e-6, 20e-6, 10)[1:]#[::-1]
uglobals.abort.clear()

for t in t_waits:
    qmeas = setup_deer_params(f_deer, tau_start=5e-9, tau_step=2e-9, n_tau=40, 
                          he_tau=300e-9, second_rabi_period=deer_rabi_period,
                          two_deer_pi=True)
    qmeas['wait_time'] = t
    
    qmeas['n_sweeps'] = 1e6
    qmeas['name'] = 'deer_pi_start'

    qmeas['save_tag'] = f'DEER_pi_start_t_wait={t}'

    #run_deer(qmeas, save_tag=qmeas['save_tag'])
    qmeas['two_deer_pi'] = False
    run_deer_pi_start(qmeas, save_tag=qmeas['save_tag'])

### pODMR DEER pi

In [62]:
qmeas

OrderedDict([('name', 'pulsedodmr_deer_pi'), ('experiment', 'pulsedodmr_deer_pi'), ('rabi_period', 5.64e-07), ('freq_step', 500000.0), ('num_of_points', 75), ('freq_start', 1431550000.0), ('microwave_amplitude', 0.004166666666666667), ('deer_amplitude', 0.25), ('measurement_time', 1500), ('optimize_time', None), ('fit_experiment', 'lor_dip'), ('f_mw_deer', 737000000.0), ('t_pi_deer', 3.4e-08)])

In [71]:
f_deer = 0.737e9
#f_deer = 1.2e9   # off-resonant dummy frequency
t_pi_deer = 68e-9 / 2

uglobals.abort.clear()

qmeas = setup_podmr_deer_pi_params(f_res=common.mw_freq, f_deer=f_deer, t_pi_deer=t_pi_deer,
                           p_scale=60, f_step=500e3, n_f=150) # range ~ 1 MHz, lowest AWG power

run_generic(qmeas)

### Ramsey DEER pi

Ramsey with a pi pulse on the 2nd electron spin right after the 1st pi/2 on the NV.

In [71]:
f_deer = 0.737e9
f_deer = 1.2e9   # off-resonant dummy frequency
t_pi_deer = 68e-9 / 2


uglobals.abort.clear()
#qmeas = setup_ramsey_deer_pi_params(f_deer, t_pi_deer, 
#                                tau_start=50e-9, n_tau=50, tau_step=20e-9,
#                                alternating_mode=DeerAltModes.NVPi3Half)
qmeas = setup_ramsey_deer_pi_params(f_deer, t_pi_deer, 
                                tau_start=50e-9, n_tau=160, tau_step=10e-9,
                                alternating_mode=DeerAltModes.DeerPiOff_plus_NVPi3Half,
                                two_deer_pi=True)


run_generic(qmeas)

In [59]:
f_deer

1200000000.0

## Charge read

In [53]:
def setup_charge_read_params(t_cinit_green=500e-9, t_cread_red=50e-6, t_wait=1e-6,
                             laser_red_ch='d_ch3', add_gate_ch='',
                                 gate_channel=None):

    qmeas=OrderedDict()
    
    qmeas['experiment'] = 'laser_strob'
    qmeas['name'] = 'charge_read'
    qmeas['listfile'] = True 

    
        
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    # charge read
    qmeas['t_laser_init'] = t_cinit_green
    qmeas['t_laser_read'] = t_cread_red
    qmeas['t_wait_between'] = t_wait
    qmeas['t_aom_safety'] = 750e-9

    
    qmeas['laser_read_ch'] = laser_red_ch
    qmeas['add_gate_ch'] = add_gate_ch
    
   
    qmeas['measurement_time'] = 9999 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = None


    return qmeas


def setup_charge_read_fci_params(t_cinit_green=500e-9, t_cinit_red=10e-6,
                                 t_cread_red=50e-6, t_wait=1e-6, 
                                 laser_red_ch='d_ch3', add_gate_ch='', done_ch='d_ch1',
                                 gate_channel=None):

    qmeas=OrderedDict()
    
    qmeas['name'] = 'charge_read_fci'
    qmeas['experiment'] = 'charge_read_fci'
    qmeas['sequence_mode'] = True 
    qmeas['listfile'] = True 

    
    # ATTENTION: changes setup gate ch!
    if gate_channel != None:
        setup['gate_channel'] = gate_channel
        # need to manually change fastcounter trigger ovoltages!
        setup['analog_trigger_voltage'] = 0.25
    
    setup['laser_length'] = common.t_laser_pulsed
    setup['wait_time'] = common.t_wait_pulsed
    
    # charge init
    qmeas['t_cinit_green'] = t_cinit_green
    qmeas['t_cinit_red'] = t_cinit_red
    qmeas['t_wait_between'] = t_wait
    qmeas['t_aom_safety'] = 750e-9
    
    # charge read
    qmeas['t_cread_red'] = t_cread_red
    
    
    
    qmeas['laser_red_ch'] = laser_red_ch
    qmeas['add_gate_ch'] = add_gate_ch
    qmeas['done_ch'] = done_ch
    
   
    qmeas['measurement_time'] = 9999 
    qmeas['optimize_time'] = None
    qmeas['fit_experiment'] = None


    return qmeas

0

0

Need to:
1. acquire in fastcomtec listmode to create photon histogram
2. set red laser power correctly (very low!)

In [235]:
# default settings
myfastcounter.change_save_mode(0)
#awg8190.set_sample_rate(10e9)  # todo: better do manually!?

0

In [74]:
qmeas = setup_charge_read_params(t_cinit_green=30e-6, t_cread_red=3e-3, t_wait=1e-6, #15e-3
                             laser_red_ch='d_ch3', add_gate_ch='d_ch4')
#qmeas = setup_charge_read_params(t_cinit_green=30e-6, t_cread_red=5e-3, t_wait=1e-6,
#                             laser_red_ch='d_ch3', add_gate_ch='d_ch4')
# debug: short red pulse
#qmeas = setup_charge_read_params(t_cinit_green=10e-6, t_cread_red=100e-6, t_wait=1e-6,
#                             laser_red_ch='d_ch3', add_gate_ch='d_ch4')

# if using external fci circuit, other fastcomtec gate/sync trigger channel 
setup['gate_channel'] = 'a_ch2'
qmeas['bin_width'] = 4.096e-7
#setup['gated'] = True
#setup['sync_channel'] = ''
#setup['gate_channel'] = 'd_ch1'

# need to manually change fastcounter trigger ovoltages!
setup['analog_trigger_voltage'] = 0.25

qmeas['savetag'] = f"charge_read"


uglobals.abort.clear()
run_generic(qmeas)

qmeas


OrderedDict([('experiment', 'laser_strob'), ('name', 'charge_read'), ('listfile', True), ('t_laser_init', 3e-05), ('t_laser_read', 0.003), ('t_wait_between', 1e-06), ('t_aom_safety', 7.5e-07), ('laser_read_ch', 'd_ch3'), ('add_gate_ch', 'd_ch4'), ('measurement_time', 9999), ('optimize_time', None), ('fit_experiment', None), ('bin_width', 4.096e-07), ('savetag', 'charge_read'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 1), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', ''), ('gate_channel', 'a_ch2'), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 1.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', '/Dev1/PFI0'), ('readout_end', 3e-07), ('max_tau', 0.001), ('max_tau_start', 0.001), (

In [847]:

t_reads = np.linspace(2e-3, 12e-3, 6)
#t_reads = np.concatenate([t_reads, np.linspace(3.2e-3, 25e-3, 22)])

uglobals.abort.clear()

for idx, t_read in enumerate(t_reads):
    t_cinit_green = 30e-6
    t_read_red = t_read

    if uglobals.abort.is_set():
        break
        
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        #optimize_position()
        laser_off()
        #"""
    
        qmeas = setup_charge_read_params(t_cinit_green=t_cinit_green, t_cread_red=t_read_red, t_wait=1e-6,
                                     laser_red_ch='d_ch3', add_gate_ch='d_ch4')

        setup['gate_channel'] = 'a_ch2'
        qmeas['bin_width'] = 4.096e-7
        setup['analog_trigger_voltage'] = 0.25

        qmeas['savetag'] = f"charge_read_t_read={1e3*t_read_red:.1f}ms_idx={idx}"
        qmeas['measurement_time'] = 5*60
        uglobals.abort.clear()
        

        run_generic(qmeas)

    except:
        logger.exception("")
        uglobals.abort.set()

qmeas

OrderedDict([('experiment', 'laser_strob'), ('name', 'charge_read'), ('listfile', True), ('t_laser_init', 3e-05), ('t_laser_read', 0.012), ('t_wait_between', 1e-06), ('t_aom_safety', 7.5e-07), ('laser_read_ch', 'd_ch3'), ('add_gate_ch', 'd_ch4'), ('measurement_time', 300), ('optimize_time', None), ('fit_experiment', None), ('bin_width', 4.096e-07), ('savetag', 'charge_read_t_read=12.0ms_idx=5'), ('gated', True), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 1), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', ''), ('gate_channel', 'a_ch2'), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 1.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', '/Dev1/PFI0'), ('readout_end', 3e-07), ('max_tau', 0.001), ('max_ta

### Plot laser trace in qdplot

In [291]:
def moving_average(a, n=3) :
    return np.convolve(a, np.ones(n)/n, mode='same')

y_laser = pulsedmeasurementlogic.laser_data.flatten()
bin_width = pulsedmasterlogic.fast_counter_settings['bin_width']
x_laser = np.arange(y_laser.size, dtype=float) * bin_width

# smooth
n_smooth = 1
y_laser = moving_average(y_laser, n=n_smooth)
# undersample
fac_us = 1
x_laser = x_laser[::fac_us]
y_laser = y_laser[::fac_us]


qdplotlogic.set_data(x_laser, y_laser)

len(x_laser), len(y_laser)

(1216, 1216)

### Charge (NV minus) init

Read the current sequence table

In [35]:
get_sequence_parameter_list('charge_read_fci')[0][1]

{'repetitons': 0, 'segment_advance_mode': 'conditional', 'event_trigger': 'OFF', 'event_jump_to': 0, 'wait_for': 'OFF', 'repetitions': 0, 'go_to': -1, 'ensemble': 'nvmin_init', 'flag_trigger': [], 'flag_high': []}

In [38]:
[(idx, entry[1]['ensemble'], f"rep={entry[1]['repetitions']}") for (idx,entry) in enumerate(get_sequence_parameter_list('charge_read_fci'))]

[(0, 'nvmin_init', 'rep=0'), (1, 'charge_read', 'rep=0'), (2, 'epoch_done', 'rep=0')]

Other than for MFL, we don't use the dynamic input port for conditional jumping. Dynamic mode doesn't allow to use advancement modes different than 'Auto' on AWG8190A. Instead use the Input Trigger as advancement event and set segment advance mode = conditional.

ATTENTION: Need to
1. Execute cell below
2. Manually, Set Fastcomtec to correct START Trig level. Using a_ch2 with max_ampl=0.25 V here!

In [65]:
awg8190.set_sample_rate(1e9)
try:
    add_fci(enable=True)
except:
    logger.exception("")
awg8190.get_sample_rate()


1000000000.0

In [143]:
# send reset to fast circuit. not needed if reset after every green-red cycle
channel_reset = '1'  # d_ch1

qmeas = {}
qmeas['experiment'] = 'n_digital_high'
qmeas['name'] = 'circuit_reset'
qmeas['digital_channels'] = channel_reset

qmeas['measurement_time'] = 2 

run_generic(qmeas)

In [68]:
# for init
t_init_red = 20e-6  # fast charge init like Hopper
t_init_red = 3e-3   # slow, pre-processing regime
# for readout
t_read_red = 15e-3 
#t_read_red = 350e-6
# TODO: Check that sequencer is reset correctly!

qmeas = setup_charge_read_fci_params(t_cinit_green=500e-9, t_cinit_red=t_init_red, 
                                     #t_cread_red=60e-6,
                                     t_cread_red=t_read_red,
                                     gate_channel='a_ch2', add_gate_ch='d_ch4')


qmeas['bin_width'] = 4.096e-7
setup['analog_trigger_voltage'] = 0.25

qmeas['savetag'] = f"charge_read_fci_t_init={1e6*t_init_red:.1f}us_t_read={1e3*t_read_red:.1f}ms"

uglobals.abort.clear()
run_generic(qmeas)

qmeas

OrderedDict([('name', 'charge_read_fci'), ('experiment', 'charge_read_fci'), ('sequence_mode', True), ('listfile', True), ('t_cinit_green', 5e-07), ('t_cinit_red', 0.005), ('t_wait_between', 1e-06), ('t_aom_safety', 7.5e-07), ('t_cread_red', 0.015), ('laser_red_ch', 'd_ch3'), ('add_gate_ch', 'd_ch4'), ('done_ch', 'd_ch1'), ('measurement_time', 9999), ('optimize_time', None), ('fit_experiment', None), ('bin_width', 4.096e-07), ('savetag', 'charge_read_fci_t_init=5000.0us_t_read=15.0ms'), ('gated', True), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 1), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', ''), ('gate_channel', 'a_ch2'), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 1.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_channel', 

In [91]:
qmeas['savetag']

'charge_read_fci_t_init=20.0us_t_read=0.0ms'

Vary t_read_red

In [84]:
t_reads

array([0.002     , 0.00214286, 0.00228571, 0.00242857, 0.00257143,
       0.00271429, 0.00285714, 0.003     ])

In [123]:
t_reads = np.linspace(2e-3, 3e-3, 8)
t_reads = np.concatenate([t_reads, np.linspace(3.2e-3, 25e-3, 22)])
t_reads

array([0.002     , 0.00214286, 0.00228571, 0.00242857, 0.00257143,
       0.00271429, 0.00285714, 0.003     , 0.0032    , 0.0042381 ,
       0.00527619, 0.00631429, 0.00735238, 0.00839048, 0.00942857,
       0.01046667, 0.01150476, 0.01254286, 0.01358095, 0.01461905,
       0.01565714, 0.01669524, 0.01773333, 0.01877143, 0.01980952,
       0.02084762, 0.02188571, 0.02292381, 0.0239619 , 0.025     ])

In [843]:
filename_default_lst = 'fastcounter.lst'
t_reads = np.linspace(2e-3, 10e-3, 6)
#t_reads = np.concatenate([t_reads, np.linspace(3.2e-3, 25e-3, 22)])

uglobals.abort.clear()

for idx, t_read in enumerate(t_reads):
    t_init_red = 20e-6
    t_init_red =  5e-3
    t_read_red = t_read

    if uglobals.abort.is_set():
        break
        
    try:
        #pass
        #"""
        laser_on()
        time.sleep(1)
        optimize_position()
        #optimize_position()
        laser_off()
        #"""
    
        qmeas = setup_charge_read_fci_params(t_cinit_green=500e-9, t_cinit_red=t_init_red, 
                                             #t_cread_red=60e-6,
                                             t_cread_red=t_read_red,
                                             gate_channel='a_ch2', add_gate_ch='d_ch4')


        qmeas['bin_width'] = 4.096e-7
        setup['analog_trigger_voltage'] = 0.25

        qmeas['savetag'] = f"charge_read_fci_t_init={1e6*t_init_red:.1f}us_t_read={1e3*t_read_red:.1f}ms"
        qmeas['savetag'] += "_thresh=6"
        qmeas['measurement_time'] = 20*60
        uglobals.abort.clear()   

        run_generic(qmeas)

        #move_list_file(qmeas)
    except:
        logger.exception("")
        uglobals.abort.set()

qmeas

OrderedDict([('name', 'charge_read_fci'), ('experiment', 'charge_read_fci'), ('sequence_mode', True), ('listfile', True), ('t_cinit_green', 5e-07), ('t_cinit_red', 0.005), ('t_wait_between', 1e-06), ('t_aom_safety', 7.5e-07), ('t_cread_red', 0.0036), ('laser_red_ch', 'd_ch3'), ('add_gate_ch', 'd_ch4'), ('done_ch', 'd_ch1'), ('measurement_time', 1200), ('optimize_time', None), ('fit_experiment', None), ('bin_width', 4.096e-07), ('savetag', 'charge_read_fci_t_init=5000.0us_t_read=3.6ms_thresh=6'), ('gated', True), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 1), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', ''), ('gate_channel', 'a_ch2'), ('laser_channel', 'd_ch2'), ('laser_length', 3e-06), ('wait_length', 1e-06), ('trigger_length', 2e-08), ('delay_length', 1.7e-07), ('channel_amp', 1.0), ('microwave_channel', 'a_ch1'), ('optimize_c

# Randomized benchmarking

In [111]:
data_dir = r"E:\Data\2023\01\20230127\rand_bench"
fname = '20230127-1553-55_rb_seq_2q.json'

with open(data_dir + "/" + fname) as f:
    qudi_gates = json.load(f)

data = parse_rbjson(qudi_gates)
0

0

In [120]:
qmeas['rotations']

In [271]:
common = common.swap_nvs([1,2])

f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]

dd_cnot_tau = 115e-9#-75.5e-9
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order =  common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


# load from manually defined gate list of lists
rots = [
        [TomoRotations.ux90_on_1, TomoRotations.ux90_on_2, TomoRotations.ux90_on_1, TomoRotations.ux90_on_2,
        TomoRotations.c2not1]]

# repetitions of cnots
rots, xticks = [[TomoRotations.none]], [0]
for idx in range(0, 10):
    n_cnots = 2*(idx+1)
    rots.append([TomoRotations.c2not1]*n_cnots)
    xticks.append(n_cnots)


qmeas = setup_rand_benchmark_params(rotations=rots, x_max=1,
                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                 tau_dd_fix=dd_cnot_tau_fix,
                                 alternating=True, charge_init=False)

qmeas['xticks'] = list_2_csv(xticks)

# load from (qiskit generated) file
data_dir =  os.path.abspath(savelogic.get_daily_directory())
data_dir = r"E:\Data\2023\01\20230127\rand_bench"
fname = '20230127-1553-55_rb_seq_2q.json'
#
#qmeas = setup_rand_benchmark_params(rotations=data_dir + "/" + fname, x_max=1,
#                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
#                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
#                                 tau_dd_fix=dd_cnot_tau_fix,
#                                 alternating=True, xticks_ncliff=True)

#qmeas['t_idle'] = 25e-9

qmeas['optimize_time'] = 15*60
qmeas['optimize_func_toggle_pause'] = toggle_pg

#qmeas['alternating'] = False
#qmeas['xticks'] = ''

uglobals.abort.clear()
run_generic(qmeas)
qmeas

#common = common.swap_nvs([1,2])
qmeas['xticks']


qmeas

OrderedDict([('name', 'rand_benchmark_2q'), ('experiment', 'rand_benchmark'), ('savetag', 'rand_benchmark'), ('microwave_amplitude', 0.25), ('microwave_frequency', 2729800000.0), ('rabi_period', 6.4e-08), ('rotations', '<TomoRotations.none: 0>; <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>; <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>; <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>; <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>; <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>, <TomoRotations.c2not1: 18>,

# Entanglement

## Tomography

In [883]:

f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]

# tomo rotattions
#init = TomoInit.ent_create_bell
init = TomoInit.none
#init = TomoInit.ux90_on_1

phase = 0
read_on_nv =1

# all needed rotations for full tomography
# tuple(on_nv, phase, rotation)
tomo_rotations = [
                  #(1, 0, TomoRotations.none),
                  #(1, 90, TomoRotations.none),
                  #(2, 0, TomoRotations.none),
                  #(2, 90, TomoRotations.none),

                  #(1, 0, TomoRotations.ux180_on_2),
                  #(1, 90, TomoRotations.ux180_on_2),
                  #(2, 0, TomoRotations.ux180_on_2),
                  #(2, 90, TomoRotations.ux180_on_2),
                  #(1, 0, TomoRotations.ux180_on_1),
                  #(1, 90, TomoRotations.ux180_on_1),
                  #(2, 0, TomoRotations.ux180_on_1),
                  #(2, 90, TomoRotations.ux180_on_1),
                  #(2, 0, TomoRotations.c2not1_ux180_on_1),
                  #(2, 90, TomoRotations.c2not1_ux180_on_1),
                  #(1, 0, TomoRotations.c2not1_ux180_on_1),
                  #(1, 90, TomoRotations.c2not1_ux180_on_1),
                  #(1, 0, TomoRotations.c1not2),
                  #(1, 90, TomoRotations.c1not2),
                  #(2, 0, TomoRotations.c1not2),
                  #(2, 90, TomoRotations.c1not2),
    
                  # for debug, not needed for reconstruction
                  (1, 0, TomoRotations.c2not1),
                  (1, 90, TomoRotations.c2not1),
                  #(2, 0, TomoRotations.c2not1),
                  #(2, 90, TomoRotations.c2not1)
                 ]
        
# for cnot


dd_cnot_tau = -100e-9#-75.5e-9
dd_cnot_tau = -tau2_pspacing/2   # first data point in calibration mes above
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


tomo_rotations = tomo_rotations[::-1]
tomo_rotations 

[(1, 90, <TomoRotations.c2not1: 6>), (1, 0, <TomoRotations.c2not1: 6>)]

In [254]:
f_deer

[2830200000.0]

In [33]:
rotation = TomoRotations.c2not1
on_nv=1
try:

    qmeas = setup_tomography_params(n_tau=1, 
                                    read_rotation=rotation, init_state=init,
                                    rabi_on_nv=on_nv, rabi_phase_deg=phase,
                                    f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                     tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                     use_shaped_cnot=False, tau_dd_fix=dd_cnot_tau_fix,
                                     alternating=True)


    #qmeas['num_of_points'] = 1

    # tomography with rect pulses, but shaped cnot
    """
    cnot_kwargs = {'env_type':Evm.parabola, 
                   'order_P': 1,
                   'tau_dd_fix': 500e-9,
                   'rabi_period': common.rabi_shaped['period'],
                   'rabi_period_mw_2': list_2_csv(common.t_rabi_2_shaped)}
    """
    #qmeas['cnot_kwargs'] = cnot_kwargs
    #qmeas['tau_cnot'] = 70e-9
except:
    logger.exception("")
    
uglobals.abort.clear()

run_generic(qmeas)
qmeas

OrderedDict([('name', 'tomo_rot=c2not1'), ('experiment', 'tomography'), ('savetag', 'tomography_on1_rot=c2not1_phi=0'), ('microwave_amplitude', 0.075), ('microwave_frequency', 2735400000.0), ('rabi_period', 1.164e-07), ('tau_start', 0.0), ('tau_step', 5.82e-09), ('rabi_on_nv', 1), ('rabi_phase_deg', 0), ('rotation', <TomoRotations.c2not1: 6>), ('init_state', <TomoInit.none: 0>), ('init_state_kwargs', {}), ('cnot_kwargs', {'tau_dd_fix': 5e-07}), ('tau_cnot', -1.93375e-07), ('dd_type_cnot', <DDMethods.XY4: [0.0, 90.0, 0.0, 90.0]>), ('dd_order', 1), ('alternating', True), ('num_of_points', 1), ('f_mw_2', '2825300000.0'), ('rabi_period_mw_2', '1.101e-07'), ('ampl_mw_2', '0.175'), ('measurement_time', None), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('analysis_interval', None), ('samplin

In [179]:
setup_tomography_params

<function setup_tomography_params at 0x0000000117743D08>

### Full tomographic set

In [281]:
tomo_inits =  [TomoInit(val) for val in TomoInit][:10][::-1][3:]
tomo_inits = [init]
tomo_inits = []
#tomo_inits += [TomoInit.none]
tomo_inits += [TomoInit.ux180_on_2]
#tomo_inits += [TomoInit.ux180_on_1]
#tomo_inits += [TomoInit.ux180_on_both]
#tomo_inits += [TomoInit.ux90_on_1]
#tomo_inits += [TomoInit.ux90_on_1_ux180_on_2]
tomo_inits

[<TomoInit.ux180_on_2: 8>]

Attention: Need to run 'Tomography' cell above, especiall if changed common params (eg. shaping off/on).

In [78]:
uglobals.abort.clear()

use_shaped_cnot = False
use_ac_stark_comp = False

ampl_acstark_comp_2on1 = 0.045
ampl_acstark_comp_1on2 = 0#0.053
df_calib_offset = -15e6
df_offset_2on1 = (ampl_acstark_comp_2on1/common.mw_amp)**2*df_calib_offset
df_offset_1on2 = (ampl_acstark_comp_1on2/common.mw_amps_2[0])**2*df_calib_offset
fac_comp = 3


#tomo_inits =  [TomoInit(val) for val in TomoInit][:10][::-1][
debug = []
tomo_rots_init = []

try:
    # Create list of measurements
    for init in tomo_inits:
        
        # reference measurements for init of any state, except 00
        if init != TomoInit.none:
            tomo_add = [
                  (1, 0, TomoRotations.none, TomoInit.none),
                  (1, 90, TomoRotations.none, TomoInit.none),  # not needed as ref, but usefull to check init state
                  (2, 0, TomoRotations.none, TomoInit.none),
                  (2, 90, TomoRotations.none, TomoInit.none)   # not needed as ref, but usefull to check init state
            ]
            #[tomo_rots_init.append(rot) for rot in tomo_add]
        
        # for 00, we add again (possibly missing) basic tomo rots. They also server as ref measurements.
        elif init == TomoInit.none:
            tomo_add = [
                  (1, 0, TomoRotations.none, TomoInit.none),
                  (1, 90, TomoRotations.none, TomoInit.none),  # must, because not only ref, but also tomography
                  (2, 0, TomoRotations.none ,TomoInit.none),
                  (2, 90, TomoRotations.none, TomoInit.none)   # must, because not only ref, but also tomography
            ]
            #[tomo_rots_init.append(rot) for rot in tomo_add if rot[0:-1] not in tomo_rotations]
        
        for (on_nv, phase,  rot) in tomo_rotations:
            tomo_rots_init.append([on_nv, phase, rot, init])
        
    # do experiments
    for (on_nv, phase,  rot, init) in tomo_rots_init:
        qmeas = setup_tomography_params(n_tau=2, 
                                    read_rotation=rot, init_state=init,
                                    rabi_on_nv=on_nv, rabi_phase_deg=phase,
                                    f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                     tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                     #tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=1,  # DEBUG ONLY
                                     use_shaped_cnot=use_shaped_cnot, tau_dd_fix=dd_cnot_tau_fix,
                                     alternating=True)

        qmeas['measurement_time'] = 5*60
        qmeas['savetag'] += f"_init_{init.name}" 
        
        if use_ac_stark_comp:
            qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp_2on1,ampl_acstark_comp_1on2], 
                                  fac_comp=fac_comp, df_offset=[df_offset_2on1,df_offset_1on2])
        
        if 'c1not2' in rot.name or 'c2not1' in rot.name:
            qmeas['measurement_time'] = 30*60

        debug.append((rot, qmeas))
        run_generic(qmeas)
            

except:
    logger.exception("")

[qm[1]['savetag'] for qm in debug]

['tomography_on1_rot=c2not1_phi=90_init_none', 'tomography_on1_rot=c2not1_phi=0_init_none', 'tomography_on1_rot=c2not1_phi=90_init_ux180_on_2', 'tomography_on1_rot=c2not1_phi=0_init_ux180_on_2']

In [2439]:
debug[-1]

(<TomoRotations.c2not1: 6>, OrderedDict([('name', 'tomo_rot=c2not1'), ('experiment', 'tomography'), ('savetag', 'tomography_on1_rot=c2not1_phi=0_init_ux180_on_both'), ('microwave_amplitude', 0.075), ('microwave_frequency', 2736500000.0), ('rabi_period', 1.162e-07), ('tau_start', 0.0), ('tau_step', 5.8099999999999996e-09), ('rabi_on_nv', 1), ('rabi_phase_deg', 0), ('rotation', <TomoRotations.c2not1: 6>), ('init_state', <TomoInit.ux180_on_both: 9>), ('init_state_kwargs', {}), ('cnot_kwargs', {'tau_dd_fix': 5e-07}), ('tau_cnot', -7.55e-08), ('dd_type_cnot', <DDMethods.KDD4: [30.0, 0.0, 90.0, 0.0, 30.0, 120.0, 90.0, 180.0, 90.0, 120.0, 30.0, 0.0, 90.0, 0.0, 30.0, 120.0, 90.0, 180.0, 90.0, 120.0]>), ('dd_order', 2), ('alternating', True), ('num_of_points', 50), ('f_mw_2', '2824600000.0'), ('rabi_period_mw_2', '1.098e-07'), ('ampl_mw_2', '0.175'), ('measurement_time', 1800), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.recta

### tomographic set for different tau_cnot

In [170]:
tomo_inits, tomo_rots_init

([<TomoInit.none: 0>, <TomoInit.ux180_on_2: 8>], [(1, 0, <TomoRotations.none: 0>, <TomoInit.none: 0>), (1, 90, <TomoRotations.none: 0>, <TomoInit.none: 0>), (2, 0, <TomoRotations.none: 0>, <TomoInit.none: 0>), (2, 90, <TomoRotations.none: 0>, <TomoInit.none: 0>), [1, 90, <TomoRotations.c2not1: 6>, <TomoInit.none: 0>], [1, 0, <TomoRotations.c2not1: 6>, <TomoInit.none: 0>], (1, 0, <TomoRotations.none: 0>, <TomoInit.none: 0>), (2, 0, <TomoRotations.none: 0>, <TomoInit.none: 0>), [1, 90, <TomoRotations.c2not1: 6>, <TomoInit.ux180_on_2: 8>], [1, 0, <TomoRotations.c2not1: 6>, <TomoInit.ux180_on_2: 8>]])

In [242]:
dd_cnot_taus = np.linspace(-170e-9, -300e-9, 6)#[::-1]
dd_cnot_taus = np.insert(dd_cnot_taus, -1, 0e-9)
dd_cnot_taus*1e9

array([-170., -196., -222., -248., -274.,    0., -300.])

In [275]:
uglobals.abort.clear()

use_shaped_cnot = False
use_ac_stark_comp = False
use_oc_both = False
dd_cnot_taus = np.linspace(-170e-9, -300e-9, 6)#[::-1]
dd_cnot_taus = np.insert(dd_cnot_taus, -1, 0e-9)
#dd_cnot_taus = np.insert(dd_cnot_taus, 0, 0e-9)

#ampl_acstark_comp_2on1 = 0.045
#ampl_acstark_comp_1on2 = 0#0.053
#df_calib_offset = -15e6
#df_offset_2on1 = (ampl_acstark_comp_2on1/common.mw_amp)**2*df_calib_offset
#df_offset_1on2 = (ampl_acstark_comp_1on2/common.mw_amps_2[0])**2*df_calib_offset
#fac_comp = 3


#tomo_inits =  [TomoInit(val) for val in TomoInit][:10][::-1][
debug = []
tomo_rots_init = []

try:
    # Create list of measurements
    for init in tomo_inits:
        
        # reference measurements for init of any state, except 00
        if init != TomoInit.none:
            tomo_add = [
                  (1, 0, TomoRotations.none, TomoInit.none),
                  #(1, 90, TomoRotations.none, TomoInit.none),  # not needed as ref, but usefull to check init state
                  (2, 0, TomoRotations.none, TomoInit.none),
                  #(2, 90, TomoRotations.none, TomoInit.none)   # not needed as ref, but usefull to check init state
            ]
            [tomo_rots_init.append(rot) for rot in tomo_add]
        
        # for 00, we add again (possibly missing) basic tomo rots. They also server as ref measurements.
        elif init == TomoInit.none:
            tomo_add = [
                  (1, 0, TomoRotations.none, TomoInit.none),
                  (1, 90, TomoRotations.none, TomoInit.none),  # must, because not only ref, but also tomography
                  (2, 0, TomoRotations.none ,TomoInit.none),
                  (2, 90, TomoRotations.none, TomoInit.none)   # must, because not only ref, but also tomography
            ]
            [tomo_rots_init.append(rot) for rot in tomo_add if rot[0:-1] not in tomo_rotations]
        
        for (on_nv, phase,  rot) in tomo_rotations:
            tomo_rots_init.append([on_nv, phase, rot, init])
        
    # do experiments
    for tau_cnot in dd_cnot_taus:
        for (on_nv, phase,  rot, init) in tomo_rots_init:
            qmeas = setup_tomography_params(n_tau=50, 
                                        read_rotation=rot, init_state=init,
                                        rabi_on_nv=on_nv, rabi_phase_deg=phase,
                                        f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                         tau_cnot=tau_cnot, dd_type_cnot=dd_type, dd_order=dd_order,
                                         #tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=1,  # DEBUG ONLY
                                         use_shaped_cnot=use_shaped_cnot, tau_dd_fix=dd_cnot_tau_fix,
                                         alternating=True)

            qmeas['measurement_time'] = 2*60
            qmeas['savetag'] += f"_init_{init.name}_tau_cnot={1e9*tau_cnot:.1f}ns" 

            if use_ac_stark_comp:
                qmeas = add_ac_stark_compensation(qmeas, ampl_comp=[ampl_acstark_comp_2on1,ampl_acstark_comp_1on2], 
                                      fac_comp=fac_comp, df_offset=[df_offset_2on1,df_offset_1on2])

            if 'c1not2' in rot.name or 'c2not1' in rot.name:
                qmeas['measurement_time'] = 10*60
                if use_oc_both:
                    qmeas['cnot_kwargs']['env_type'] = Evm.optimal

            debug.append((rot, qmeas))
            
            #continue
            laser_on()
            time.sleep(1)
            optimize_position()
            #optimize_position()
            laser_off()
            
            run_generic(qmeas)
            
            

except:
    logger.exception("")

[qm[1]['savetag'] for qm in debug][:10]

Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A974838410>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A974838468>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A974838410>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator o

    yield lh_compliment, rh_compliment + self._b
SystemError: error return without exception set
Exception ignored in: <generator object Transform._iter_break_from_left_to_right at 0x000000A974838A40>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 1324, in _iter_break_from_left_to_right
    yield IdentityTransform(), self
SystemError: error return without exception set
Exception ignored in: <generator object CompositeGenericTransform._iter_break_from_left_to_right at 0x000000A974838360>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 2457, in _iter_break_from_left_to_right
    yield lh_compliment, rh_compliment + self._b
SystemError: error return without exception set
Exception ignored in: <generator object Transform._iter_break_from_left_to_right at 0x000000A974838360>
Traceback (most recent call last):
  File "C:\ProgramData\

Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000A974838E60>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set
Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000A974838F10>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set
Exception ignored in: <generator object Gcf.destroy_fig.<locals>.<genexpr> at 0x000000A902C42BF8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\_pylab_helpers.py", line 67, in <genexpr>
    if manager.canvas.figure == fig), N

    yield IdentityTransform(), self
SystemError: error return without exception set
Exception ignored in: <generator object CompositeGenericTransform._iter_break_from_left_to_right at 0x000000A974838FC0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 2457, in _iter_break_from_left_to_right
    yield lh_compliment, rh_compliment + self._b
SystemError: error return without exception set
Exception ignored in: <generator object Transform._iter_break_from_left_to_right at 0x000000A974838F68>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 1324, in _iter_break_from_left_to_right
    yield IdentityTransform(), self
SystemError: error return without exception set
Exception ignored in: <generator object CompositeGenericTransform._iter_break_from_left_to_right at 0x000000A974838FC0>
Traceback (most recent call last):
  File "C:\ProgramDa

Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000B438C30150>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000B438C30150>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000B438C303B8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set

...

    yield IdentityTransform(), self
SystemError: error return without exception set
Exception ignored in: <generator object CompositeGenericTransform._iter_break_from_left_to_right at 0x000000A974838F68>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 2457, in _iter_break_from_left_to_right
    yield lh_compliment, rh_compliment + self._b
SystemError: error return without exception set
Exception ignored in: <generator object Transform._iter_break_from_left_to_right at 0x000000A974838F68>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\transforms.py", line 1324, in _iter_break_from_left_to_right
    yield IdentityTransform(), self
SystemError: error return without exception set
Exception ignored in: <generator object CompositeGenericTransform._iter_break_from_left_to_right at 0x000000A9748380F8>
Traceback (most recent call last):
  File "C:\ProgramDa

Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000A974838D58>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A9748380F8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A9748380F8>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set

    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A902C42A98>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A902C42308>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A902C42308>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in s

Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000A902C422B0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set
Exception ignored in: <generator object ScalarFormatter._compute_offset.<locals>.<genexpr> at 0x000000A902C422B0>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\matplotlib\ticker.py", line 696, in <genexpr>
    if abs_min // 10 ** oom != abs_max // 10 ** oom)
SystemError: error return without exception set
Exception ignored in: <generator object Cycler.__iter__.<locals>.<genexpr> at 0x000000A974838F68>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\qudi\lib\site-packages\cycler.py", line 227, in <genexpr>
    return iter(dict(l) for l in self._left)
SystemError:

['tomography_on1_rot=none_phi=0_init_none_tau_cnot=-170.0ns', 'tomography_on1_rot=none_phi=90_init_none_tau_cnot=-170.0ns', 'tomography_on2_rot=none_phi=0_init_none_tau_cnot=-170.0ns', 'tomography_on2_rot=none_phi=90_init_none_tau_cnot=-170.0ns', 'tomography_on1_rot=c2not1_phi=90_init_none_tau_cnot=-170.0ns', 'tomography_on1_rot=c2not1_phi=0_init_none_tau_cnot=-170.0ns', 'tomography_on1_rot=none_phi=0_init_none_tau_cnot=-170.0ns']

In [266]:
debug[0][1]

OrderedDict([('name', 'tomo_rot=none'), ('experiment', 'tomography'), ('savetag', 'tomography_on1_rot=none_phi=0_init_none_tau_cnot=-170.0ns'), ('microwave_amplitude', 0.125), ('microwave_frequency', 2729600000.0), ('rabi_period', 8.8e-08), ('tau_start', 0.0), ('tau_step', 4.4e-09), ('rabi_on_nv', 1), ('rabi_phase_deg', 0), ('rotation', <TomoRotations.none: 0>), ('init_state', <TomoInit.none: 0>), ('init_state_kwargs', {}), ('cnot_kwargs', {'tau_dd_fix': 1e-06}), ('tau_cnot', -1.7e-07), ('dd_type_cnot', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 2), ('alternating', True), ('num_of_points', 50), ('f_mw_2', '2830200000.0'), ('rabi_period_mw_2', '9.53e-08'), ('ampl_mw_2', '0.25'), ('measurement_time', 120), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('a

## Tomography with single points (no Rabi)

In [23]:
f_deer = [common.mw_freq_2[0]]
ampl_2 = common.mw_amps_2[0]
t_rabi_2 = common.rabi_period_2[0]


dd_cnot_tau = -225e-9#-75.5e-9
dd_cnot_tau_fix = common.dd_tau  # set 'None' for deer_dd_par_tau
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8

dd_type, dd_order, dd_cnot_tau

(<DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>, 2, -2.25e-07)

benchmark c2not1 with all inits

In [368]:
rots = [TomoRotations.c2not1, TomoRotations.c2not1, TomoRotations.c2not1, TomoRotations.c2not1]
#rots = [TomoRotations.none, TomoRotations.none, TomoRotations.none, TomoRotations.none]
inits = [TomoInit.none, TomoInit.ux180_on_1, TomoInit.ux180_on_2, TomoInit.ux180_on_both]


try:
    qmeas = setup_tomography_single_params(
                                 read_rotation=rots, init_state=inits,
                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                 tau_dd_fix=dd_cnot_tau_fix,
                                 alternating=True)
        
    qmeas['cnot_kwargs']['env_type'] = Evm.optimal
    
    uglobals.abort.clear() 
    run_generic(qmeas)
except:
    logger.exception("")
    

qmeas

OrderedDict([('name', 'tomography_single'), ('experiment', 'tomography_single'), ('savetag', 'tomography_single'), ('microwave_amplitude', 0.083328), ('microwave_frequency', 2729700000.0), ('rabi_period', 6.122e-08), ('rotations', '[<TomoRotations.c2not1: 6>, <TomoRotations.c2not1: 6>, <TomoRotations.c2not1: 6>, <TomoRotations.c2not1: 6>]'), ('init_states', '[<TomoInit.none: 0>, <TomoInit.ux180_on_1: 7>, <TomoInit.ux180_on_2: 8>, <TomoInit.ux180_on_both: 9>]'), ('init_state_kwargs', {}), ('cnot_kwargs', {'tau_dd_fix': 1e-06, 'env_type': <EnvelopeMethods.optimal: 'OptimalControl'>}), ('tau_cnot', -2.25e-07), ('dd_type_cnot', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 2), ('alternating', True), ('f_mw_2', '2825300000.0'), ('rabi_period_mw_2', '6.14e-08'), ('ampl_mw_2', '0.18737600000000001'), ('measurement_time', None), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('g

In [638]:
qmeas

In [568]:
qmeas['experiment'] = 'tomography_single'

In [35]:
rots = [TomoRotations.none, TomoRotations.none]
inits = [TomoInit.ux180_on_both, TomoInit.ux180_on_both]


try:
    qmeas = setup_tomography_single_params(
                                 read_rotation=rots, init_state=inits,
                                 f_mw_2=f_deer, ampl_mw_2=ampl_2, rabi_period_mw_2=t_rabi_2,
                                 tau_cnot=dd_cnot_tau, dd_type_cnot=dd_type, dd_order=dd_order,
                                 tau_dd_fix=dd_cnot_tau_fix,
                                 alternating=True)
        
    qmeas['init_state_kwargs']['env_type'] = Evm.optimal
    
    uglobals.abort.clear() 
    run_generic(qmeas)
except:
    logger.exception("")
    

qmeas

OrderedDict([('name', 'tomography_single'), ('experiment', 'tomography_single'), ('savetag', 'tomography_single'), ('microwave_amplitude', 0.083328), ('microwave_frequency', 2729700000.0), ('rabi_period', 6.122e-08), ('rotations', '[<TomoRotations.none: 0>, <TomoRotations.none: 0>]'), ('init_states', '[<TomoInit.ux180_on_both: 9>, <TomoInit.ux180_on_both: 9>]'), ('init_state_kwargs', {'env_type': <EnvelopeMethods.optimal: 'OptimalControl'>}), ('cnot_kwargs', {'tau_dd_fix': 1e-06}), ('tau_cnot', -2.25e-07), ('dd_type_cnot', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 2), ('alternating', True), ('f_mw_2', '2825300000.0'), ('rabi_period_mw_2', '6.14e-08'), ('ampl_mw_2', '0.18737600000000001'), ('measurement_time', None), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cyc

## Create Bell

In [399]:
# drive all HFS lines
common.mw_freq = 2736.2e6
common.mw_amp = 0.25/(2*1.3)
common.rabi_period_rect = 131.42e-9

mw_freq_nv2 = 2820.59e6
mw_amp_nv2 = 0.25/1.6
t_rabi_nv2 = 131.8e-9#common.rabi_period

ampl_mw2 = [mw_amp_nv2]#[common.mw_amp]
f_mw2 = [mw_freq_nv2]

np.sum([common.mw_amp] + ampl_mw2)


0.25240384615384615

7.09e-08

In [449]:
tau_start = np.max([t_rabi_nv2, common.rabi_period])/2 + 5e-9

# drive only one, SQT
qmeas = setup_ent_create_bell_params(tau_start=tau_start, tau_step=20e-9, n_tau=75,
                             f_mw_1=[], f_mw_2=[], ampl_mw_2=[], rabi_period_mw_2=[],
                             dd_type=DDMethods.XY8, dd_order=4, alternating=True)

# drive two NVs, SQT
qmeas = setup_ent_create_bell_params(tau_start=tau_start, tau_step=20e-9, n_tau=60,
                             f_mw_1=[], f_mw_2=f_mw2, ampl_mw_2=ampl_mw2, rabi_period_mw_2=[t_rabi_nv2],
                             dd_type=DDMethods.XY8, dd_order=4, alternating=True)

uglobals.abort.clear()
run_generic(qmeas)

qmeas

OrderedDict([('name', 'ent_create_bell'), ('experiment', 'ent_create_bell'), ('microwave_amplitude', 0.09615384615384615), ('microwave_frequency', 2736200000.0), ('rabi_period', 1.3142e-07), ('tau_start', 7.09e-08), ('tau_step', 2e-08), ('read_phase_deg', 90), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 4), ('alternating', True), ('num_of_points', 60), ('f_mw_2', '2820590000.0'), ('rabi_period_mw_2', '1.318e-07'), ('ampl_mw_2', '0.15625'), ('measurement_time', None), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('savetag', 'ent_create_bell'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel', ''

## Bell Ramsey

In [86]:
tau_cnot = -300e-9

f_mw2 = common.mw_freq_2[0]
ampl_mw2 = common.mw_amps_2[0]
t_rabi_nv2 = common.rabi_period_2[0]
dd_cnot_tau_fix = common.dd_tau
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


In [90]:

# drive two NVs, SQT
try:
    qmeas = setup_bell_ramsey_params(tau_start=10e-9, tau_step=2e-9, n_tau=20, tau_cnot=tau_cnot,
                                     tau_dd_fix = dd_cnot_tau_fix,
                                 f_mw_1=[], f_mw_2=[f_mw2], ampl_mw_2=ampl_mw2, rabi_period_mw_2=[t_rabi_nv2],
                                 dd_type=dd_type, dd_order=dd_order, alternating=True)
except:
    logger.exception("")
uglobals.abort.clear()
run_generic(qmeas)

qmeas

OrderedDict([('name', 'bell_ramsey'), ('experiment', 'bell_ramsey'), ('microwave_amplitude', 0.083328), ('microwave_frequency', 2729700000.0), ('rabi_period', 6.122e-08), ('tau_start', 1e-08), ('tau_step', 2e-09), ('tau_cnot', -3e-07), ('cnot_kwargs', {'tau_dd_fix': 1e-06}), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 2), ('alternating', True), ('num_of_points', 50), ('f_mw_2', '2825300000.0'), ('rabi_period_mw_2', '6.14e-08'), ('ampl_mw_2', '0.18737600000000001'), ('measurement_time', None), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('savetag', 'bell_ramsey'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'

## Bell Hahn

In [93]:
tau_cnot = -125e-9
use_oc_both = True

f_mw2 = common.mw_freq_2[0]
ampl_mw2 = common.mw_amps_2[0]
t_rabi_nv2 = common.rabi_period_2[0]
dd_cnot_tau_fix = common.dd_tau
dd_order = common.dd_order #15
dd_type =  common.dd_type #DDMethods.SE   #DDMethods.XY8


In [98]:

# drive two NVs, SQT
try:
    qmeas = setup_bell_hahnecho_params(tau_start=10e-9, tau_step=10e-9, n_tau=20, tau_cnot=tau_cnot,
                                     tau_dd_fix = dd_cnot_tau_fix,
                                 f_mw_1=[], f_mw_2=[f_mw2], ampl_mw_2=ampl_mw2, rabi_period_mw_2=[t_rabi_nv2],
                                 dd_type=dd_type, dd_order=dd_order, alternating=True)
    if use_oc_both:
        qmeas['cnot_kwargs']['env_type'] = Evm.optimal
except:
    logger.exception("")

uglobals.abort.clear()
run_generic(qmeas)

qmeas

OrderedDict([('name', 'bell_hahn'), ('experiment', 'bell_hahnecho'), ('microwave_amplitude', 0.083328), ('microwave_frequency', 2729700000.0), ('rabi_period', 6.122e-08), ('tau_start', 1e-08), ('tau_step', 1e-08), ('tau_cnot', -1.25e-07), ('cnot_kwargs', {'tau_dd_fix': 5e-07, 'env_type': <EnvelopeMethods.optimal: 'OptimalControl'>}), ('dd_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('dd_order', 4), ('alternating', True), ('num_of_points', 20), ('f_mw_2', '2825300000.0'), ('rabi_period_mw_2', '6.14e-08'), ('ampl_mw_2', '0.18737600000000001'), ('measurement_time', None), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'decay_exp'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('savetag', 'bell_hahn'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('analysis_interval', None), ('sampling_freq', 12000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_de

# Auto pilot multiple NVs

## define pois

In [25]:
opt_on_poi =  poimanagerlogic.poi_names[1]  #poimanagerlogic.poi_names[7]
opt_on_poi



'nv_20221109_2'

In [21]:
pois_double, pois_special = [],[]
# south
# need to redo xy8
for idx in [42, 45, 28, 39, 8, 20, 43, 25, 27, 18]:
    pois_double.append(poimanagerlogic.poi_names[int(idx)-1])
# north
#for idx in [28, 39, 84, 101, 104]:
#    pois_double.append(poimanagerlogic.poi_names[int(idx)+53])
#pois_double

# for DEER
pois_special = []
# south
#25, 43, 20, 8, 53, 2, 30, 29, 24, 31, 47, 14, 26, 33, 23, 58, 60, 48, 
# 18, 41, 55
for idx in [80]:
    pois_special.append(poimanagerlogic.poi_names[int(idx)-1])
# north
#for idx in [90, 108, 67, 84]:
#    pois_special.append(poimanagerlogic.poi_names[int(idx)+53])
pois_special

In [ ]:
"21_1",  "20_7", "20_2", "20_4"

Rename pois (here swapped day and month in POI string)

In [35]:
[poimanagerlogic.rename_poi(p.replace('0301','0103'),p) for p in poimanagerlogic.poi_names[65:]]

In [167]:
poimanagerlogic.poi_names[9:]

['poi_20221110091743937079', 'poi_20221110091830281709', 'poi_20221110091901440047', 'poi_20221110091922399905', 'poi_20221110091949488227', 'poi_20221110091959556080', 'poi_20221110092022494899', 'poi_20221110092034402210', 'poi_20221110092053527107', 'poi_20221110092112523418', 'poi_20221110092138400495', 'poi_20221110092205784094', 'poi_20221110092220145847', 'poi_20221110092234500747', 'poi_20221110092247967979']

In [164]:
[f"nv_20221109_{i+10}" for i, p in enumerate(poimanagerlogic.poi_names[9:]) if 'poi' in p]

['nv_20221109_10', 'nv_20221109_11', 'nv_20221109_12', 'nv_20221109_13', 'nv_20221109_14', 'nv_20221109_15', 'nv_20221109_16', 'nv_20221109_17', 'nv_20221109_18', 'nv_20221109_19', 'nv_20221109_20', 'nv_20221109_21', 'nv_20221109_22', 'nv_20221109_23', 'nv_20221109_24']

In [170]:
[poimanagerlogic.rename_poi(f"nv_20221109_{i+10}",p) for i, p in enumerate(poimanagerlogic.poi_names[9:]) if 'poi' in p]

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]

In [200]:
# CAREFUL: might be buggy
[poimanagerlogic.rename_poi(f"nv_mayd_20220103_{i+17}b",p) for i, p in enumerate(poimanagerlogic.poi_names[-20:-10])]
#[(f"nv_mayd_20220103_{i+17}a") for i, p in enumerate(poimanagerlogic.poi_names[80:90])]

[None, None, None, None, None, None, None, None, None, None]

'nv_20220221_08'

['nv_20220221_80']

## pODMR (with adjusted Rabi times) for 4 NV orientations

['nv_20220221_80']

In [35]:
uglobals.abort.clear()

# for pois=None, all experiment will run on the current confocal pos
pois = None
pois = poimanagerlogic.poi_names[:]
#pois = [poimanagerlogic.poi_names[120]]#,
       # poimanagerlogic.poi_names[3+1],
       # poimanagerlogic.poi_names[3+3],
       # poimanagerlogic.poi_names[3+5]]
opt_on_poi = poimanagerlogic.poi_names[0]

logger.info("Autopilot is taking of for pois {}".format(pois))

common.rabi_period_rect = 75.0e-9 
f_center = 2.75e9
f_step_odmr_fine = 2e6
n_f_odmr_fine = 150
assume_double_nv = True

qmeas_list = []

#qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=200)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=3000e3, n_f=100)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_coarse'
qexp['measurement_time'] = 60#200
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
if assume_double_nv:
    qexp['fit_experiment'] = 'lor_dip_2x'
    qexp['fit_parameter'] = 'Position 0'
    
qexp['update_parameters'] = {'rabi': 'microwave_frequency',
                             #'Hahn_Echo': 'microwave_frequency',
                             #'Ramsey': 'microwave_frequency',
                             #'podmr_fine': {'target_name': 'freq_start',
                             #               'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"
                             #              }
                            }
if assume_double_nv:
    qexp['update_parameters'] = {'rabi': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 0']['value']"},
                                 'rabi_B': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 1']['value']"},
                             #'Hahn_Echo': 'microwave_frequency',
                             #'Ramsey': 'microwave_frequency',
                             #'podmr_fine': {'target_name': 'freq_start',
                             #               'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"
                             #              }
                              }

qmeas_list.append(copy.deepcopy(qexp))
#"""

#Experiment2
#"""
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['measurement_time'] = 60#60#2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {'Ramsey': 'rabi_period',
                             'Hahn_Echo': 'rabi_period',
                             'podmr_fine': 'rabi_period'}  # feed forware parameters

qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_B'
    qexp['measurement_time'] = 60#60#2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'
    qexp['update_parameters'] = {'Ramsey': 'rabi_period',
                                 'Hahn_Echo': 'rabi_period',
                                 'podmr_fine': 'rabi_period'}  # feed forware parameters

    qmeas_list.append(copy.deepcopy(qexp))
                                 
                                 
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_fine'
qexp['measurement_time'] = 60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'Rabi': 'microwave_frequency',
                             'Hahn_Echo': 'microwave_frequency',
                             'Ramsey': 'microwave_frequency',
                              }
#qmeas_list.append(copy.deepcopy(qexp))

"""
#Experiment3
qexp = setup_ramsey_params(tau_start=50e-9, n_tau=400, tau_step=50e-9)  # res= 50 kHz 
qexp['experiment'] = 'ramsey'
qexp['measurement_time'] = 120
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
qexp['fit_experiment'] = 'double_sine_decay'
qmeas_list.append(copy.deepcopy(qexp))

"""

laser_off()

try:
    #run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False, poi_list=pois)
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=True,
                        opt_on_poi=opt_on_poi,  # NV as reference
    #                    poi_list=pois)
    pass

except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))




In [31]:
qmeas_list[1]

OrderedDict([('name', 'rabi'), ('experiment', 'rabi'), ('rabi_period', 8.5e-08), ('tau_step', 4.249999999999999e-09), ('tau_start', 0.0), ('alternating', False), ('leave_out_tau_idx', ''), ('num_of_points', 100), ('microwave_frequency', 3022400000.0), ('microwave_amplitude', 0.25), ('measurement_time', 60), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.rectangle: 'Sin'>), ('type', 'rabi'), ('meas_type', <function conventional_measurement at 0x0000001BC10D5510>), ('meas_info', <function add_conventional_information at 0x0000001BBCB65D90>), ('fit_parameter', 'Period'), ('update_parameters', {'Ramsey': 'rabi_period', 'Hahn_Echo': 'rabi_period', 'podmr_fine': 'rabi_period'})])

['nv_20220228_43', 'nv_20220228_20', 'nv_20220228_8', 'nv_20220228_19', 'nv_20220228_53', 'nv_20220228_2', 'nv_20220228_30', 'nv_20220228_29', 'nv_20220228_24', 'nv_20220228_31', 'nv_20220228_47', 'nv_20220228_14', 'nv_20220228_26', 'nv_20220228_33', 'nv_20220228_23', 'nv_20220228_58', 'nv_20220228_60', 'nv_20220228_48', 'nv_20220228_66']

In [31]:
opt_on_poi = poimanagerlogic.poi_names[10]
opt_on_poi

'nv_20220221_12'

In [89]:
pois = poimanagerlogic.poi_names[8:]
pois

['nv_20220228_9', 'nv_20220228_10', 'nv_20220228_11', 'nv_20220228_12', 'nv_20220228_13', 'nv_20220228_14', 'nv_20220228_15', 'nv_20220228_16', 'nv_20220228_17', 'nv_20220228_18', 'nv_20220228_19', 'nv_20220228_20', 'nv_20220228_21', 'nv_20220228_22', 'nv_20220228_23', 'nv_20220228_24', 'nv_20220228_25', 'nv_20220228_26', 'nv_20220228_27', 'nv_20220228_28', 'nv_20220228_29', 'nv_20220228_30', 'nv_20220228_31', 'nv_20220228_32', 'nv_20220228_33', 'nv_20220228_34', 'nv_20220228_35', 'nv_20220228_36', 'nv_20220228_37', 'nv_20220228_38', 'nv_20220228_39', 'nv_20220228_40', 'nv_20220228_41', 'nv_20220228_42', 'nv_20220228_43', 'nv_20220228_44', 'nv_20220228_45', 'nv_20220228_46', 'nv_20220228_47', 'nv_20220228_48', 'nv_20220228_49', 'nv_20220228_50', 'nv_20220228_51', 'nv_20220228_52', 'nv_20220228_53', 'nv_20220228_54', 'nv_20220228_55', 'nv_20220228_56', 'nv_20220228_57', 'nv_20220228_58', 'nv_20220228_59', 'nv_20220228_60', 'nv_20220228_61', 'nv_20220228_62', 'nv_20220228_63', 'nv_202202

## T2 char script

If NV counts while pulsed mes low, make sure tilt correction is off before running!

In [ ]:
common.rabi_period

In [123]:
poimanagerlogic.poi_names[20:]

['nv_20221109_21', 'nv_20221109_22', 'nv_20221109_23', 'nv_20221109_24']

In [27]:
# In order to stop the automized measurement insert pulsedmasterlogic.break_variable = True into manager
uglobals.abort.clear()

# for pois=None, all experiment will run on the current confocal pos
#pois = None
# start from south
#pois = poimanagerlogic.poi_names[9:]
# start with potential doubles, then rest
#pois = (pois_double + poimanagerlogic.poi_names[12+9:])
#pois = pois_special
pois =  poimanagerlogic.poi_names[13:]#[::-1]
opt_on_poi = opt_on_poi
opt_on_poi  =  poimanagerlogic.poi_names[0]

logger.info("Autopilot is taking of for pois {}".format(pois))

common.rabi_period_rect = 70e-9#85.0e-9 
common.mw_amp = 0.25
f_center = 2.750e9
f_step_odmr_fine = 1.0e6  # make tight to avoid fitting the wrong NV
n_f_odmr_fine = 50
p_scale_odmr_fine = 1
assume_double_nv = False

# if rabi frequencies are known already and roughly matched 
#common.rabi_period_rect = 131e-9#85.0e-9 
#common.mw_amp = 0.25/(2*1.3)
mw_amp_nv2 = None
#mw_amp_nv2 = 0.25/1.6


qmeas_list = []

#qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=200)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=3000e3, n_f=80)
qexp =  setup_fake_cwodmr_params(f_res=f_center, f_step=3e6, n_f=80, t_single=10e-6, mw_ampl=50e-3)

qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_coarse'
qexp['measurement_time'] = 1*60  # 6*60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
if assume_double_nv:
    qexp['fit_experiment'] = 'lor_dip_2x'
    qexp['fit_parameter'] = 'Position 0'
#qexp['fit_experiment'] = 'lor_dip'
#qexp['fit_parameter'] = 'Position 0'
qexp['update_parameters'] = {'rabi': 'microwave_frequency',
                             #'Hahn_Echo': 'microwave_frequency',
                             #'Ramsey': 'microwave_frequency',
                             'podmr_fine': {'target_name': 'freq_start',
                                          'func': f"fit_result.result_str_dict['Position']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"},
                                           
                            }
if assume_double_nv:
    qexp['update_parameters'] = {'rabi': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 0']['value']"},
                                 'rabi_B': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 1']['value']"},
                                 'podmr_fine': {'target_name': 'freq_start',
                                          'func': f"fit_result.result_str_dict['Position 0']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"},
                                 'podmr_fine_B': {'target_name': 'freq_start',
                                  'func': f"fit_result.result_str_dict['Position 1']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"}
                                }
qmeas_list.append(cp.deepcopy(qexp))
#"""

#Experiment2
#"""
qexp = setup_rabi_params(tau_step=1*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['measurement_time'] = 1*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                             #'Hahn_Echo': 'rabi_period',
                             'podmr_fine': 'rabi_period'}  # feed forware parameters

qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=1*common.rabi_period/50, alternating=False, n_tau=100)
    if mw_amp_nv2:
        qexp['microwave_amplitude'] = mw_amp_nv2
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_B'
    qexp['measurement_time'] = 1*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'

    qexp['update_parameters'] = {#'Ramsey_B': 'rabi_period',
                                 #'Hahn_Echo_B': 'rabi_period',
                                 'podmr_fine_B': 'rabi_period'}  # feed forware parameters

    
    qmeas_list.append(cp.deepcopy(qexp))

# next Exp
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=p_scale_odmr_fine, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp =  setup_podmr_params(f_res=f_center, p_scale=p_scale_odmr_fine, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_fine'
qexp['measurement_time'] = 1*60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'rabi_fine': 'microwave_frequency',
                             'ramsey': 'microwave_frequency',
                             'hahn_echo': 'microwave_frequency',
                             'xy8_n': 'microwave_frequency',
                              }
qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=p_scale_odmr_fine, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
    qexp =  setup_podmr_params(f_res=f_center, p_scale=p_scale_odmr_fine, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
    if mw_amp_nv2:
        qexp['microwave_amplitude'] = mw_amp_nv2
    qexp['type'] = 'pulsedodmr'
    qexp['name'] = 'podmr_fine_B'
    qexp['measurement_time'] = 1 *60
    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None
    qexp['generate_new'] = True
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information
    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'lor_dip'
    qexp['fit_parameter'] = 'Position'
    qexp['update_parameters'] = {'rabi_fine_B': 'microwave_frequency',
                                 'ramsey': 'microwave_frequency',
                                 'hahn_echo_B': 'microwave_frequency',
                                 'xy8_n_B': 'microwave_frequency',
                                  }
    qmeas_list.append(cp.deepcopy(qexp))

    
# next Exp
qexp = setup_rabi_params(tau_step=1*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['name'] = 'rabi_fine'
qexp['measurement_time'] = 1*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {'ramsey': 'rabi_period',
                             'hahn_echo': 'rabi_period',
                             'xy8_n': 'rabi_period'  # feed forware parameters
                            }
qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=1*common.rabi_period/50, alternating=False, n_tau=100)
    if mw_amp_nv2:
        qexp['microwave_amplitude'] = mw_amp_nv2
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_fine_B'
    qexp['measurement_time'] = 1*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'
    qexp['update_parameters'] = {'ramsey_B': 'rabi_period',
                                 'hahn_echo_B': 'rabi_period',
                                 'xy8_n_B': 'rabi_period'  # feed forware parameters
                            }
    qmeas_list.append(cp.deepcopy(qexp))

# next Exp Ramsey
ramsey_n_tau = 75
ramsey_tau_step = 25e-9
qmeas = setup_ramsey_params(ramsey_tau_step, n_tau=ramsey_n_tau, tau_step=ramsey_tau_step, alternating=True)
qexp['type'] = 'Ramsey'
qexp['name'] = 'ramsey'
qexp['measurement_time'] = 5*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

qexp['fit_experiment'] = 'sine_33'
#qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qmeas = setup_ramsey_params(ramsey_tau_step, n_tau=ramsey_n_tau, tau_step=ramsey_tau_step, alternating=True)
    qexp['type'] = 'Ramsey'
    qexp['name'] = 'ramsey_B'
    qexp['measurement_time'] = 5*60  #15
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    qexp['fit_experiment'] = 'sine_33'
    #qmeas_list.append(cp.deepcopy(qexp))



# next Exp Hahn     
he_tau_step = 5*100e-9
he_n_tau = 50
#he_n_tau = 50
qexp = setup_hahn_params(tau_start=10e-9, n_tau=he_n_tau, tau_step=he_tau_step, alternating=True)
qexp['type'] = 'hahn_echo'
qexp['name'] = 'hahn_echo'
qexp['measurement_time'] = 3*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

qexp['fit_experiment'] = 'decay_exp'
qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    #he_tau_step = 50e-9 #400e-9
    #he_n_tau = 50
    #he_n_tau = 50
    qexp = setup_hahn_params(tau_start=10e-9, n_tau=he_n_tau, tau_step=he_tau_step, alternating=True)
    qexp['type'] = 'hahn_echo'
    qexp['name'] = 'hahn_echo_B'
    qexp['measurement_time'] = 3*60 #15
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    qexp['fit_experiment'] = 'decay_exp'
    qmeas_list.append(cp.deepcopy(qexp))
    

# next Exp XY8

# MUST not be shorter than any pi pulse!
xy_tau = 200e-9 # 200e-9#440.0e-9

qexp = setup_xy8_nsweep_params(xy_tau, n_n=40, xy8_start=1, xy8_step=1, alternating=True)  
qexp['type'] = 'xy8_nsweep'
qexp['name'] = 'xy8_n'
qexp['measurement_time'] = 12*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

qexp['fit_experiment'] = 'decay_exp'
#qmeas_list.append(cp.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_xy8_nsweep_params(xy_tau, n_n=40, xy8_start=1, xy8_step=2, alternating=True)  
    qexp['type'] = 'xy8_nsweep'
    qexp['name'] = 'xy8_n_B'
    qexp['measurement_time'] = 12*60 #15
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    qexp['fit_experiment'] = 'decay_exp'
    #qmeas_list.append(cp.deepcopy(qexp))

# next Exp
#

    
    
    
laser_off()
try:
    #run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False, poi_list=pois)
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=True,
                        opt_on_poi=opt_on_poi,  # reference for sample shift (on stable, unique NV)
                        poi_list=pois)
    #pass
except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))




In [71]:
qmeas_list[-5]

OrderedDict([('name', 'rabi_fine_B'), ('experiment', 'rabi_shaped'), ('rabi_period', 1.83e-08), ('tau_step', 9.149999999999999e-10), ('tau_start', 0.0), ('alternating', False), ('leave_out_tau_idx', ''), ('num_of_points', 100), ('microwave_frequency', 3085400000.0), ('microwave_amplitude', 0.25), ('measurement_time', 120), ('optimize_time', None), ('freq_optimize_time', None), ('fit_experiment', 'sine'), ('env_type', <EnvelopeMethods.parabola: 'SinEnvelopeParabolaN'>), ('order_P', 1), ('type', 'rabi'), ('meas_type', <function conventional_measurement at 0x000000B108E1BB70>), ('meas_info', <function add_conventional_information at 0x000000B108E1B620>), ('fit_parameter', 'Period'), ('update_parameters', {'hahn_echo_B': 'rabi_period', 'xy8_n_B': 'rabi_period'})])

## DEER

DEER DD requires shaped pulses that are currently not implemented for xy8 sequences. Thus, we have to run the calibration completly again!

In [35]:
pois_special[:]

['nv_20220221_80']

In [61]:
if common.rabi_shaped['shaped_on'] == False:
    raise ValueError("DEER DD only supports shaped pulses.")

import copy
uglobals.abort.clear()

# for pois=None, all experiment will run on the current confocal pos
pois = None
# start from south
#pois = poimanagerlogic.poi_names[10:]
# start with potential doubles, then rest
#pois = (pois_double + poimanagerlogic.poi_names[12+9:])
pois = pois_special
opt_on_poi = opt_on_poi #poimanagerlogic.poi_names[64]

logger.info("Autopilot is taking of for pois {}".format(pois))

common.rabi_period_rect = 90e-9#85.0e-9 
f_center = 2.780e9
f_step_odmr_fine = 0.5e6  # make tight to avoid fitting the wrong NV
n_f_odmr_fine = 50
common.he_tau = 200e-9
assume_double_nv = True

qmeas_list = []

#qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=200)
qexp =  setup_podmr_params(f_res=f_center, p_scale=1, f_step=3000e3, n_f=70)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_coarse'
qexp['measurement_time'] = 6*60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
if assume_double_nv:
    qexp['fit_experiment'] = 'lor_dip_2x'
    qexp['fit_parameter'] = 'Position 0'
#qexp['fit_experiment'] = 'lor_dip'
#qexp['fit_parameter'] = 'Position 0'

qexp['update_parameters'] = {'rabi': 'microwave_frequency',
                             #'Hahn_Echo': 'microwave_frequency',
                             #'Ramsey': 'microwave_frequency',
                             #'podmr_fine': {'target_name': 'freq_start',
                             #               'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"
                             #              }
                            }
if assume_double_nv:
    qexp['update_parameters'] = {'rabi': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 0']['value']"},
                                 'rabi_B': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 1']['value']"},
                                 'deer_f': {'target_name': 'microwave_frequency',
                                         'func': f"fit_result.result_str_dict['Position 0']['value']"},
                                 'podmr_fine': {'target_name': 'freq_start',
                                          'func': f"fit_result.result_str_dict['Position 0']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"},
                                 'podmr_fine_B': {'target_name': 'freq_start',
                                  'func': f"fit_result.result_str_dict['Position 1']['value'] - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"}
                                }
qmeas_list.append(copy.deepcopy(qexp))
#"""

#Experiment2
#"""
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['name'] = 'rabi'
qexp['measurement_time'] = 2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                             #'Hahn_Echo': 'rabi_period',
                             'podmr_fine': 'rabi_period',
                             'deer_f':'rabi_period',
                             'deer_t':'rabi_period'}  

qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_B'
    qexp['measurement_time'] = 2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'

    qexp['update_parameters'] = {#'Ramsey_B': 'rabi_period',
                                 #'Hahn_Echo_B': 'rabi_period',
                                 'podmr_fine_B': 'rabi_period',
                                 }
    
    qmeas_list.append(copy.deepcopy(qexp))

# next Exp
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp =  setup_podmr_params(f_res=f_center, p_scale=2, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
qexp['type'] = 'pulsedodmr'
qexp['name'] = 'podmr_fine'
qexp['measurement_time'] = 2*60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'rabi_fine': 'microwave_frequency',
                             'hahn_echo': 'microwave_frequency',
                             'xy8_n': 'microwave_frequency',
                             'deer_f':'microwave_frequency',
                             'deer_t':'microwave_frequency'
                              }
qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
    qexp =  setup_podmr_params(f_res=f_center, p_scale=2, f_step=f_step_odmr_fine, n_f=n_f_odmr_fine)
    qexp['type'] = 'pulsedodmr'
    qexp['name'] = 'podmr_fine_B'
    qexp['measurement_time'] = 2*60
    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None
    qexp['generate_new'] = True
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information
    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'lor_dip'
    qexp['fit_parameter'] = 'Position'
    qexp['update_parameters'] = {'rabi_fine_B': 'microwave_frequency',
                                 'hahn_echo_B': 'microwave_frequency',
                                 'xy8_n_B': 'microwave_frequency',
                                 'deer_t':'mw_freq2'
                                  }
    qmeas_list.append(copy.deepcopy(qexp))

    
# next Exp
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
qexp['type'] = 'rabi'
qexp['name'] = 'rabi_fine'
qexp['measurement_time'] = 2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                             'hahn_echo': 'rabi_period',
                             'xy8_n': 'rabi_period',
                             'deer_f':'rabi_period',
                             'deer_t': 'rabi_period'
                            }
qmeas_list.append(copy.deepcopy(qexp))

if assume_double_nv:
    qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50, alternating=False, n_tau=100)
    qexp['type'] = 'rabi'
    qexp['name'] = 'rabi_fine_B'
    qexp['measurement_time'] = 2*60
    qexp['meas_type'] = conventional_measurement
    qexp['meas_info'] = add_conventional_information

    qexp['optimize_time'] = None
    qexp['freq_optimize_time'] = None

    # if empty  or commented out, no fit is used
    qexp['fit_experiment'] = 'sine'
    qexp['fit_parameter'] = 'Period'
    qexp['update_parameters'] = {#'Ramsey': 'rabi_period',
                                 'hahn_echo_B': 'rabi_period',
                                 'xy8_n_B': 'rabi_period',
                                 'deer_f': 'rabi_period2',
                                 'deer_t': 'rabi_period2'
                            }
    qmeas_list.append(copy.deepcopy(qexp))

# next Exp
#

qexp = setup_deer_dd_f_params(f_center, dd_tau=common.he_tau, f_step=3e6, n_f=70,
                               t_rabi_2=None, dd_type=DDMethods.XY8, dd_order=30)
qexp['type'] = 'DEER_DD_freq'
qexp['name'] = 'deer_f'
qexp['measurement_time'] = 25*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

qexp['fit_experiment'] = 'lor_dip_2x'
#qmeas_list.append(copy.deepcopy(qexp))


t_rabi_2 = 196.5e-9
f_deer = 1500e6
# range of tau2: 0 to (tau/2-t_pi_1/2-t_pi_2/2)
# highest g= n_pi*1/tau_2
# lowest  g= n_pi*n_tau/tau_2
# T2= 8*order*he_tau
qexp = setup_deer_dd_tau_params(f_deer, dd_tau=common.he_tau, tau2_step=-2e-9, n_tau=50,
                                t_rabi_2=t_rabi_2, dd_type=DDMethods.XY8, dd_order=45)
qexp['type'] = 'DEER_DD_tau'
qexp['name'] = 'deer_t'
qexp['measurement_time'] = 15*60  #15
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qmeas_list.append(copy.deepcopy(qexp))
    
    
laser_off()
try:
    #run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=False, poi_list=pois)
    run_generic_autoexp(qmeas_list,  generate_new=True, optimize_between_exps=True,
                        opt_on_poi=opt_on_poi,  # reference for sample shift (on stable, unique NV)
                        poi_list=pois)
    #pass
except Exception as e:
    stack_trace = traceback.format_exc()
    logger.exception("auto run failed: {}.\n".format(stack_trace))




In [176]:
qexp

OrderedDict([('experiment', 'DEER_DD_tau'), ('name', 'deer_dd_tau'), ('microwave_frequency', 2765000000.0), ('microwave_amplitude', 0.15), ('rabi_period', 1.3676809838312765e-07), ('mw_freq2', 1500000000.0), ('num_of_points', 50), ('tau2_start', 0.0), ('tau2_incr', 3e-09), ('measurement_time', 150000), ('optimize_time', None), ('alternating', True), ('tau', 2e-07), ('rabi_period2', 1.3158247448460443e-07), ('mw_amp2', 0.15), ('tau2_rel_to_pi1', True), ('DD_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('order', 10), ('env_type', <EnvelopeMethods.parabola: 'SinEnvelopeParabolaN'>), ('order_P', 1), ('savetag', 'deer_dd_tau'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('freq_optimize_time', None), ('analysis_interval', None)])

In [26]:
common.he_tau = 0.2e-6
t_rabi_2 = 10e-9
qexp = setup_deer_dd_tau_params(1e9, dd_tau=common.he_tau, tau2_step=-5e-9, n_tau=50,
                                t_rabi_2=t_rabi_2, dd_type=DDMethods.XY8, dd_order=10)
qexp['rabi_period'] = 1.3676809838312765e-07
qexp['rabi_period2'] = 1.3158247448460443e-07

uglobals.abort.clear()
run_generic(qexp)

In [68]:
qexp

OrderedDict([('experiment', 'DEER_DD_tau'), ('name', 'deer_dd_tau'), ('microwave_frequency', 2765000000.0), ('microwave_amplitude', 0.15), ('rabi_period', 1.3676809838312765e-07), ('mw_freq2', 1000000000.0), ('num_of_points', 50), ('tau2_start', 0.0), ('tau2_incr', 6e-10), ('measurement_time', 150000), ('optimize_time', None), ('alternating', True), ('tau', 2e-07), ('rabi_period2', 1.3158247448460443e-07), ('mw_amp2', 0.15), ('tau2_rel_to_pi1', True), ('DD_type', <DDMethods.XY8: [0.0, 90.0, 0.0, 90.0, 90.0, 0.0, 90.0, 0.0]>), ('order', 10), ('env_type', <EnvelopeMethods.parabola: 'SinEnvelopeParabolaN'>), ('order_P', 1), ('savetag', 'deer_dd_tau'), ('gated', False), ('sequence_mode', False), ('ctr_single_sweeps', False), ('ctr_n_sweeps', 0), ('ctr_n_cycles', 0), ('freq_optimize_time', None), ('analysis_interval', None), ('sampling_freq', 8000000000.0), ('bin_width', 4e-09), ('wait_time', 1e-06), ('laser_delay', 2e-07), ('laser_safety', 2e-07), ('sync_channel', 'd_ch1'), ('gate_channel'

## OLD auto logic

In [43]:
# In order to stop the automized measurement insert pulsedmasterlogic.break_variable = True into manager

uglobals.abort.clear()
qmeas=OrderedDict()
# Either specify a list with points of interest or take all poi from the current ROI (use empty list)
#qmeas['list_pois'] = []
qmeas['list_pois'] = poimanagerlogic.poi_names[10:]
qmeas['generate_new'] = True
qmeas['no_optimize'] = True
logger.info("Autopilot is taking of for pois {}".format(qmeas['list_pois']))
# Create another dictionary for the experiments and parameters
autoexp=OrderedDict()
# Then specify all necessary experiment and parameters


# optional parameters 
qmeas['optimize_between_experiments'] = False


#Experiment1
# need to specify for the automatic update
f_step_odmr_fine=100e3
n_f_odmr_fine=100

#"""
experiment1 = 'pODMR'
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=1, f_step=3000e3, n_f=150)

qexp['type'] = 'pulsedodmr'
qexp['measurement_time'] = 60
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_dip'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'Rabi': 'microwave_frequency',
                             'Hahn_Echo': 'microwave_frequency',
                             'Ramsey': 'microwave_frequency',
                             'pODMR_fine': {'target_name': 'freq_start',
                                            'func': f"_x_ - 0.5*({f_step_odmr_fine}*{n_f_odmr_fine})"}
                            }
autoexp[experiment1] = copy.deepcopy(qexp)

#"""
#Experiment2
#"""

experiment2 = 'Rabi'
qexp = setup_rabi_params(tau_step=2.5*common.rabi_period/50)
qexp['type'] = 'rabi'
qexp['measurement_time'] = 60#2*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {'Ramsey': 'rabi_period',
                             'Hahn_Echo': 'rabi_period',
                             'pODMR_fine': 'rabi_period'}  # feed forware parameters

autoexp[experiment2] = copy.deepcopy(qexp)


#"""
#Experiment2
#"""
"""
experiment3 = 'Hahn_Echo'
#qexp = setup_hahn_params(tau_start=10e-9, n_tau=50, tau_step=0.2e-6, alternating=True)
#
#qexp = setup_hahn_params(tau_start=10e-9, n_tau=150, tau_step=125e-9, alternating=True)
qexp = setup_hahn_params(tau_start=10e-9, n_tau=75, tau_step=500e-9, alternating=True)
qexp['type'] = 'hahnecho'
qexp['measurement_time'] = 10*60
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information

qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None

# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'decay'
qexp['fit_parameter'] = 'Lifetime'

autoexp[experiment3] = copy.deepcopy(qexp)
"""



#Experiment4
#"""
experiment4 = 'Ramsey'
qexp = setup_ramsey_params(tau_start=50e-9, n_tau=200, tau_step=10e-9)  # res= 50 kHz 
qexp['type'] = 'ramsey'
qexp['measurement_time'] = 240
qexp['alternating'] = True
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
qexp['fit_experiment'] = 'sine_2x'
autoexp[experiment4] = copy.deepcopy(qexp)

#"""

"""
experiment3 = 'pODMR_fine'
qexp =  setup_podmr_params(f_res=common.mw_freq, p_scale=15, f_step=100e3, n_f=100)

qexp['type'] = 'pulsedodmr'
qexp['measurement_time'] = 15
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'lor_n15'
qexp['fit_parameter'] = 'Position'
qexp['update_parameters'] = {'Rabi': 'microwave_frequency',
                             'pODMR_fine':'microwave_frequency',
                             'xy8': 'microwave_frequency',
                             'xy8_2': 'microwave_frequency'}

autoexp[experiment3] = copy.deepcopy(qexp)
"""
# create another dictionary for the parameters
"""
autoexp[experiment1]=OrderedDict()
autoexp[experiment1]['type'] = 'pulsed_odmr'
autoexp[experiment1]['name'] = experiment1
autoexp[experiment1]['rabi_period'] = common.rabi_period
autoexp[experiment1]['freq_start'] = common.mw_freq -25e6
autoexp[experiment1]['freq_step'] = 1e6
autoexp[experiment1]['num_of_points'] = 50
autoexp[experiment1]['microwave_amplitude'] = common.mw_amp
autoexp[experiment1]['measurement_time'] = 10
autoexp[experiment1]['optimize_time'] = None
autoexp[experiment1]['freq_optimize_time'] = None
autoexp[experiment1]['generate_new'] = False
autoexp[experiment1]['meas_type'] = conventional_measurement
autoexp[experiment1]['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
autoexp[experiment1]['fit_experiment'] = 'lor_neg'
autoexp[experiment1]['fit_parameter'] = 'Position'
autoexp[experiment1]['update_parameters'] = {'Rabi': 'microwave_frequency', 'xy8': 'microwave_frequency', 'xy8_2': 'microwave_frequency'}
"""

# broken to add experiments in loop
#autoexp[experiment1] = copy.deepcopy(qexp)
#"""

#Experiment2
"""
experiment2 = 'Rabi'
qexp = setup_rabi_params(tau_step=10e-9)
qexp['type'] = 'rabi'
qexp['measurement_time'] = 30
qexp['optimize_time'] = None
qexp['freq_optimize_time'] = None
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
# if empty  or commented out, no fit is used
qexp['fit_experiment'] = 'sine'
qexp['fit_parameter'] = 'Period'
qexp['update_parameters'] = {'ramsey': 'rabi_period'}  # feed forware parameters
autoexp[experiment2] = copy.deepcopy(qexp)

#Experiment3
experiment3 = 'ramsey'
qexp = setup_ramsey_params(tau_start=50e-9, n_tau=400, tau_step=50e-9)  # res= 50 kHz 
qexp['type'] = 'ramsey'
qexp['measurement_time'] = 120
qexp['generate_new'] = True
qexp['meas_type'] = conventional_measurement
qexp['meas_info'] = add_conventional_information
qexp['fit_experiment'] = 'double_sine_decay'
autoexp[experiment3] = copy.deepcopy(qexp)


"""

laser_off()
do_automized_measurements(qm_dict=qmeas, autoexp=autoexp)




In [320]:
poimanagerlogic.poi_names[-1]

'poi_20210720135257025577'

In [434]:
qmeas

OrderedDict([('name', 'hahnecho'), ('rabi_period', 5.35e-08), ('microwave_frequency', 2776000000.0), ('microwave_amplitude', 0.25), ('tau_start', 1e-08), ('tau_step', 2e-07), ('num_of_points', 50), ('alternating', True), ('optimize_time', None), ('fit_experiment', 'exp')])